In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time 

from collections import Counter
from numpy import nan as NA
from dateutil.parser import parse
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="huangxq",
          password="f_epz6hDmZ0Hzi0OzI0N",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 1.样本数据整理

## 1_11.获取_租后数据

In [4]:
sql = '''-- 租后应收监控  
SELECT  tprm.* 
,om.`status` as 订单状态值
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as  订单状态
,om.order_finish_date as 订单完成时间,om.order_method
,tmu.true_name ,tmu.id_card_num,tmu.mobile 
,tod.product_name,tod.sku_attributes
from db_digua_business.t_postlease_receivables_monitoring   tprm 
left join db_digua_business.t_order om on tprm.order_id = om.id
left join  db_digua_business.t_member_user tmu on om.user_id = tmu.id
left join  db_digua_business.t_order_details tod on tprm.order_id = tod.order_id
;
'''

df_zhys = query(sql)
# (27787, 62)
df_zhys.shape

(34119, 64)

## 1_12. 剔除商家数据

In [5]:
# df_zhys["年"] = df_zhys["下单月份"].str[:4]
# pd.crosstab(dfzh1["商家名称"],dfzh1["年"],margins=True).loc[["北京海鸟窝科技有限公司","深圳优优大数据科技有限公司","租着用电脑数码","苏州蚁诺宝"],:]
# 剔除商家数据
def drop_sj(df_zhys,col1,col2):
    df_zhys.drop(df_zhys[df_zhys[col1]=="深圳优优大数据科技有限公司"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="优优2店"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="小豚租（代收）"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="苏州蚁诺宝"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="租着用电脑数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="北京海鸟窝科技有限公司"].index,inplace=True)

    df_zhys.drop(df_zhys[df_zhys[col1]=="汇客好租"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="澄心优租"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="CPS渠道合作"].index,inplace=True)
    
    df_zhys.drop(df_zhys[df_zhys[col1]=="趣智数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="格木木二奢名品"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="广州康基贸易有限公司"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="线下小店"].index,inplace=True)  # 计划
    df_zhys.drop(df_zhys[df_zhys[col1]=="小蚂蚁租机"].index,inplace=True)  #拒量订单
   

    df_zhys.drop(df_zhys[df_zhys[col2].str.contains(pat='探路者',regex=False)==True].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys['merchant_name'] == "乙辉数码"].index, inplace=True)
    return df_zhys

df_zhys = drop_sj(df_zhys,'merchant_name','model_number')
df_zhys.shape

(32002, 64)

## 1_13.历史订单渠道名称补充

In [6]:
# 1148 条历史订单渠道名称补充
qudao_name_df = pd.read_excel('F:/myfile/租后数据/历史订单渠道名称补充.xlsx')
qudao_name_df.drop(columns=['渠道id','下单时间'],inplace = True)

# dfzh1 = df_zh.merge(qudao_name_df,on = '订单号',how = 'left')
dfzh1 =  df_zhys.merge(qudao_name_df,left_on = 'order_number',right_on = '订单号',how = 'left')
# dfzh1.columns

# qudao_name_df.shape
# qudao_name_df[:2]
# qudao_name_df['渠道名称'].value_counts()

# 渠道名称缺失补充
# dfzh1["渠道名称"] = dfzh1["渠道名称_x"].fillna(dfzh1.渠道名称_y)
dfzh1["channel_name"] = np.where(dfzh1.渠道名称.notnull(),dfzh1.渠道名称,dfzh1["channel_name"])
dfzh1["channel_name"].isnull().sum()

# dfzh1[['order_number','channel_name','渠道名称']][:10]
# df_zhys[df_zhys['order_number'] == 'A202207131230485'][['order_number','channel_name']]

0

## 1_14.渠道归属

In [7]:
def qudao_type(a,b,c):
    a = str(a)
    b = str(b)
    if "租物" in b:
        return "芝麻租物"
    elif "芝麻" in b:
        return "芝麻租物"
    elif "抖音" in b:
        return "抖音渠道"
    elif "搜索" in a:
        return "搜索渠道"
    elif "租物" in a:
        return "芝麻租物"
    elif "芝麻" in a:
        return "芝麻租物"
    elif c == 1:
        return "芝麻租物" 
    elif "叮咚直播" in a:
        return "叮咚直播"
    elif "八派信息" in a:
        return "八派信息"
    elif "租瓜直播2号" in a:
        return "租瓜直播2号"
    elif "租瓜直播" in a:
        return "租瓜直播"
    elif "直播" in a:
        return "小舞直播"
    elif "繁星" in a:
        return "繁星"
    elif "生活号" in a:
        return "生活号"
    elif "社群" in a:
        return "支付宝社群"
    elif "付费灯火" in a:
        return "付费灯火"
    else :
        return a

In [8]:
# df.loc[:,"归属渠道"]=df.apply(lambda x:qudao_type(x["来源渠道"],x["activity_name"],x["order_method"]),axis=1)
dfzh1["activity_source"] = dfzh1["activity_source"].fillna('未知活动') 
dfzh1["channel_name"] = dfzh1["channel_name"].fillna('未知渠道') 

dfzh1.loc[:,"归属渠道"]=dfzh1.apply(lambda x:qudao_type(x["channel_name"],x["activity_source"],x["order_method"]),axis=1)

# pd.crosstab(dfzh1["下单月份"],dfzh1["归属渠道"],margins=True)

## 1_15.判断颜色 内存

In [9]:
dfzh1["内存"]=dfzh1["specification"].str.split("内存：").str[1].str.split("颜色：").str[0].str.split(" ").str[0]
# dfzh1["内存"].value_counts()[14:]

dfzh1["颜色"]=dfzh1["specification"].str.split("内存：").str[1].str.split("颜色：").str[1].str.split(" ").str[0]
# dfzh1["颜色"].value_counts()[1:10]

## 1_16.判断二手、带锁

In [10]:
# dfzh1['是否二手'] = np.where(dfzh1['model_number'].str.contains('95新')|dfzh1['model_number'].str.contains('99新')|dfzh1['model_number'].str.contains('准新'),"二手","全新")
# note 20241124 修改口径，用product_name判断是否二手
dfzh1['是否二手'] = np.where(dfzh1['product_name'].str.contains('95新')|dfzh1['product_name'].str.contains('99新')|dfzh1['product_name'].str.contains('准新'),"二手","全新")
dfzh1['是否二手'].value_counts()

是否二手
全新    31170
二手      832
Name: count, dtype: int64

In [11]:
#读取导入最新出库台账数据,2023.9及以后的数据
# （1）512 带锁
# （2）非512的多头带锁
# （3）抖音带锁
# note 20241204
# （1）512 带锁，但每天豁免一台 512P 不带锁，目前还没量
# （2）非512的命中多头提示带锁，但出库是否带锁由人工控制
# （3）抖音带锁（苹果11-15）
# 维客壹佰2023&2024年台账1204.xlsz 为截止到20241203的数据
# f_path_ck = "维客壹佰2023&2024年台账1204.xlsx"
f_path_ck = "维客壹佰2025年台账.xlsx"
path = 'F:/myfile/p站数据/台账数据/'
dfck = pd.read_excel(path + f_path_ck,sheet_name="2023")   #,skiprows=0 header 

# dfck['是否带锁']  = np.where((dfck['型号'].str.contains(r'带锁|有锁'))|(dfck['内存']=='512G')|(dfck['内存']=='512GB'), 1, 0)
dfck['是否带锁']  = np.where((dfck['型号'].str.contains(r'带锁|有锁'))|(dfck['内存'].str.contains(r'512')), '带锁', '未带锁')
dfck = dfck[[u'订单号',u'是否带锁']]
dfck.shape
dfck['是否带锁'].value_counts()

dfck.drop_duplicates(subset=["订单号"],inplace=True)

# 合并到主表中去
# dfck 的带锁不一定出库，dfzh1 是出库的
dfck = dfck.rename(columns= {'订单号': 'order_number'})
dfzh1 = dfzh1.merge(dfck, on = 'order_number', how = 'left')
dfzh1['是否带锁'] = dfzh1['是否带锁'].fillna('未带锁') 
dfzh1.shape

pd.pivot_table(dfzh1,index='是否二手',columns='是否带锁',values='order_id',aggfunc='count')

(27824, 2)

是否带锁
未带锁    25728
带锁      2096
Name: count, dtype: int64

(32002, 71)

是否带锁,带锁,未带锁
是否二手,,
二手,174,658
全新,1602,29568


## 1_17.日期处理

In [12]:
dfzh1["下单日期"] = dfzh1['order_create_time'].dt.date
dfzh1["下单日期"] = pd.to_datetime(dfzh1["下单日期"],errors="coerce")
dfzh1["下单日期"].head()

dfzh1["月份"]=dfzh1["下单日期"].dt.month
dfzh1["年份"]=dfzh1["下单日期"].dt.year
# df_total["下单月份"].value_counts()

0   2022-07-05
1   2022-07-13
2   2022-07-15
3   2022-07-16
4   2022-07-17
Name: 下单日期, dtype: datetime64[ns]

In [13]:
def getjidu(s):
    if s<4:
        return "1季度"
    elif 4<=s<7:
        return "2季度"
    elif 7<=s<10:
        return "3季度"
    else:
        return "4季度"
dfzh1["季度"]= dfzh1.apply(lambda x:getjidu(x["月份"]),axis=1)  
dfzh1["季度"].value_counts()

季度
4季度    11213
3季度     9679
2季度     6164
1季度     4946
Name: count, dtype: int64

## 1_21.获取_买断数据

In [14]:
sql1 = ''' --   买断信息 
with out_order as ( 
SELECT too.order_id,too.`status` 
,too.real_pay_money ,too.create_time,too.update_time
,too.pay_date ,too.actual_money 
,rank() over(partition by too.order_id order by too.pay_date desc) as rn 
from db_digua_business.t_order_out too
where  too.`status` not in (1)   -- 买断状态：1 未买断 2 已买断  3 部分买断 
-- and too.pay_status= 2    -- '支付状态 1、待支付 2、已支付 3、退款中 4、已退款 6、支付取消 7、已关闭',
-- GROUP BY 1 ,2
-- ORDER BY 5 desc,1
)
# note 20241126 增加distinct，买断可能分多次付款
# 181950     2
# 338547     2
# 1083822    2
# 303756     2
SELECT distinct t1.*
,t2.`status`
,case when t2.`status`=2 then '已买断' when  t2.`status`=3 and om.order_finish_date is not null then '已买断'  else '部分买断' end as if_outpay 
,om.order_finish_date
,case when t2.`status`=2 and om.order_finish_date is null then t1.pay_date else om.order_finish_date end as finish_date_new 
from (
SELECT too.order_id  -- ,too.`status` 
-- ,count(DISTINCT too.`status`) nn 
,sum(too.real_pay_money) as outpay_money     -- ,max(too.update_time) as outpay_time
,max(too.pay_date) as pay_date 
from out_order  too
GROUP BY 1 -- ,2
ORDER BY 2 desc ,1) t1 
left join out_order t2 on t1.order_id = t2.order_id and t2.rn=1 
left join db_digua_business.t_order om on t1.order_id = om.id 
;
'''

df_out = query(sql1)
df_out.shape

(3095, 7)

## 1_22.租后数据 关联 买断数据

In [15]:
dfzh2 = dfzh1.merge(df_out[['order_id', 'outpay_money', 'pay_date', 'status', 'if_outpay']], on = 'order_id', how = 'left')
dfzh2.shape
dfzh1.shape

(32002, 79)

(32002, 75)

In [16]:
dfzh2['finish_date_new'] = dfzh2.apply(lambda x:pd.to_datetime(x.pay_date, errors='coerce') if pd.isnull(x.order_finish_date) and x.status==2
                                       else pd.to_datetime(x.订单完成时间, errors='coerce') if pd.isnull(x.order_finish_date)  and pd.notnull(x.订单完成时间)
                                       else pd.to_datetime(x.order_finish_date, errors='coerce'),axis = 1 ) 
# dfzh2['finish_date_new'] = np.where((pd.isnull(dfzh2.order_finish_date)) & (dfzh2.status==2),pd.to_datetime(dfzh2.pay_date, errors='coerce'),pd.to_datetime(dfzh2.order_finish_date, errors='coerce'))
dfzh2['finish_date_new'].notnull().sum()

4026

In [17]:
dfzh2['if_outpay'] = dfzh2.apply(lambda x: '已买断' if x['status'] ==2 
                                 else  '已买断'  if  x['status'] ==3 and pd.notnull(x.finish_date_new) 
                                 else '部分买断' if x['status'] ==3 and pd.isnull(x.finish_date_new) 
                                 else '未知',axis =1)
dfzh2['if_outpay'].value_counts()

if_outpay
未知      29015
已买断      2918
部分买断       69
Name: count, dtype: int64

## 1_31.获取_分期取消数据

In [18]:
sql1 = '''-- 订单分期表  取消的分期
SELECT tos.order_id,count(*) as nn
,count(tos.reality_refund_date) as  reality_refund_date
,sum(tos.real_pay_money) as real_pay_money
,sum(tprm.purchase_amount)/count(tos.order_id) as purchase_amount
,avg(tprm.advance_periods) as  advance_periods
,avg(tprm.advance_sum) as advance_sum
from db_digua_business.t_order_stages tos 
inner join db_digua_business.t_postlease_receivables_monitoring   tprm 
on tos.order_id = tprm.order_id 
# 订单取消
where tos.`status`=4 
GROUP BY 1 order by 2 desc 
;
'''

df_cancel = query(sql1)
df_cancel.shape

(1818, 7)

## 1_32.租后数据关联分期取消数据

In [19]:
dfzh3 = dfzh2.merge(df_cancel[['order_id', 'nn', 'reality_refund_date', 'real_pay_money']],on = 'order_id',how = 'left')
dfzh3.shape
dfzh2.shape

# 不同还款状态下  完成时间的情况
# dfzh3.groupby(['rembursement_status']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count','finish_date_new':'count','pay_date':'count','nn':'count','outpay_money':'sum'})

# 不同订单状态下  完成时间的情况
# dfzh3.groupby(['订单状态']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count','finish_date_new':'count','pay_date':'count','nn':'count','outpay_money':'sum'})

# 有取消记录的用户分布情况
# dfzh3[dfzh3.nn>=1].groupby(['rembursement_status']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count'
                                                        #  ,'finish_date_new':'count','pay_date':'count','nn':'count','real_pay_money':'sum'})

# 有取消记录的用户分布情况
# dfzh3[dfzh3.nn>=1].groupby(['订单状态']).agg({'order_id':'count','order_finish_date':'count','订单完成时间':'count'
                                                        #  ,'finish_date_new':'count','pay_date':'count','nn':'count','real_pay_money':'sum'})

(32002, 83)

(32002, 80)

## 1_41.按风控策略调整时间节点划分时间段

In [20]:
#按风控策略调整时间节点划分时间段
dfzh1.loc[:"2023-01-09","时间段"]=" -2023.1.9前"
dfzh1.loc["2023-01-10":"2023-03-03","时间段"]="2023.1.10-2023.3.3"
dfzh1.loc["2023-03-04":"2023-03-24","时间段"]="2023.3.4-2023.3.24"
dfzh1.loc["2023-03-25":"2023-04-28","时间段"]="2023.3.25-2023.4.28"
dfzh1.loc["2023-04-29":"2023-05-17","时间段"]="2023.4.29-2023.5.17"
dfzh1.loc["2023-05-18":"2023-05-31","时间段"]="2023.5.18-2023.5.31"
dfzh1.loc["2023-06-01":"2023-06-30","时间段"]="2023.6.1-2023.6.30"
dfzh1.loc["2023-07-01":"2023-08-11","时间段"]="2023.7.1-2023.8.11"
dfzh1.loc["2023-08-12":"2023-09-15","时间段"]="2023.8.12-2023.9.15"
dfzh1.loc["2023-09-16":"2023-09-22","时间段"]="2023.9.16-2023.9.22"
dfzh1.loc["2023-09-23":"2023-11-16","时间段"]="2023.9.23-2023.11.16"
dfzh1.loc["2023-11-17":"2023-11-28","时间段"]="2023.11.17-2023.11.28"
dfzh1.loc["2023-11-29":"2023-12-05","时间段"]="2023.11.29-2023.12.5"
dfzh1.loc["2023-12-06":"2024-01-05","时间段"]="2023.12.06-2024.1.5"
dfzh1.loc["2024-01-06":"2024-01-23","时间段"]="2024.1.6-2024.1.23"
dfzh1.loc["2024-01-24":"2024-2-21","时间段"]="2024.1.24-2024.2.21"
dfzh1.loc["2024-02-22":"2024-02-29","时间段"]="2024.2.22-2024.2.29"
dfzh1.loc["2024-03-01":"2024-03-12","时间段"]="2024.3.1-2024.3.12"
dfzh1.loc["2024-03-13":"2024-04-03","时间段"]="2024.3.13-2024.4.3"
dfzh1.loc["2024-04-04":"2024-04-10","时间段"]="2024.4.4-2024.4.10"
dfzh1.loc["2024-04-11":"2024-04-18","时间段"]="2024.4.11-2024.4.18"
dfzh1.loc["2024-04-19":"2024-04-24","时间段"]="2024.4.19-2024.4.24"
dfzh1.loc["2024-04-25":"2024-04-27","时间段"]="2024.4.25-2024.4.27"
dfzh1.loc["2024-04-28":,"时间段"]="2024.4.28- "

## 1_51.获取_账期数据（含续租）

In [21]:
sql_xz = ''' 
-- 状态,1:未支付  2:扣款失败 3:已支付 4：已取消 5、申请扣款、6已退款',
-- '是否续租账期,0非续租账期，1预授权账期，2自定义续租账期',
SELECT ymos.*
from db_rent.ya_merchant_order_stages ymos 
inner join db_digua_business.t_postlease_receivables_monitoring tprm 
on ymos.order_id = tprm.order_id 
;
'''

df_xzfq = query(sql_xz)
df_xzfq.shape

(424286, 23)

In [22]:
# note 20241227 号卡 应付日期更新
# 分期表
sql = '''-- 租后应收监控  
SELECT distinct hk.order_number, tos.order_id, tos.refund_date as refund_date_1
from db_digua_business.t_order_stages tos
inner join
    (select distinct om.id as order_id, om.order_number
    from db_digua_business.t_order om
    left join db_digua_business.t_platform_activity tpa 
    on tpa.id=om.activity_id
    where tpa.type = 4) hk
on tos.order_id = hk.order_id
'''
tmp_hk = query(sql)
tmp_hk['refund_date_ym'] = tmp_hk['refund_date_1'].dt.strftime('%Y-%m')
tmp_hk.shape
tmp_hk[:2]

df_xzfq['refund_date_ym'] = df_xzfq['refund_date'].dt.strftime('%Y-%m')
df_xzfq = df_xzfq.merge(tmp_hk[['order_id', 'refund_date_1', 'refund_date_ym']], on = ['order_id', 'refund_date_ym'], how = 'left')
df_xzfq['refund_date_2'] = np.where(df_xzfq.refund_date_1.notnull(), df_xzfq.refund_date_1, df_xzfq.refund_date)
df_xzfq.shape

df_xzfq.drop(columns=['refund_date_1','refund_date_ym','refund_date'],inplace = True)
df_xzfq = df_xzfq.rename(columns = {'refund_date_2':'refund_date'})
df_xzfq.shape

(3180, 4)

,order_number,order_id,refund_date_1,refund_date_ym
0,A20240927154904302,1533488,2024-09-30,2024-09
1,A20240927154904302,1533488,2024-10-30,2024-10


(424286, 26)

(424286, 23)

In [23]:
# 数据检查
# tmp1 = df_xzfq.drop_duplicates(subset = ['order_id'])[['order_id']]
# tmp2 = tmp_hk.drop_duplicates(subset = ['order_id'])[['order_id', 'order_number']]
# tmp1.shape
# tmp2.shape

# tmp3 = tmp1.merge(tmp2, on = 'order_id', how = 'inner')
# tmp3.shape

# tmp4 = tmp3.merge(df_xzfq, on = 'order_id', how = 'inner')
# tmp4.shape

# tmp4["日"]=tmp4["refund_date"].dt.day
# tmp4["日"].value_counts()
# tmp4[tmp4.日.isin([2,11])][['order_number','refund_date']]

# df_xzfq[df_xzfq.refund_date_2 != df_xzfq.refund_date].shape
# df_xzfq[df_xzfq.refund_date_2 == df_xzfq.refund_date].shape
# df_xzfq[df_xzfq.refund_date_2 != df_xzfq.refund_date][['order_id', 'refund_date', 'refund_date_1', 'refund_date_2', 'refund_date_ym']]
# 以下四个订单的应付日期<15号，订单分期表更正了  正常的订单的应付日期
# A20240929094315174 1539249  已退款
# A2024092919033313  1541755  正常
# A2024092919182056  1541798  已退款
# A2024100810242488  1568606  已退款

# tmp_hk[tmp_hk.order_number.isin(['A2024100810242488'])]
# df_xzfq[df_xzfq.order_id.isin([1539249,1541755,1541798,1568606])][['order_id', 'refund_date', 'refund_date_1', 'refund_date_2', 'refund_date_ym']]

In [24]:
# df_xz_concat[df_xz_concat.order_number.isin(['A2024080919264964','A2024092919033313','A20241218184711224'])][['order_number','应付日期','实付日期','逾期天数']]
# A2024080919264964
# A2024092919033313
# tmp = df_xz_concat[df_xz_concat.逾期天数 > 30].drop_duplicates(subset=['order_number'])[['order_number']]
# tmp = tmp.merge(df_hk, on = 'order_number', how = 'inner')
# tmp.shape
# tmp

## 1_52.重命名

In [25]:
df_xzfq = df_xzfq.rename(columns={'money':'当前应付租金','part_payment':'实付金额','refund_date':'应付日期','reality_refund_date':'实付日期','sort':'当前期数'})

## 1_53.实还金额核查

In [26]:
# total_check = df_ddfq.merge(df_xzfq,on=['订单ID','当前期数'],how='inner')
# total_check.info()

# total_check1= total_check[(total_check.实付金额_y < total_check.实付金额_x)][['订单ID','订单号','应付日期_x','实付日期_x','实付日期_y','实付金额_x','实付金额_y','状态','status']]    #   #(total_check.应付日期_x.notnull()) & (total_check.应付日期_y.isnull())
# total_check1= total_check[(total_check.实付金额 < total_check.实际支付金额)][['订单ID','订单号','还款日期','实际还款日期','实付日期','实际支付金额','实付金额','状态','status']]
# total_check1

# total_check2= total_check[(total_check.实付金额_y > total_check.实付金额_x)][['订单ID','订单号','应付日期_x','实付日期_x','实付日期_y','实付金额_x','实付金额_y','状态','status']]    #   #(total_check.应付日期_x.notnull()) & (total_check.应付日期_y.isnull())
# total_check2= total_check[(total_check.实付金额 > total_check.实际支付金额)][['订单ID','订单号','还款日期','实际还款日期','实付日期','实际支付金额','实付金额','状态','status']]
# total_check2

# total_check2.to_excel('E:/myfile/业务相关数据/total_check_0709-1.xlsx')

## 1_61.数据合并

In [27]:
df_xz_concat = df_xzfq.merge(dfzh3,how='right',on ='order_id')
# df_xz_concat = df_fq.merge(dfzh1,how='right',right_on ='订单id',left_on='订单ID')
# note 20241126 增加
# df_xz_concat[df_xz_concat.当前应付租金.isnull()].shape
# df_xz_concat[df_xz_concat.当前应付租金.isna()].shape
# 实付日期为空，但当前应付租金为空
# 1681669 A20241104144812228
df_xz_concat.实付金额 = df_xz_concat.实付金额 + df_xz_concat.sesame_promo_money_pay + df_xz_concat.promo_money
df_xz_concat = df_xz_concat[df_xz_concat.当前应付租金.notnull()]

# order_id == 1681669 的当前期数为空
# 1681669	A20241104144812228
# 1669581	A2024110109350797
df_xz_concat['当前期数'].notnull().sum()
df_xz_concat.shape
df_xzfq.shape
dfzh3.shape

# df_xz_concat.drop_duplicates(subset=["order_id"]).shape
# dfzh3.drop_duplicates(subset=["order_id"]).shape

# df_xz_concat['order_id'].nunique()
# dfzh3['order_id'].nunique()
# df_xz_concat[["order_id","当前期数"]].info()

424171

(424171, 105)

(424286, 23)

(32002, 83)

## 1_62.去重

In [28]:
check = df_xz_concat[["order_id","当前期数"]]  # ,"逾期天数"
all_duplicates = check[check.duplicated(keep=False)]  
# all_duplicates.to_excel('E:/myfile/业务相关数据/all_duplicates.xlsx')
print(all_duplicates.sort_values(by=['order_id','当前期数']))

        order_id  当前期数
146240    869632   1.0
146252    869632   1.0
422488    869632   1.0
422500    869632   1.0
146241    869632   2.0
...          ...   ...
416495    871154  11.0
146771    871154  12.0
146783    871154  12.0
416484    871154  12.0
416496    871154  12.0

[144 rows x 2 columns]


In [29]:
#删除重复订单
df_xz_concat.drop_duplicates(subset=["order_id","当前期数"],inplace=True)
df_xz_concat.shape

(424063, 105)

In [30]:
# 119321
# 1083822 303756
# all_duplicates[all_duplicates['order_id'] == 181950].shape
# df_xz_concat[df_xz_concat['order_id'] == 181950].shape

# df_xz_concat.to_excel('E:/myfile/业务相关数据/df_xz_concat.xlsx')

# 3.计算逾期天数

## 3_11.异常查询

In [31]:
# df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.status_x==1)][['order_number', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new']]

# 实付金额>0 实付日期为空 finish_date_new非空 :实付日期=订单完成时间
# df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new']]

# part_payment==0 reality_refund_date.notnull() 还款状态== '续租中' part_payment=money 还款状态== '已逾期' reality_refund_date=pd.na
# dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test

# dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test

# 实付为0，实付日期非空，完成时间为空，无期数取消：实付日期清空
# dd_test = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test.订单状态值.value_counts()

# 实付日期为空，完成时间为空，取消期数>=1 ：状态=已取消
# dd_test = df_xz_concat[ (df_xz_concat.实付日期.isnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn>=1)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test.订单状态值.value_counts()

# 部分还款，实付日期非空，完成时间为空 用户： 实付日期清空
# dd_test1 = df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test1

# 部分还款，实付日期为空，完成时间非空 用户： 实付日期=完成时间
# dd_test1 = df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn']]
# dd_test1.订单状态值.value_counts()

# 已买断 但无实付日期及金额：实付日回写
# dd_test1 = df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.isnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]
# # dd_test1.订单状态值.value_counts()
# dd_test1

## 3_12.实还日期修正

In [32]:
df_xz_concat['实付日期'] = pd.to_datetime(df_xz_concat['实付日期']).dt.date
df_xz_concat['finish_date_new'] = pd.to_datetime(df_xz_concat['finish_date_new']).dt.date

In [33]:
# （1）部分还款 用户 实付日期非空 且非买断 非完成 ： 实付日期清空  
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull()),pd.NaT,df_xz_concat.实付日期)

df_xz_concat[(df_xz_concat.实付金额>0)&(df_xz_concat.实付金额<df_xz_concat.当前应付租金) & (df_xz_concat.实付日期.notnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']].tail()     

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
330139,A2024110123471583,1672926,499.85,499.85,2025-02-04,2025-02-01,3.0,4.0,还款中,NaT,4,NaT,NaN,NaT
333762,A20241106135426228,1689980,499.85,499.85,2024-12-09,2024-12-08,3.0,2.0,还款中,NaT,4,NaT,NaN,NaT
333763,A20241106135426228,1689980,499.85,499.85,2025-01-09,2025-01-05,3.0,3.0,还款中,NaT,4,NaT,NaN,NaT
334327,A2024110708472957,1692436,457.98,457.98,2025-01-10,2025-01-08,3.0,3.0,还款中,NaT,4,NaT,NaN,NaT
334328,A2024110708472957,1692436,457.98,457.98,2025-02-10,2025-02-08,3.0,4.0,还款中,NaT,4,NaT,NaN,NaT


In [34]:
# （2）已完成 还款 用户  实付日期 补充 订单完成时间       实付金额>0  实付日期为空  finish_date_new非空  :实付日期=订单完成时间finish_date_new
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull()),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)

df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']].tail()       

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
333763,A20241106135426228,1689980,499.85,499.85,2025-01-09,2025-01-05,3.0,3.0,还款中,NaT,4,NaT,NaN,NaT
334327,A2024110708472957,1692436,457.98,457.98,2025-01-10,2025-01-08,3.0,3.0,还款中,NaT,4,NaT,NaN,NaT
334328,A2024110708472957,1692436,457.98,457.98,2025-02-10,2025-02-08,3.0,4.0,还款中,NaT,4,NaT,NaN,NaT
371409,A20241210113506313,1839860,695.21,665.65,2025-01-13,NaT,1.0,2.0,还款中,NaT,4,NaT,NaN,NaT
371410,A20241210113506313,1839860,695.21,665.65,2025-02-13,NaT,1.0,3.0,还款中,NaT,4,NaT,NaN,NaT


In [35]:
# （3）租完即送用户 A2023081019051714 补还款时间 
df_xz_concat['实付日期1'] = np.where((df_xz_concat.order_number=='A2023081019051714') & (df_xz_concat.当前期数==12),'2024-07-22',df_xz_concat['实付日期1'])
df_xz_concat[(df_xz_concat.order_number=='A2023081019051714') & (df_xz_concat.当前期数==12)][['实付日期1']]

,实付日期1
60964,2024-07-22


In [36]:
# （4）未还款 且 未完成 未取消用户 ： 实付日期清空
df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull()),pd.NaT,df_xz_concat.实付日期1)

df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期.notnull())& (df_xz_concat.finish_date_new.isnull())& (df_xz_concat.nn.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']].tail()

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
8862,A2023021015233353,125911,633.21,0.0,2023-03-13,2023-03-18,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
9558,A2023021414250354,132517,334.26,0.0,2023-03-17,2023-03-18,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
9636,A2023021504342232,133713,396.66,0.0,2023-03-18,2023-03-18,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
10650,A2023022114312437,145022,556.04,0.0,2023-03-24,2023-03-20,2.0,2.0,已逾期,NaT,5,NaT,NaN,NaT
12255,A2023030700325746,166163,288.77,0.0,2024-01-10,2023-04-17,1.0,11.0,已逾期,NaT,5,NaT,NaN,NaT


In [37]:
# （5）实付为0，实付日期为空，但 完成时间非空，买断金额>0： 实付日期回写

df_xz_concat['实付日期1'] = np.where((df_xz_concat.实付金额==0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)

df_xz_concat[(df_xz_concat.实付金额==0) & (df_xz_concat.实付日期1.notnull()) & (df_xz_concat.finish_date_new.notnull())& (df_xz_concat.outpay_money>0)][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','实付日期1']] .tail()

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,实付日期1
181250,A2024041316125534,995895,1061.16,0.0,2024-12-16,NaT,1.0,9.0,已买断,2024-05-17 15:02:17,8,2024-05-17,10.0,2024-05-17
181251,A2024041316125534,995895,1061.16,0.0,2025-01-16,NaT,1.0,10.0,已买断,2024-05-17 15:02:17,8,2024-05-17,10.0,2024-05-17
181252,A2024041316125534,995895,1061.16,0.0,2025-02-16,NaT,1.0,11.0,已买断,2024-05-17 15:02:17,8,2024-05-17,10.0,2024-05-17
181253,A2024041316125534,995895,1063.94,0.0,2025-03-16,NaT,1.0,12.0,已买断,2024-05-17 15:02:17,8,2024-05-17,10.0,2024-05-17
210816,A2024060716074819,1111255,1085.68,0.0,2025-05-10,NaT,1.0,12.0,已买断,2024-12-14 14:59:21,8,2024-12-14,7.0,2024-12-14


In [38]:
# （6）部分还款 
# df_xz_concat[(df_xz_concat.实付金额>0) & (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

In [39]:
# （7）未完成 但含有分期取消的 订单
# check = df_xz_concat[(df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.isnull()) & df_xz_concat.nn>=1][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]
# check.订单状态值.value_counts()

In [40]:
# （8）实付0，实付日期为空，但 完成时间非空：为取消订单，实付日期=完成时间
df_xz_concat['实付日期new'] = np.where( (df_xz_concat.实付日期1.isnull()) & (df_xz_concat.finish_date_new.notnull()),df_xz_concat.finish_date_new,df_xz_concat.实付日期1)
df_xz_concat[ (df_xz_concat.实付日期new.isnull()) & (df_xz_concat.finish_date_new.notnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期1'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期1,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,outpay_money


In [41]:
# （9）属于部分还款 :无实付日期
# dd_test2 = df_xz_concat[(df_xz_concat.实付金额 >0) & (df_xz_concat.实付日期new.isnull())][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]
# dd_test2

In [42]:
# （10）df_xz_concat[df_xz_concat.应付日期.isnull()][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
# , 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

In [43]:
# （11）A202207050851373、A20241104144812228 应付日期修复
df_xz_concat['应付日期'] = np.where((df_xz_concat.应付日期.isnull()) & (df_xz_concat.order_number.isin(['A202207050851373','A20241104144812228'])),pd.to_datetime('2023-07-05') ,df_xz_concat.应付日期 )
df_xz_concat[df_xz_concat.应付日期.isnull()][['order_number','order_id', '当前应付租金', '实付金额','应付日期', '实付日期new'
, 'status_x', '当前期数','rembursement_status','订单完成时间','订单状态值','finish_date_new','nn','outpay_money']]

,order_number,order_id,当前应付租金,实付金额,应付日期,实付日期new,status_x,当前期数,rembursement_status,订单完成时间,订单状态值,finish_date_new,nn,outpay_money


In [44]:
# （12）实付金额>0  实付日期.isnull()  还款状态==已买断  :  逾期天数=0  实付日期=订单完成时间
# 实付金额>0  实付日期.isnull()  还款状态==已逾期  :实付金额 =0  
# 实付金额==0  实付日期.notnull()  还款状态== '续租中' 实付金额=当前应付租金   还款状态== '已逾期' 实付日期=pd.na
# 0<实付金额<当前应付租金  实付日期.notnull()  还款状态== '已逾期'  :  实付金额=0  & 实付日期=pd.na

# df_xz_concat['实付金额new'] = df_xz_concat.apply(lambda x:
#     0 if x.实付金额>0 and pd.isna(x.实付日期) and x.还款状态=='已逾期'
#     else 0 if x.实付金额>0 and x.实付金额<x.当前应付租金 and pd.notna(x.实付日期) and x.还款状态=='已逾期'
#     else x.当前应付租金 if x.实付金额==0 and pd.notna(x.实付日期) and x.还款状态=='续租中'
#     else x.实付金额
#     ,axis = 1)

In [45]:
# （13）df_xz_concat['实付日期new'] = df_xz_concat.apply(lambda x:
#     x.订单完成时间 if x.实付金额>0 and pd.isna(x.实付日期) and x.还款状态=='已买断'
#     else np.nan if x.实付金额>=0 and x.实付金额 < x.当前应付租金 and pd.notna(x.实付日期) and x.还款状态=='已逾期'
#     else x.实付日期
#     ,axis = 1)

## 3_13.获取逾期截止时间

In [46]:
df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
df_xz_concat['now_date'].head()

0   2025-02-09
1   2025-02-09
2   2025-02-09
3   2025-02-09
4   2025-02-09
Name: now_date, dtype: datetime64[s]

## 3_14.重新定义状态

In [47]:
df_xz_concat[(df_xz_concat.实付日期new.isnull()) & (~df_xz_concat.订单状态值.isin([8,10]))][['rembursement_status']].value_counts()
df_xz_concat[(df_xz_concat.实付日期new.isnull())& (pd.isna(df_xz_concat.finish_date_new))][['订单状态']].value_counts()

rembursement_status
还款中                    89172
已逾期                    42659
续租中                    17207
未到首期还款日                13784
检测中                       87
Name: count, dtype: int64

订单状态
租赁中     154820
待归还       6199
已退款       4308
待收货       1321
待发货        446
已完成        219
检测中         87
订单取消        36
Name: count, dtype: int64

In [48]:
conditions = [  
    ((pd.isna(df_xz_concat['实付日期new'])) & (pd.isna(df_xz_concat['finish_date_new'])) & (df_xz_concat['订单状态值'].isin([8, 10]))),  
    ((pd.to_datetime(df_xz_concat['now_date']) <= pd.to_datetime(df_xz_concat['应付日期'])) ),  
    ((pd.notna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['实付日期new']) <= pd.to_datetime(df_xz_concat['应付日期']))),  
    ((pd.notna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['实付日期new']) > pd.to_datetime(df_xz_concat['应付日期']))) ,
    ((pd.isna(df_xz_concat['实付日期new'])) & (pd.to_datetime(df_xz_concat['now_date']) > pd.to_datetime(df_xz_concat['应付日期'])) )  

]  
  
choices = ['已取消', '未到还款日', '正常还款', '已逾期支付','已逾期']  
  
# 使用numpy.where来应用条件  
df_xz_concat['状态'] = np.select(conditions, choices, default='其他')

df_xz_concat['状态'].value_counts()

状态
正常还款     198322
未到还款日    164160
已逾期支付     29835
已逾期       27219
已取消        4527
Name: count, dtype: int64

In [49]:
# df_xz_concat['实付日期new'] = pd.to_datetime(df_xz_concat['实付日期new'], errors='coerce')
# df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['实付日期new'], errors='coerce')

# # E:/myfile/业务相关数据/风控相关数据输出_{Today}.xlsx
# df_xz_concat.to_excel('E:/myfile/业务相关数据/报错表0902.xlsx')

## 3_15.重新定义逾期天数

In [50]:
df_xz_concat['实付日期new'].isnull().sum()
empty_rows = df_xz_concat[df_xz_concat['应付日期'].isna()]
empty_rows.order_number  

167436

Series([], Name: order_number, dtype: object)

In [51]:
# 异常值删除，应付时间为空
df_xz_concat.shape
df_xz_concat=df_xz_concat[~df_xz_concat['应付日期'].isnull()]
df_xz_concat.shape

(424063, 109)

(424063, 109)

In [52]:
#实付日期为空且完成时间为空  订单状态为已退款或已完成  :  逾期天数=0 
#note 20241127 逾期天数 截止目前的逾期天数 ；逾期一直未还的客户，应付日期越靠前，逾期天数越高
# A202207181530165
# A2024092919033313
# df_xz_concat[df_xz_concat.order_number == 'A202207181530165'][['order_number','逾期天数','应付日期','']]
# df_xz_concat[df_xz_concat.order_number == 'A2024092919033313'][['应付日期','实付日期','实付日期new','实付日期1','finish_date_new','当前期数','逾期天数','状态'
# ,'实付金额','当前应付租金','sesame_promo_money_pay','promo_money']]

df_xz_concat['逾期天数'] = df_xz_concat.apply(lambda x:
    0 if x['状态'] in ['未到还款日','正常还款','已取消']  
    else (pd.to_datetime(x['实付日期new']) - pd.to_datetime(x['应付日期'])).days  if pd.to_datetime(x['实付日期new']) >= pd.to_datetime(x['应付日期'])
    else  (pd.to_datetime(x['now_date']) - pd.to_datetime(x['应付日期'])).days   if   pd.to_datetime(x['now_date']) > pd.to_datetime(x['应付日期']) and pd.isna(x['实付日期new'])
    else 0 
    ,  axis=1)

# df_xz_concat['逾期天数'].value_counts()
df_xz_concat[df_xz_concat['逾期天数']>0]['rembursement_status'].value_counts()

rembursement_status
已逾期        40486
还款中         5710
续租中         4966
已买断         4849
已完成          850
检测中          147
未到首期还款日       33
Name: count, dtype: int64

In [53]:
# liudan
# # 定义一个函数来计算逾期天数
# def calculate_overdue_days(row):
#     if row['状态'] in ['未到还款日', '正常还款', '已取消']:
#         return 0
    
#     # 转换日期为 datetime 类型，并处理 None 和 NaN
#     实付日期new = pd.to_datetime(row['实付日期new'], errors='coerce')
#     应付日期 = pd.to_datetime(row['应付日期'], errors='coerce')
#     now_date = pd.to_datetime(row['now_date'], errors='coerce')

#     if pd.isna(实付日期new) or pd.isna(应付日期):
#         return 0
    
#     if 实付日期new >= 应付日期:
#         return (实付日期new - 应付日期).days
    
#     if pd.isna(实付日期new) and now_date > 应付日期:
#         return (now_date - 应付日期).days
    
#     return 0

# # 应用函数并创建新的 '逾期天数' 列
# df_xz_concat['逾期天数'] = df_xz_concat.apply(calculate_overdue_days, axis=1)

# # 显示结果
# print(df_xz_concat['逾期天数'].value_counts())

## 3_16.逾期天数处理

In [54]:
def over_due_days(df):  
    # 转换日期为pandas的日期类型
    df['实付日期'] = pd.to_datetime(df['实付日期'])  
    df['应付日期'] = pd.to_datetime(df['应付日期'])  
      
    # 创建一个新的列来存储结果  
    df['逾期天数new'] = np.nan  # 初始化为NaN  
      
    # 应用逻辑判断  
    mask1 = df['状态'].isin(['已逾期', '未到还款日', '已取消','已提前支付'])  
    mask2 = df['实付日期'].isnull()  
    mask3 = (df['实付日期'] - df['应付日期']).dt.days <= 0  
      
    # 更新计算天数列  
    df.loc[mask1, '逾期天数new'] = df.loc[mask1, '逾期天数']  
    df.loc[~mask1 & mask2, '逾期天数new'] = 0   
    df.loc[~mask1 & ~mask2 & mask3, '逾期天数new'] = 0  
    df.loc[~mask1 & ~mask2 & ~mask3, '逾期天数new'] = (df['实付日期'] - df['应付日期']).dt.days  
      
    # 实付日期为空时设置计算天数为0：  
    # df.loc[df['实付日期'].isnull(), '逾期天数new'] = 0  
      
    return df 

    # df_ddfq = over_due_days(df_ddfq)

## 3_17.稽查

In [55]:
# df_ddfq.columns
# df_xz_concat.shape
# check_fq_df = df_xz_concat.merge(df_ddfq,on=['订单ID','当前期数'],how='left')
# check_fq_df.shape
# check_fq_df.columns
# check_fq_df[['订单ID', '当前应付租金_x', '实付金额','应付日期_x', '实付日期_x','status', '当前期数', 'is_relet','订单号_x','下单月份', '下单日期','还款状态', '实付金额new', '实付日期new', '逾期天数_x','状态_x'
            #  ,'当前应付租金_y', '实际支付金额', '应付日期_y', '实付日期_y', '状态_y', '逾期天数_y','逾期天数new']].to_excel('E:/myfile/业务相关数据/check_fq_df.xlsx')


## 3_18.确认是否到表现期

In [56]:
# import os
# file_name = os.path.basename(f_path_ck)  

# df_xz_concat['date_str'] = file_name.split('excel')[1].split('.')[0]

# df_xz_concat['date_str'].head(1)

In [57]:
# df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
# df_xz_concat['now_date']

In [58]:
# dfzh1 = over_due_days(dfzh1)

df_xz_concat['now_date'] = pd.Timestamp(dt.now().date() )
df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['应付日期']).dt.date
# df_xz_concat['agr_days'] = (pd.to_datetime(df_xz_concat['now_date']) - df_xz_concat['应付日期']).dt.days 

In [59]:
# from datetime import timedelta
df_xz_concat['agr_1d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=1)
df_xz_concat['agr_4d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=4)
df_xz_concat['agr_7d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=7)
df_xz_concat['agr_15d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=15)
df_xz_concat['agr_30d'] = pd.to_datetime(df_xz_concat['应付日期']) + timedelta(days=30)

df_xz_concat[['agr_30d','agr_15d','agr_7d','agr_4d','agr_1d','应付日期']].head()

,agr_30d,agr_15d,agr_7d,agr_4d,agr_1d,应付日期
0,2022-08-04,2022-07-20,2022-07-12,2022-07-09,2022-07-06,2022-07-05
1,2022-09-04,2022-08-20,2022-08-12,2022-08-09,2022-08-06,2022-08-05
2,2022-10-05,2022-09-20,2022-09-12,2022-09-09,2022-09-06,2022-09-05
3,2022-11-04,2022-10-20,2022-10-12,2022-10-09,2022-10-06,2022-10-05
4,2022-12-05,2022-11-20,2022-11-12,2022-11-09,2022-11-06,2022-11-05


In [60]:
import os

# file_name = os.path.basename(f_path_ck)  
df_xz_concat['date_str'] = df_xz_concat['now_date']
df_xz_concat['agr_1d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_1d) <= pd.to_datetime(df_xz_concat.date_str),1,0)

df_xz_concat['agr_4d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_4d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_7d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_7d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_15d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_15d) <= pd.to_datetime(df_xz_concat.date_str),1,0)
df_xz_concat['agr_30d_cum'] = np.where(pd.to_datetime(df_xz_concat.agr_30d) <= pd.to_datetime(df_xz_concat.date_str),1,0)


df_xz_concat[['agr_30d','应付日期','date_str','agr_30d_cum']].head(5)
# df_xz_concat[(df_xz_concat.agr_7d_cum == 0) & (df_xz_concat.agr_4d_cum == 1)].head()
# df_xz_concat[df_xz_concat.order_id == 254588]

,agr_30d,应付日期,date_str,agr_30d_cum
0,2022-08-04,2022-07-05,2025-02-09,1
1,2022-09-04,2022-08-05,2025-02-09,1
2,2022-10-05,2022-09-05,2025-02-09,1
3,2022-11-04,2022-10-05,2025-02-09,1
4,2022-12-05,2022-11-05,2025-02-09,1


In [61]:
# note 20241211 数据检查
# df_xz_concat[(df_xz_concat.应付日期>='2024-12-01')&(df_xz_concat.应付日期<='2024-12-31')&(df_xz_concat.order_number.isin(
# ['A20241101174043208'
# ,'A2024110313351882'
# ,'A20241103155713186'
# ,'A2024110318311457'
# ,'A20241104145450273'
# ,'A20241104164506212'
# ,'A20241105105325369'
# ,'A20241105154302243'
# ,'A2024110515050030'
# ,'A2024110610041421'
# ,'A20241106144302240'
# ,'A2024110709254879'
# ,'A20241107103123201'
# ,'A20241107195535157'
# ,'A20241108104251268'
# ,'A20241108155751456'
# ,'A2024110715142073'
# ,'A2024110817092834'
# ]))][['order_number','agr_1d_cum','agr_4d_cum','agr_7d_cum','agr_4d','应付日期','date_str','agr_1d','下单日期']]

## 3_19.观察日定义 每月月底时间

In [62]:
from pandas.tseries.offsets import MonthEnd
def mob_date(df):
    df['下单日期'] = pd.to_datetime(df['下单日期'])
    df['实付日期new'] = pd.to_datetime(df['实付日期new'])
    # note n 续租的没有统计
    for n in range(0,13):
    # 2025.2.8 
    # for n in range(0,16):  
        # df[f'mob_date_{n}'] = df['下单日期'] + MonthEnd(n+1)
        # note 20241205 修改 月末的的跳级了，所以不要加1，mob0对齐
        # df[f'mob_date_{n}'] = np.where(df["下单日期"].dt.day.isin([28,29,30,31]), df['下单日期'] + MonthEnd(n), df['下单日期'] + MonthEnd(n+1))
        df[f'mob_date_{n}'] = np.where((df["下单日期"].dt.day == 31)|(df["下单日期"].isin(['2023-02-28','2024-02-29','2025-02-28']))|((df["下单日期"].dt.month.isin([4,6,9,11]))&(df["下单日期"].dt.day == 30)), df['下单日期'] + MonthEnd(n), df['下单日期'] + MonthEnd(n+1))
        df[f'mob_date_{n}'] = pd.to_datetime(df[f'mob_date_{n}'])
        df[f'paid_date_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],pd.NaT,df['实付日期new'])
        df[f'paid_money_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],np.nan,df['实付金额'])
        df[f'paid_date_mob{n}'] = pd.to_datetime(df[f'paid_date_mob{n}'])
    return df

df_xz_concat.shape
df_xz_concat = mob_date(df_xz_concat)
df_xz_concat.shape

# A2023072814030812  2023-7-28 加收二期，月末为账单日
# A2023073012170638  2023-7-30 3+1，2023.8无账单日
# A2023070115535442  2023-7-1
# A2023073114301076  2023-7-31 1元+1期，续租

# A202301011402371 2023-01-01
# A202301290815565 2023-01-29
# A2023013100341134 2023-01-31
# A2023022801142011 2023-02-28
# A2023022709283120  2023-02-27
# A202302050022319  2023-02-05
# A2023040503314720 2023-04-05
# A2023042803470516  2023-04-28
# A2023043000524068 2023-04-30
# A2023063003392242 2023-06-30
# A202306280105556 2023-06-28
# A202306050324407  2023-06-05
# A202310050100222 2023-10-05
# A2023102901224646 2023-10-29
# A2023103101255825 2023-10-31

(424063, 121)

(424063, 160)

In [63]:
df_xz_concat['date_str']  = pd.to_datetime( df_xz_concat['date_str'] )
df_xz_concat['应付日期'] = pd.to_datetime(df_xz_concat['应付日期']) 

In [64]:
# var = ['order_number','当前期数','mob_date_12','paid_date_mob12','paid_money_mob12','mob_date_13','paid_date_mob13','paid_money_mob13','mob_date_14','paid_date_mob14','paid_money_mob14','mob_date_15','paid_date_mob15','paid_money_mob15']
# df_xz_concat[df_xz_concat.order_number.isin(['A2024060200290566','A2024063001194047','A202308262317036','A2024031113200319','A202207050851373'])][var]
# df_xz_concat[df_xz_concat.order_number.isin(['A202207050851373'])][var]

## 3_20.更新逾期天数

In [65]:
# （1）逾期天数校正
# note 20241213
# 首期实际付款日期为空的订单，为首期取消或者待发货的，贷后表现逾期天数更新为0（各期更新为0）

# 把符合条件的订单拿出来，逾期天数全部更新为0
sql_xz = ''' 
SELECT distinct tprm.order_number,1 as label
# ,ymos.reality_refund_date,ymos.sort as '当前期数',ymos.status
from db_rent.ya_merchant_order_stages ymos 
inner join db_digua_business.t_postlease_receivables_monitoring tprm 
on ymos.order_id = tprm.order_id 
where ymos.sort=1 and ymos.reality_refund_date is null
;
'''
tmp = query(sql_xz)
tmp.shape
# tmp[:2]

df_xz_concat = df_xz_concat.merge(tmp, on = 'order_number', how = 'left')
df_xz_concat['label'] = df_xz_concat['label'].fillna(0)
df_xz_concat['label'].value_counts()

df_xz_concat[df_xz_concat['label'] == 1].drop_duplicates(subset = 'order_number').shape

df_xz_concat['逾期天数'] = np.where(df_xz_concat['label'] == 1, 0, df_xz_concat['逾期天数'])
# df_xz_concat['逾期天数_new'] = np.where(df_xz_concat['label'] == 1, 0, df_xz_concat['逾期天数'])
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']].shape
# df_xz_concat[(df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']) & (df_xz_concat.逾期天数 > 30)].shape
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']][['order_number','逾期天数','逾期天数_new']]
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']].逾期天数.max()
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']].逾期天数.min()
# df_xz_concat[df_xz_concat['逾期天数'] != df_xz_concat['逾期天数_new']][['order_number','逾期天数','逾期天数_new']].to_excel('tmp.xlsx', index = False)

# A202207181530165
# A2024092919033313 号卡
# A2023011718580545 vintage分子是月末状态，FSTQPD是发生过逾期
# df_xz_concat[df_xz_concat.order_number == 'A2023011718580545'][['order_number','应付日期','实付日期','实付金额','逾期天数']]
# df_mob_total[df_mob_total.order_number == 'A2023011718580545'][['order_number','下单日期','paid_money_mob','overdue_days_mob','当前应付租金','MOB','overdue_days_1','overdue_days_4','overdue_days_7','overdue_days_15','overdue_days_31']]

(671, 2)

label
0.0    416167
1.0      7896
Name: count, dtype: int64

(650, 161)

## 3_20.加工资方所需字段

In [66]:
# 20250207
df_xz_concat_1 = df_xz_concat.copy()
df_xz_concat_1.loc[:,"租赁方案"]=np.where(df_xz_concat_1["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")

# （1）取首期应还日期，区分续租和非续租  首期应还日期
tmp1 = df_xz_concat_1[(df_xz_concat_1.is_relet == 0) & (df_xz_concat_1.当前期数 == 1)][['order_number','应付日期']]
tmp1 = tmp1.rename(columns = {'应付日期': '首期应还日期'})
tmp2 = df_xz_concat_1[df_xz_concat_1.is_relet == 0].merge(tmp1, on = 'order_number', how = 'left')

tmp3 = df_xz_concat_1[df_xz_concat_1.is_relet != 0].groupby(['order_number']).agg({'应付日期':'min'}).reset_index()
tmp3 = tmp3.rename(columns = {'应付日期': '首期应还日期'})
tmp4 = df_xz_concat_1[df_xz_concat_1.is_relet != 0].merge(tmp3, on = 'order_number', how = 'left')

df_xz_concat_1 = pd.concat([tmp2, tmp4])
df_xz_concat_1.shape

del tmp1
del tmp2
del tmp3
del tmp4

# （2）提取买断金等信息，区分续租和非续租 买断价，续租的买断价包含了首租的买断价
# 续租买断尾款 为续租总租金，续租买断尾款其实为0
# 首租订单，首租买断尾款为续租总租金，实际中，续租总租金 大于 首租买断尾款
# all_money：续租的已完成的话，为首租+续租总租金；续租的未完成的话，为首租的总租  'A202308262317036','A202207050851373'
sql = '''
SELECT tprm.order_id, tprm.order_number, tprm.order_create_time 下单日期
    ,tod.actual_money 首租买断价
    ,tod.new_actual_money 续租买断价
    , tprm.all_deposit 总押金
    , tprm.total_receivable 总应收
    , tprm.total_received 总已收
    # , om.all_money 首租总租金
    # , (tod.new_actual_money-om.all_money) 续租总租金
    # , (tod.actual_money-om.all_money) 首租买断尾款
    , tprm.purchase_amount 采购金额
    , om.has_actual 是否买断
    , om.relet_status 是否续租
    , tprm.paid_periods 已还期数
    , tod.early_completion_mark 是否提前完成
    , tmu.id_card_num 身份证号
    , tol.delivery_province_name 地区
    , pa.type 活动类型
    , pa.type, too.money 滞纳金, tprm.liquidated_damages 违约金
from db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_order om on om.id=tprm.order_id
left join db_digua_business.t_platform_activity pa on om.activity_id=pa.id
left join db_digua_business.t_member_user tmu on om.user_id=tmu.id
left join db_digua_business.t_order_details tod on tod.order_id=tprm.order_id
left join db_digua_business.t_order_logistics tol on tol.order_id=tprm.order_id
left join (SELECT t.order_id,sum(case when t.capital_type =6 and t.`status`=2 then t.money end) as other_money 
,sum(case when t.capital_type != 5 AND  t.`status` = 2 then t.money end) as money
from db_digua_business.t_order_other t GROUP BY 1 ) too on too.order_id=om.id
'''

df_temp_1 = query(sql)
df_temp_1.shape
df_temp_1 = df_temp_1.drop_duplicates(subset = ['order_number'])
df_temp_1.shape

df_temp_1_1 = df_xz_concat_1[(df_xz_concat_1.is_relet == 0)].merge(df_temp_1[['order_number','首租买断价']], on = 'order_number', how = 'left')
df_temp_1_1 = df_temp_1_1.rename(columns = {'首租买断价': '买断价'})
df_temp_1_1.shape

df_temp_1_2 = df_xz_concat_1[(df_xz_concat_1.is_relet != 0)].merge(df_temp_1[['order_number','续租买断价']], on = 'order_number', how = 'left')
df_temp_1_2 = df_temp_1_2.rename(columns = {'续租买断价': '买断价'})
df_temp_1_2.shape

df_xz_concat_1 = pd.concat([df_temp_1_1, df_temp_1_2])

# （3）加工应付总租金，区分续租和非续租 应付总租金
df_temp_2 = df_xz_concat_1[(df_xz_concat_1.is_relet == 0)].groupby(['order_number']).agg({'当前应付租金':'sum'}).reset_index()
df_temp_2 = df_temp_2.rename(columns = {'当前应付租金': '应付总租金'})
df_temp_2.shape

df_temp_3 = df_xz_concat_1[(df_xz_concat_1.is_relet != 0)].groupby(['order_number']).agg({'当前应付租金':'sum'}).reset_index()
df_temp_3 = df_temp_3.rename(columns = {'当前应付租金': '应付总租金'})
df_temp_3.shape

df_temp_4 = df_xz_concat_1[(df_xz_concat_1.is_relet == 0)].merge(df_temp_2, on = 'order_number', how = 'left')
df_temp_4.shape

df_temp_5 = df_xz_concat_1[(df_xz_concat_1.is_relet != 0)].merge(df_temp_3, on = 'order_number', how = 'left')
df_temp_5.shape

df_xz_concat_1 = pd.concat([df_temp_4, df_temp_5])

#（4）买断金 续租的买断尾款为 0 
df_xz_concat_1['买断尾款'] = df_xz_concat_1['买断价'] - df_xz_concat_1['应付总租金']
df_xz_concat_1['买断尾款'] = np.where(df_xz_concat_1.is_relet == 0, df_xz_concat_1['买断尾款'], 0)
df_xz_concat_1.shape

del df_temp_1_1
del df_temp_1_2
del df_temp_1
del df_temp_2
del df_temp_3
del df_temp_4
del df_temp_5

# （5）补三期数据
# 当前期数、应付日期 改变，其他属性不变
tmp1 = df_xz_concat_1.groupby(['order_number']).agg({'当前期数':'max'}).reset_index()
var = ['order_number','order_id','is_relet','当前期数', '下单日期', '应付日期','首期应还日期', '买断价', '应付总租金', '买断尾款','租赁方案','rembursement_status','finish_date_new']
tmp1.shape

tmp2 = tmp1.merge(df_xz_concat_1[var], on = ['order_number', '当前期数'], how = 'inner')
tmp2['当前期数'] = tmp2['当前期数'] + 1
tmp2['应付日期'] = tmp2['应付日期'] + pd.DateOffset(months = 1)

tmp3 = tmp1.merge(df_xz_concat_1[var], on = ['order_number', '当前期数'], how = 'inner')
tmp3['当前期数'] = tmp3['当前期数'] + 2
tmp3['应付日期'] = tmp3['应付日期'] + pd.DateOffset(months = 2)

tmp4 = tmp1.merge(df_xz_concat_1[var], on = ['order_number', '当前期数'], how = 'inner')
tmp4['当前期数'] = tmp4['当前期数'] + 3
tmp4['应付日期'] = tmp4['应付日期'] + pd.DateOffset(months = 3)

tmp5 = pd.concat([tmp2, tmp3, tmp4])
tmp5.shape

df_xz_concat_1 = pd.concat([df_xz_concat_1, tmp5])
df_xz_concat_1.shape
# tmp1[tmp1.下单日期 == '2024-01-27'][:1]
# A2024012707300311 1-30 02-28
# A2023032611070524 2-29 3-29
# A2022092917545514 3-31 4-30
# A202207050851373 5-31 6-30
# tmp5[tmp5.order_number.isin(['A202207050851373'])]

# （6）订单状态定位到 当月
df_xz_concat_1['date_tmp_1'] = df_xz_concat_1['应付日期'].dt.strftime('%Y-%m')
df_xz_concat_1['date_tmp_1'] = pd.to_datetime(df_xz_concat_1['date_tmp_1'])
df_xz_concat_1['date_tmp_2'] = np.where(df_xz_concat_1['finish_date_new'].notnull(), pd.to_datetime(df_xz_concat_1['finish_date_new']).dt.strftime('%Y-%m'), np.nan)
df_xz_concat_1['date_tmp_2'] = pd.to_datetime(df_xz_concat_1['date_tmp_2'])

# 判断订单结束日期非空的订单是买断还是非买断，未结束的订单不判断（资方所需的是是否结清，分买断或完成）
df_xz_concat_1['订单状态_new'] = df_xz_concat_1.apply(lambda x: '已买断' if x.date_tmp_1 == x.date_tmp_2 and x.rembursement_status == '已买断'
                                              else '已完成' if x.date_tmp_1 == x.date_tmp_2 and x.rembursement_status == '已完成'
                                              else '还款中' if x.date_tmp_1 < x.date_tmp_2
                                              else '已结束' if x.date_tmp_1 > x.date_tmp_2
                                              else '其他', axis = 1
                                              )
# df_xz_concat_1 = df_xz_concat_1.drop(columns = ['date_tmp_1', 'date_tmp_2'])
df_xz_concat_1.shape

# (7)关联买断数据 只考虑总期数往后加三期的
sql1 = '''-- 买断信息   
SELECT too.order_id
,too.real_pay_money 
,too.pay_date
from db_digua_business.t_order_out too
where  too.`status` not in (1)   -- 买断状态：1 未买断 2 已买断  3 部分买断
and too.real_pay_money > 0 and too.pay_date is not null
'''
df_out_1 = query(sql1)
df_out_1.shape
df_out_1["pay_date"] = df_out_1['pay_date'].dt.date
df_out_1["pay_date"] = pd.to_datetime(df_out_1["pay_date"],errors="coerce")
df_out_1['pay_date_m'] = df_out_1['pay_date'].dt.strftime('%Y-%m')
df_out_1['pay_date_m'] = pd.to_datetime(df_out_1['pay_date_m'])

df_out_2 = df_out_1.groupby(['order_id','pay_date_m']).agg({'real_pay_money':'sum'}).reset_index()
df_out_2 = df_out_2.rename(columns = {'real_pay_money': 'real_pay_money_new'})
df_out_2.shape

df_xz_concat_1 = df_xz_concat_1.merge(df_out_2[['order_id', 'real_pay_money_new', 'pay_date_m']], left_on = ['order_id', 'date_tmp_1'], right_on = ['order_id','pay_date_m'], how = 'left')
df_xz_concat_1 = df_xz_concat_1.drop(columns = ['date_tmp_1', 'date_tmp_2', 'pay_date_m'])

# 计算剩余买断尾款
df_xz_concat_1['real_pay_money_new'] = df_xz_concat_1['real_pay_money_new'].fillna(0)
df_xz_concat_1['剩余总应收买断金'] = df_xz_concat_1['买断尾款'] - df_xz_concat_1['real_pay_money_new'] 
df_xz_concat_1['剩余总应收买断金'] = np.where(df_xz_concat_1['剩余总应收买断金'] < 0 , 0, df_xz_concat_1['剩余总应收买断金'])
df_xz_concat_1['剩余总应收买断金'] = np.where(df_xz_concat_1['订单状态_new'].isin(['已结束' ,'已完成','已买断']), 0, df_xz_concat_1['剩余总应收买断金'])
df_xz_concat_1.shape
df_xz_concat_1[:1]

# var1 = ['order_number','is_relet','当前期数', '下单日期', '应付日期', '当前应付租金', '实付日期new','实付金额','rembursement_status','订单状态','if_outpay','状态','订单状态_new']
# var2 = ['mob_date_11','paid_date_mob11','paid_money_mob11','mob_date_12','paid_date_mob12','paid_money_mob12','mob_date_13','paid_date_mob13','paid_money_mob13','mob_date_14','paid_date_mob14','paid_money_mob14','mob_date_15','paid_date_mob15','paid_money_mob15']
# var3 = ['order_id','当前期数','应付日期','实付日期new','实付金额','finish_date_new','订单状态_new','首期应还日期','买断价','应付总租金','买断尾款','订单状态_new','real_pay_money_new','剩余总应收买断金']
# var = var1 + var2 + var3

# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024060200290566'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024063001194047'])][var3]
# 订单已完成
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024031113200319'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202308262317036'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207050851373'])][var3]
# 在15期之前买断
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024020317504938'])][var3]
# 在15期之后买断
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207171820512'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207181530165'])][var3]
# 已买断但 买断尾款	!= real_pay_money_new  可能使用优惠劵
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207151707153'])][var3]
# 续租的实付买断金非0

(424063, 163)

(34119, 19)

(34116, 19)

(384318, 164)

(39745, 164)

...

(520033, 169)

(2817, 3)

(2744, 3)

(520033, 169)

,id_x,order_id,当前应付租金,实付金额,interest_money,实付日期,status_x,当前期数,order_pay_id,create_time_x,...,paid_money_mob12,label,租赁方案,首期应还日期,买断价,应付总租金,买断尾款,订单状态_new,real_pay_money_new,剩余总应收买断金
0,6506.0,8924,357.95,357.95,0.0,2022-07-05,3.0,1.0,9576,2022-07-05 08:51:37,...,357.95,0.0,租完归还,2022-07-05,7530.0,4295.4,3234.6,还款中,0.0,3234.6


## 3_22.更新租期到期的逾期天数

In [67]:
# （1）更新归还逾期的订单的逾期天数
# 本次未统计在内，应该用 df_xz_concat

# （1-1）未归还逾期或未到期或系统当天到期
df_xz_concat.shape
conditions_1 = (df_xz_concat.is_relet == 0)
tmp1 = df_xz_concat[(conditions_1 & (df_xz_concat.end_date >= df_xz_concat.finish_date_new)) | (conditions_1 & (df_xz_concat.end_date >= df_xz_concat.date_str))]
tmp1.shape

# （1-2）续租
tmp2 = df_xz_concat[df_xz_concat.is_relet > 0 ]
tmp2.shape
# （1-3）出现过归还逾期（包含租期一直逾期直到归还逾期的）--仅首续有归还逾期，被减数为最近未支付的应付日期
conditions_2 = (df_xz_concat.end_date < df_xz_concat.finish_date_new)
conditions_3 = (df_xz_concat.finish_date_new.isnull())
conditions_4 = (df_xz_concat.end_date < df_xz_concat.date_str)
tmp3 = df_xz_concat[(conditions_1 & conditions_2) | (conditions_1 & conditions_3 & conditions_4)]
tmp3.shape

# 367644 + 55490

# 计算到期后逾期天数--订单已经完成的： 逾期天数为 finish_date_new- end_date；订单未完成的，逾期天数为 date_str- 实付日期new为空的最小的 应付日期
# ((df_xz_concat['finish_date_new'] - df_xz_concat['end_date']).dt.days > 0) 
# A2024020317504938--订单已完成
# A202207171820512 --订单已完成
# A202207181530165--订单未完成
# var = ['order_number','当前期数','应付日期','实付日期new','end_date','date_str','逾期天数','pay_date','order_finish_date','订单完成时间','finish_date_new']
# df_xz_concat[df_xz_concat.order_number == 'A202207181530165'][var]

(424063, 161)

(329931, 161)

(39745, 161)

(54387, 161)

In [68]:
# （2）数据检查
# df_fq_concat[df_fq_concat.下单日期 == '2024-06-30'][['order_number']][:2]
# A2024060200290566  A2024063001194047 A202308262317036（续租） A2024031113200319（提早归还） A202207050851373（续租提前买断）
# var = ['order_number', '当前期数', '下单日期', '应付日期', '首期应还日期', '当前应付租金','rembursement_status','订单状态','if_outpay','状态','是否提前还款','实付日期new','逾期天数','order_finish_date','finish_date_new']
# df_fq_concat[df_fq_concat.order_number.isin(['A2024060200290566','A2024063001194047','A202308262317036','A2024031113200319','A202207050851373'])][var]

# var = ['order_number', '当前期数', '下单日期', '应付日期', '当前应付租金','rembursement_status','订单状态','if_outpay','状态','实付日期new','finish_date_new','status_x','paid_periods','实付金额','date_tmp_1','date_tmp_2','订单状态_new','pay_date']
# var = ['order_number','is_relet','当前期数', '下单日期', '应付日期', '当前应付租金', '实付金额','rembursement_status','订单状态','if_outpay','状态','首期应还日期', '买断价', '应付总租金', '买断尾款', '订单状态_new','租赁方案']
# df_xz_concat[df_xz_concat.order_number.isin(['A2024060200290566','A2024063001194047','A2024031113200319'])][var]
# df_xz_concat[df_xz_concat.order_number.isin(['A202308262317036'])][var]
# df_xz_concat[df_xz_concat.order_number.isin(['A202207050851373'])][var]

# 4.非续租订单

<!-- （1）每个mob 观察日 对应的逾期天数
（2）实付日期逻辑：如实付日期大于观察日，实付日期清空
（3）达到表现期逻辑：应付日期小于观察日，且应付日期小于当前日期
（4）逾期天数判断逻辑:如状态为已取消状态则0，如实付日期不为空则 0，如实付日期为空且观察日期大于当前日期则 当前日期-应付日期，否则 观察日期-应付日期 -->

## 4_11.数据处理

In [159]:
# 非续租租期数据
# (384318, 161)
# df_fq_concat = df_xz_concat[df_xz_concat.is_relet==0]
# 2025.2.9
df_xz_concat_2 = df_xz_concat.merge(df_xz_concat_1[['order_number', '订单状态_new','当前期数']],on = ['order_number','当前期数'], how = 'inner')
# conditions_1 = (df_xz_concat_2.订单状态_new != '已结束') | ((df_xz_concat_2.订单状态_new == '已结束') & (df_xz_concat_2.实付金额 > 0))
# df_xz_concat_2 = df_xz_concat_2[conditions_1]
df_fq_concat = df_xz_concat_2[df_xz_concat_2.is_relet==0]
df_fq_concat['当前应付租金'] = np.where((df_fq_concat.订单状态_new == '已结束') & (df_fq_concat.实付金额 == 0), 0, df_fq_concat.当前应付租金)

df_fq_concat.shape
df_xz_concat.is_relet.value_counts()

# note 20241205 首个应付日期更新为下单日期，避免 首个应付日期为下个月的订单没有 把mob0的已付统计在内，首个应收日期为下个月的订单，mob1没有账单日
df_fq_concat['应付日期_new'] = np.where(df_fq_concat.当前期数 == 1, df_fq_concat.下单日期, df_fq_concat.应付日期)
# note 20241206 更新 应付日期，提前还款的统计在内
df_fq_concat['应付日期_new'] = np.where((df_fq_concat['实付日期new'].notna())&((df_fq_concat['应付日期_new'] - df_fq_concat['实付日期new']).dt.days >= 0),df_fq_concat['实付日期new'],df_fq_concat['应付日期_new'])
df_fq_concat.shape

# A2023072814030812  2023-7-28 加收二期，月末为账单日
# A2023073012170638  2023-7-30 3+1，2023.8无账单日
# A2023070115535442  2023-7-1
# A2023073114301076  2023-7-31 1元+1期，续租

(384318, 162)

is_relet
0.0    384318
1.0     39683
2.0        62
Name: count, dtype: int64

(384318, 163)

In [ ]:
# note 20241205 每月28.29.30号下单的没有统计mob_0已付的
df_mob_list = []
for i in range(0,13):
# 2025.2.9
# for i in range(0,16):
    # note 20241205 增加 (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])，不然预收的没有在第1期体现出来， A2024063001194047 下单日期!=实付日期new（第一期）
    # note 20241205 增加=，不然28或27号的没有mob0，导致 paid_money_mob 计算不正确，即还款日为月末的计算不正确
    # df_mob = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期']).dt.days > 0) 
    df_mob = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期_new']).dt.days >= 0) 
                        #   & ((df_fq_concat['date_str'] - df_fq_concat['应付日期']).dt.days > 0) 
                        # 2025.2.9
                          & ((df_fq_concat['date_str'] - df_fq_concat['应付日期_new']).dt.days > 0) 
                          & ((df_fq_concat['date_str'] - df_fq_concat[f'mob_date_{i}']).dt.days > 0)
                          | (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])]  # &((dfzh1['date_str'] - dfzh1[f'mob_date_{i}']).dt.days > 0)
# df_mob2 = dfzh1[((dfzh1['mob_date_2'] - dfzh1['应付日期']).dt.days > 0) & (dfzh1.状态 != '已取消') & ((dfzh1['date_str'] - dfzh1['mob_date_2']).dt.days > 0)]

    # df_mob['overdue_days_mob'] = df_mob.apply(lambda x:
    #                                           0 if x['逾期天数']==0  
    #                                           else 0  if pd.notnull(x[f'paid_date_mob{i}']) 
    #                                           else (x['date_str'] - x['应付日期']).days  if  ((x[f'mob_date_{i}'] - x['date_str']).days > 0)  and pd.isnull(x[f'paid_date_mob{i}'])
    #                                           else (x[f'mob_date_{i}'] - x['应付日期']).days if ((x[f'mob_date_{i}'] - x['date_str']).days <= 0)  and pd.isnull(x[f'paid_date_mob{i}'])
    #                                           else x['逾期天数'] ,axis=1 )
    # # np.where(df_mob[f'paid_date_mob{i}'].isnull(),(np.minimum(df_mob['date_str'],df_mob[f'mob_date_{i}']) - df_mob['应付日期']).dt.days  ,(df_mob[f'paid_date_mob{i}'] - df_mob['应付日期']).dt.days)
    # df_mob['paid_money_mob'] = df_mob[f'paid_money_mob{i}']

    # df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    # df_mob_group['MOB'] = f'mob_{i}'

    # df_mob_list.append(df_mob_group)

    conditions = [ ( ( df_mob['逾期天数']==0) | (pd.notnull(df_mob[f'paid_date_mob{i}']))),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days > 0)  & (pd.isnull(df_mob[f'paid_date_mob{i}'])) ),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days <= 0) & ( pd.isnull(df_mob[f'paid_date_mob{i}'])))]  
  
    choices = [0,(df_mob['date_str'] - df_mob['应付日期']).dt.days,(df_mob[f'mob_date_{i}'] - df_mob['应付日期']).dt.days]  
  
    # 使用numpy.where来应用条件  
    # note 20241127 overdue_days_mob 截止每个mob观察月的逾期天数，月末状态，若月末不逾期了，则为0
    df_mob['overdue_days_mob'] = np.select(conditions, choices, default = df_mob['逾期天数'])

    df_mob['paid_money_mob'] = df_mob[f'paid_money_mob{i}']

    # df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    # note 20241210 修改 提前支付的剔除未到期应收
    df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    df_mob_group_1 = df_fq_concat[((df_fq_concat[f'mob_date_{i}'] - df_fq_concat['应付日期']).dt.days >= 0) 
                      & ((df_fq_concat['date_str'] - df_fq_concat['应付日期']).dt.days > 0) 
                      & ((df_fq_concat['date_str'] - df_fq_concat[f'mob_date_{i}']).dt.days > 0)
                      # 2025.2.9
                    #   | (df_fq_concat['下单日期'] == df_fq_concat['实付日期new'])
                      ].groupby(by=['order_number']).agg({'当前应付租金':'sum','当前期数':'max'}).reset_index()
    
    df_mob_group = df_mob_group.merge(df_mob_group_1, on = 'order_number', how = 'left')

    df_mob_group['MOB'] = f'mob_{i}'

    df_mob_list.append(df_mob_group)

df_mob_total = pd.concat(df_mob_list, ignore_index=True)

# 2025.2.9
df_mob_total.shape
df_mob_total = df_mob_total[df_mob_total.当前应付租金 >= 0]
df_mob_total = df_mob_total.drop_duplicates(subset = ['order_number','下单日期','paid_money_mob','overdue_days_mob','当前应付租金','当前期数'])
df_mob_total.shape
df_mob_total = df_mob_total.sort_values(by = ['order_number','当前期数','overdue_days_mob'])
df_mob_total = df_mob_total.drop_duplicates(subset = ['order_number','当前期数'])

df_mob_total.tail(10)
# df_mob_total[df_mob_total.order_number == 'A202207151707153']
# var3 = ['order_id','当前期数','应付日期','当前应付租金','实付日期new','实付金额','finish_date_new','订单状态_new','首期应还日期','买断价','应付总租金','买断尾款','订单状态_new','real_pay_money_new','剩余总应收买断金']
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024060200290566'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024063001194047'])][var3]
# 订单已完成
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024031113200319'])][var3] 
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202308262317036'])][var3] 
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207050851373'])][var3]
# # 在15期之前买断
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A2024020317504938'])][var3]
# 在15期之后买断
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207171820512'])][var3]
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207181530165'])][var3] --2个12期
# 已买断但 买断尾款	!= real_pay_money_new  可能使用优惠劵
# df_xz_concat_1[df_xz_concat_1.order_number.isin(['A202207151707153'])][var3]

# paid_money_mob：当前累计实付租金  当前应付租金：当累计应付租金

(396191, 7)

(236931, 7)

,order_number,下单日期,paid_money_mob,overdue_days_mob,当前应付租金,当前期数,MOB
31685,A2025012615480550,2025-01-26,719.04,0,1.00,1.0,mob_0
31686,A2025012617483443,2025-01-26,537.94,0,1.00,1.0,mob_0
31687,A202501261811289,2025-01-26,2151.96,0,1.00,1.0,mob_0
31688,A2025012619102611,2025-01-26,599.75,0,599.75,1.0,mob_0
31689,A2025012622475925,2025-01-26,459.82,0,1.00,1.0,mob_0
31690,A2025012709163214,2025-01-27,719.04,0,1.00,1.0,mob_0
31691,A2025012710270720,2025-01-27,1082.94,0,1082.94,1.0,mob_0
31692,A2025012713364018,2025-01-27,455.93,0,1.00,1.0,mob_0
31693,A2025012714175415,2025-01-27,455.93,0,1.00,1.0,mob_0
31694,A2025012716123412,2025-01-27,744.53,0,744.53,1.0,mob_0


In [161]:
# df_xz_concat_1.shape
df_xz_concat.shape
df_fq_concat.shape
df_mob_total.shape


(424063, 161)

(384318, 163)

(234387, 7)

In [ ]:
# 关联15期的表
# 2025.2.9
df_mob_total = df_xz_concat_1[['']]
# 续租的在12期修改订单状态为已完成，同时修改 买断金为0

In [67]:
df_mob_total['overdue_days_1'] = np.where(df_mob_total['overdue_days_mob']>= 1,1,0)
df_mob_total['overdue_days_4'] = np.where(df_mob_total['overdue_days_mob']>= 4,1,0)
df_mob_total['overdue_days_7'] = np.where(df_mob_total['overdue_days_mob']>= 7,1,0)
df_mob_total['overdue_days_15'] = np.where(df_mob_total['overdue_days_mob']>= 15,1,0)
df_mob_total['overdue_days_31'] = np.where(df_mob_total['overdue_days_mob']>= 31,1,0)
df_mob_total['下单月份'] = df_mob_total['下单日期'].dt.strftime('%Y-%m')  

In [68]:
# df_mob_total['采购金额'] 
df_mob_total.shape
df_mob_total = df_mob_total.merge(dfzh3[['order_number','purchase_amount','归属渠道','是否二手']],on = 'order_number',how='left')
df_mob_total.shape

# note 20241127 707条订单采购金额是0，客户取消
# A2023032216313863
# A202207131230485
# dfzh3[dfzh3.purchase_amount == 0].shape

(394605, 12)

(394631, 15)

In [69]:
from datetime import datetime as dt    
today_date = dt.now().date()  
date_str = today_date.strftime('%Y-%m-%d')  

# df_mob_total.to_excel(f'E:/myfile/业务相关数据/df_mob_total-{date_str}.xlsx')

## 4_12.每月对应的 用户数及应还金额

In [70]:
# note 20241127 增加条件 (df_fq_concat.purchase_amount > 0) 
# note 20241205 删除条件  (df_fq_concat.purchase_amount > 0)  A202207131230485：purchase_amount 为0，有多种情况，已发货取消，未发货取消，故不删除
# （1）总体
# df_mon = df_fq_concat[(df_fq_concat.当前期数==1) & (df_fq_concat.purchase_amount > 0)].groupby(by=['search_time']).agg({'order_number': 'count', 'contract_price': 'sum','total_receivable':'sum','purchase_amount':'sum'})
# df_mon.columns = df_mon.columns.droplevel(1) 
# df_mon2 = df_fq_concat[(df_fq_concat.当前期数==1)& (df_fq_concat.purchase_amount > 0)&(df_fq_concat.归属渠道!='抖音渠道')].groupby(by=['search_time']).agg({'order_number': 'count', 'contract_price': 'sum','total_receivable':'sum','purchase_amount':'sum'})
df_mon2 = df_fq_concat[(df_fq_concat.当前期数==1)& (df_fq_concat.归属渠道!='抖音渠道')].groupby(by=['search_time']).agg({'order_number': 'count', 'contract_price': 'sum','total_receivable':'sum','purchase_amount':'sum'})
df_mon2 

# （2）租物渠道
# df_mon_zm = df_fq_concat[(df_fq_concat.当前期数==1)& (df_fq_concat.purchase_amount > 0) & (df_fq_concat.归属渠道=='芝麻租物')].groupby(by=['search_time']).agg({'order_number': 'count', 'contract_price': 'sum','total_receivable':'sum','purchase_amount':'sum'})
# df_mon.columns = df_mon.columns.droplevel(1) 
df_mon_zm = df_fq_concat[(df_fq_concat.当前期数==1)& (df_fq_concat.归属渠道=='芝麻租物')].groupby(by=['search_time']).agg({'order_number': 'count', 'contract_price': 'sum','total_receivable':'sum','purchase_amount':'sum'})
df_mon_zm

# （3）搜搜渠道
# df_mon_ss = df_fq_concat[(df_fq_concat.当前期数==1)& (df_fq_concat.purchase_amount > 0) & (df_fq_concat.归属渠道=='搜索渠道')].groupby(by=['search_time']).agg({'order_number': 'count', 'contract_price': 'sum','total_receivable':'sum','purchase_amount':'sum'})
# df_mon.columns = df_mon.columns.droplevel(1) 
df_mon_ss = df_fq_concat[(df_fq_concat.当前期数==1) & (df_fq_concat.归属渠道=='搜索渠道')].groupby(by=['search_time']).agg({'order_number': 'count', 'contract_price': 'sum','total_receivable':'sum','purchase_amount':'sum'})
df_mon_ss

# （4）抖音
# df_mon_dy = df_fq_concat[(df_fq_concat.当前期数==1)& (df_fq_concat.purchase_amount > 0) & (df_fq_concat.归属渠道=='抖音渠道')].groupby(by=['search_time']).agg({'order_number': 'count', 'contract_price': 'sum','total_receivable':'sum','purchase_amount':'sum'})
# df_mon.columns = df_mon.columns.droplevel(1) 
df_mon_dy = df_fq_concat[(df_fq_concat.当前期数==1)& (df_fq_concat.归属渠道=='抖音渠道')].groupby(by=['search_time']).agg({'order_number': 'count', 'contract_price': 'sum','total_receivable':'sum','purchase_amount':'sum'})
df_mon_dy

# note 20241209增加以下两个
# （5）二手非抖音
df_mon_old = df_fq_concat[(df_fq_concat.当前期数==1)&(df_fq_concat["是否二手"]=="二手")&(df_fq_concat.归属渠道!='抖音渠道')].groupby(by=['search_time']).agg({'order_number': 'count', 'contract_price': 'sum','total_receivable':'sum','purchase_amount':'sum'})
df_mon_old

# （6）单人会话
df_mon_drhh = df_fq_concat[(df_fq_concat.当前期数==1)&(df_fq_concat.归属渠道=='单人聊天会话中的小程序消息卡片（分享）')].groupby(by=['search_time']).agg({'order_number': 'count', 'contract_price': 'sum','total_receivable':'sum','purchase_amount':'sum'})
df_mon_drhh

,order_number,contract_price,total_receivable,purchase_amount
search_time,,,,
2022-07,22,195693.00,165422.04,125405.00
2022-08,56,527606.04,448214.72,358451.00
2022-09,44,402654.00,351061.75,277982.75
2022-10,41,337968.00,297199.52,249543.09
2022-11,43,440057.00,383153.68,344258.81
2022-12,66,740353.20,660888.72,566276.59
2023-01,221,2173455.52,1911671.70,1533459.00
2023-02,332,3218625.44,2723964.72,2181552.82
2023-03,316,3063400.23,2529539.71,2083021.00


,order_number,contract_price,total_receivable,purchase_amount
search_time,,,,
2022-09,5,34573.00,30365.41,24742.68
2022-10,10,65644.00,54724.99,48835.00
2022-11,10,89430.00,70776.64,68730.22
2022-12,2,10106.20,9409.65,7657.00
2023-01,18,158715.70,136059.35,110070.00
2023-02,38,306451.62,210243.72,206619.00
2023-03,61,550399.00,401850.75,374912.00
2023-04,240,2442118.00,1761560.32,1628122.00
2023-05,145,1413669.00,1027164.43,905764.00


,order_number,contract_price,total_receivable,purchase_amount
search_time,,,,
2022-07,7,67151.00,60701.86,46581.00
2022-08,22,210198.00,177847.99,145114.00
2022-09,8,96962.00,85808.45,67126.34
2022-10,6,59404.00,54189.52,44442.63
2022-11,10,113610.00,102084.58,88153.59
2022-12,37,424727.00,388321.05,324164.59
2023-01,134,1337111.40,1170746.57,944943.00
2023-02,200,2015258.42,1740682.21,1369414.82
2023-03,150,1468754.23,1235016.72,1002057.00


,order_number,contract_price,total_receivable,purchase_amount
search_time,,,,
2024-04,4,26875.0,21921.15,15930.0
2024-05,3,17517.0,17877.71,12690.0
2024-06,49,306191.0,308052.66,220205.0
2024-07,109,623891.0,632720.24,443358.0
2024-08,65,808429.0,660671.06,271022.0
2024-09,8,41644.0,41864.08,30173.0
2024-11,3,23037.0,13355.48,9865.0
2024-12,52,288129.0,289964.72,116135.0
2025-01,51,225493.0,261177.55,125877.0


,order_number,contract_price,total_receivable,purchase_amount
search_time,,,,
2022-07,2,12287.00,10470.16,8350.00
2022-08,4,27056.04,26022.84,19130.00
2022-09,9,65456.00,57834.71,48327.68
2022-10,9,83354.00,67603.42,62825.00
2022-11,9,91006.00,69405.89,69285.22
2022-12,3,21029.00,18383.30,13685.00
2023-01,17,143408.00,118972.28,89642.00
2023-02,30,234615.00,193137.16,147754.00
2023-03,44,347452.00,280435.60,226883.00


,order_number,contract_price,total_receivable,purchase_amount
search_time,,,,
2022-10,1,2469.0,2421.07,1620.0
2022-12,1,12869.0,11663.60,10508.0
2023-02,3,36007.0,33735.33,24698.0
2023-03,2,20408.0,18561.74,13995.0
2023-04,1,10399.0,10399.00,6840.0
2023-06,8,67982.0,64428.66,45910.0
2023-07,6,64864.0,57489.55,41960.0
2023-08,10,128667.0,105045.39,84769.0
2023-09,17,206905.7,171490.69,140380.0


## 4_13.vintage 4+ 订单 口径

In [166]:
# note 20241127 df_mon_nody 改为 df_mon2
# （1）总体非抖音df_mob_total[(df_mob_total.overdue_days_4==1) & (df_mob_total.归属渠道=='抖音渠道')]
vintage_df_d4 = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1) & (df_mob_total.归属渠道!='抖音渠道')],index='下单月份',columns='MOB',values='overdue_days_4',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d4 = vintage_df_d4[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
# vintage_df_d4
vintage_divide_d4 = vintage_df_d4.divide(df_mon2["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d4['订单量'] = df_mon2["order_number"]
vintage_divide_d4[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （2）芝麻租物渠道
vintage_df_d4_zm = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1) & (df_mob_total.归属渠道=='芝麻租物')],index='下单月份',columns='MOB',values='overdue_days_4',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d4_zm = vintage_df_d4_zm[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',

vintage_divide_d4_zm = vintage_df_d4_zm.divide(df_mon_zm["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d4_zm['订单量'] = df_mon_zm["order_number"]
vintage_divide_d4_zm[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （3）搜索渠道
vintage_df_d4_ss = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1) & (df_mob_total.归属渠道=='搜索渠道')],index='下单月份',columns='MOB',values='overdue_days_4',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d4_ss = vintage_df_d4_ss[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',

vintage_divide_d4_ss = vintage_df_d4_ss.divide(df_mon_ss["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d4_ss['订单量'] = df_mon_ss["order_number"]
vintage_divide_d4_ss[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （4）抖音渠道
vintage_df_d4_dy = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1) & (df_mob_total.归属渠道=='抖音渠道')],index='下单月份',columns='MOB',values='overdue_days_4',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d4_dy = vintage_df_d4_dy[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9']]  #'下单月份','mob_0',
# vintage_df_d4_dy = vintage_df_d4_dy[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d4_dy = vintage_df_d4_dy.divide(df_mon_dy["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d4_dy['订单量'] = df_mon_dy["order_number"]
# vintage_divide_d4_dy[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]
vintage_divide_d4_dy[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9']]

# （5）二手非抖音
vintage_df_d4_old = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1)&(df_mob_total["是否二手"]=="二手")&(df_mob_total.归属渠道!='抖音渠道')],index='下单月份',columns='MOB',values='overdue_days_4',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d4_old = vintage_df_d4_old[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d4_old = vintage_df_d4_old.divide(df_mon_old["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d4_old['订单量'] = df_mon_old["order_number"]
vintage_divide_d4_old[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （6）单人会话
vintage_df_d4_drhh = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1)&(df_mob_total.归属渠道=='单人聊天会话中的小程序消息卡片（分享）')],index='下单月份',columns='MOB',values='overdue_days_4',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d4_drhh = vintage_df_d4_drhh[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d4_drhh = vintage_df_d4_drhh.divide(df_mon_drhh["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d4_drhh['订单量'] = df_mon_drhh["order_number"]
vintage_divide_d4_drhh[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,22,9.09%,13.64%,27.27%,31.82%,36.36%,36.36%,36.36%,36.36%,40.91%,40.91%,36.36%,36.36%
2022-08,56,14.29%,21.43%,26.79%,28.57%,30.36%,35.71%,37.50%,41.07%,41.07%,41.07%,41.07%,39.29%
2022-09,44,6.82%,11.36%,18.18%,25.00%,29.55%,27.27%,25.00%,27.27%,27.27%,22.73%,22.73%,22.73%
2022-10,41,4.88%,4.88%,9.76%,14.63%,21.95%,19.51%,17.07%,21.95%,24.39%,19.51%,21.95%,19.51%
2022-11,43,6.98%,11.63%,9.30%,13.95%,13.95%,23.26%,23.26%,18.60%,20.93%,20.93%,18.60%,18.60%
2022-12,66,3.03%,7.58%,13.64%,19.70%,19.70%,18.18%,24.24%,25.76%,27.27%,25.76%,25.76%,24.24%
2023-01,221,5.43%,11.31%,14.48%,15.84%,20.81%,24.89%,26.24%,28.51%,29.86%,31.22%,30.32%,29.41%
2023-02,332,7.53%,11.45%,15.66%,17.17%,20.48%,22.29%,23.19%,25.00%,24.10%,24.40%,23.49%,23.19%
2023-03,316,6.33%,12.03%,15.51%,19.94%,22.78%,25.63%,25.95%,27.53%,28.48%,30.38%,28.80%,28.16%
2023-04,634,3.63%,8.04%,13.09%,15.62%,18.30%,20.98%,22.24%,23.82%,25.08%,25.55%,25.87%,25.24%


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-09,5,nan%,20.00%,20.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%
2022-10,10,10.00%,nan%,10.00%,10.00%,10.00%,20.00%,10.00%,10.00%,10.00%,10.00%,10.00%,10.00%
2022-11,10,nan%,10.00%,10.00%,10.00%,10.00%,10.00%,30.00%,20.00%,20.00%,20.00%,20.00%,20.00%
2022-12,2,nan%,nan%,nan%,nan%,nan%,nan%,50.00%,nan%,nan%,nan%,nan%,nan%
2023-01,18,nan%,5.56%,11.11%,11.11%,11.11%,22.22%,27.78%,22.22%,27.78%,27.78%,27.78%,27.78%
2023-02,38,2.63%,5.26%,7.89%,5.26%,5.26%,10.53%,7.89%,7.89%,5.26%,5.26%,5.26%,5.26%
2023-03,61,1.64%,1.64%,4.92%,6.56%,11.48%,11.48%,14.75%,16.39%,16.39%,18.03%,18.03%,18.03%
2023-04,240,2.92%,6.25%,9.58%,10.42%,11.67%,16.25%,17.08%,18.33%,18.75%,19.58%,19.58%,19.58%
2023-05,145,nan%,3.45%,8.28%,7.59%,10.34%,8.97%,15.17%,17.93%,19.31%,17.93%,20.00%,18.62%
2023-06,263,2.66%,6.08%,10.65%,14.45%,15.97%,18.25%,21.29%,23.95%,24.33%,27.00%,25.86%,25.48%


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,7,14.29%,14.29%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,57.14%,42.86%,42.86%,42.86%
2022-08,22,13.64%,18.18%,22.73%,22.73%,22.73%,31.82%,36.36%,45.45%,36.36%,36.36%,36.36%,36.36%
2022-09,8,37.50%,50.00%,62.50%,62.50%,62.50%,62.50%,50.00%,62.50%,62.50%,62.50%,62.50%,62.50%
2022-10,6,nan%,16.67%,16.67%,33.33%,33.33%,16.67%,16.67%,16.67%,16.67%,16.67%,16.67%,16.67%
2022-11,10,30.00%,40.00%,30.00%,40.00%,40.00%,50.00%,40.00%,50.00%,50.00%,50.00%,50.00%,50.00%
2022-12,37,2.70%,10.81%,16.22%,21.62%,24.32%,24.32%,27.03%,32.43%,29.73%,29.73%,29.73%,27.03%
2023-01,134,5.97%,12.69%,16.42%,15.67%,20.15%,23.88%,25.37%,27.61%,27.61%,29.85%,28.36%,26.87%
2023-02,200,10.00%,14.50%,18.50%,20.50%,24.00%,27.00%,28.50%,30.00%,29.00%,29.50%,28.50%,28.50%
2023-03,150,10.00%,18.67%,24.00%,27.33%,30.00%,33.33%,32.67%,34.00%,34.67%,36.67%,34.67%,34.67%
2023-04,191,4.19%,10.47%,16.75%,18.85%,21.47%,20.94%,25.13%,26.70%,28.80%,28.80%,30.89%,29.32%


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9
2024-04,4,nan%,nan%,25.00%,25.00%,25.00%,50.00%,50.00%,50.00%,50.00%
2024-05,3,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-06,49,20.41%,26.53%,26.53%,38.78%,40.82%,44.90%,48.98%,nan%,nan%
2024-07,109,15.60%,24.77%,33.94%,39.45%,39.45%,43.12%,nan%,nan%,nan%
2024-08,65,29.23%,40.00%,40.00%,30.77%,27.69%,nan%,nan%,nan%,nan%
2024-09,8,25.00%,37.50%,50.00%,62.50%,nan%,nan%,nan%,nan%,nan%
2024-11,3,nan%,33.33%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-12,52,17.31%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2025-01,51,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,2,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,50.00%,nan%,nan%,nan%
2022-08,4,nan%,nan%,nan%,nan%,nan%,nan%,25.00%,25.00%,50.00%,50.00%,25.00%,25.00%
2022-09,9,nan%,11.11%,44.44%,33.33%,33.33%,33.33%,22.22%,44.44%,44.44%,33.33%,33.33%,33.33%
2022-10,9,nan%,nan%,nan%,nan%,nan%,11.11%,11.11%,22.22%,22.22%,11.11%,22.22%,11.11%
2022-11,9,nan%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%
2022-12,3,nan%,nan%,33.33%,nan%,33.33%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-01,17,5.88%,5.88%,11.76%,5.88%,17.65%,17.65%,17.65%,23.53%,29.41%,23.53%,29.41%,23.53%
2023-02,30,13.33%,13.33%,16.67%,23.33%,16.67%,26.67%,30.00%,30.00%,30.00%,33.33%,26.67%,26.67%
2023-03,44,4.55%,11.36%,15.91%,20.45%,25.00%,31.82%,29.55%,31.82%,31.82%,31.82%,31.82%,29.55%
2023-04,74,1.35%,4.05%,6.76%,12.16%,12.16%,12.16%,13.51%,9.46%,10.81%,12.16%,13.51%,10.81%


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-10,1,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2022-12,1,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-02,3,nan%,nan%,33.33%,33.33%,33.33%,33.33%,33.33%,33.33%,33.33%,33.33%,33.33%,33.33%
2023-03,2,50.00%,50.00%,50.00%,50.00%,50.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%
2023-04,1,nan%,nan%,nan%,nan%,nan%,100.00%,100.00%,nan%,nan%,nan%,nan%,nan%
2023-06,8,nan%,nan%,nan%,12.50%,25.00%,25.00%,25.00%,37.50%,50.00%,50.00%,37.50%,37.50%
2023-07,6,nan%,nan%,nan%,16.67%,33.33%,50.00%,50.00%,33.33%,33.33%,33.33%,33.33%,33.33%
2023-08,10,20.00%,20.00%,20.00%,20.00%,30.00%,30.00%,40.00%,50.00%,50.00%,40.00%,40.00%,40.00%
2023-09,17,5.88%,5.88%,11.76%,17.65%,23.53%,23.53%,23.53%,29.41%,35.29%,35.29%,35.29%,35.29%
2023-10,32,3.12%,9.38%,18.75%,18.75%,31.25%,31.25%,31.25%,37.50%,31.25%,28.12%,28.12%,28.12%


## 4_13.vintage 7+ 订单 口径

In [165]:
# note 20241127 df_mon 改为 df_mon2，分母保持一致
# （1）总体
vintage_df_d7 = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1) & (df_mob_total.归属渠道!='抖音渠道')],index='下单月份',columns='MOB',values='overdue_days_7',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d7 = vintage_df_d7[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
# vintage_df_d7
vintage_divide_d7 = vintage_df_d7.divide(df_mon2["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d7['订单量'] = df_mon2["order_number"]
vintage_divide_d7[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （2）蚂蚁租物
vintage_df_d7_zm = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1) & (df_mob_total.归属渠道=='芝麻租物')],index='下单月份',columns='MOB',values='overdue_days_7',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d7_zm = vintage_df_d7_zm[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',

vintage_divide_d7_zm = vintage_df_d7_zm.divide(df_mon_zm["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d7_zm['订单量'] = df_mon_zm["order_number"]
vintage_divide_d7_zm[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （3）搜索渠道
vintage_df_d7_ss = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1) & (df_mob_total.归属渠道=='搜索渠道')],index='下单月份',columns='MOB',values='overdue_days_7',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d7_ss = vintage_df_d7_ss[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',

vintage_divide_d7_ss = vintage_df_d7_ss.divide(df_mon_ss["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d7_ss['订单量'] = df_mon_ss["order_number"]
vintage_divide_d7_ss[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （4）抖音渠道
vintage_df_d7_dy = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1) & (df_mob_total.归属渠道=='抖音渠道')],index='下单月份',columns='MOB',values='overdue_days_7',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d7_dy = vintage_df_d7_dy[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9']]  #'下单月份','mob_0',
# vintage_df_d7_dy = vintage_df_d7_dy[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d7_dy = vintage_df_d7_dy.divide(df_mon_dy["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d7_dy['订单量'] = df_mon_dy["order_number"]
# vintage_divide_d7_dy[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]
vintage_divide_d7_dy[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9']]

# （5）二手非抖音
vintage_df_d7_old = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1)&(df_mob_total["是否二手"]=="二手")&(df_mob_total.归属渠道!='抖音渠道')],index='下单月份',columns='MOB',values='overdue_days_7',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d7_old = vintage_df_d7_old[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d7_old = vintage_df_d7_old.divide(df_mon_old["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d7_old['订单量'] = df_mon_old["order_number"]
vintage_divide_d7_old[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （6）单人会话
vintage_df_d7_drhh = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1)&(df_mob_total.归属渠道=='单人聊天会话中的小程序消息卡片（分享）')],index='下单月份',columns='MOB',values='overdue_days_7',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d7_drhh = vintage_df_d7_drhh[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d7_drhh = vintage_df_d7_drhh.divide(df_mon_drhh["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d7_drhh['订单量'] = df_mon_drhh["order_number"]
vintage_divide_d7_drhh[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,22,4.55%,13.64%,27.27%,31.82%,36.36%,36.36%,36.36%,36.36%,40.91%,40.91%,36.36%,36.36%
2022-08,56,10.71%,21.43%,26.79%,28.57%,30.36%,33.93%,37.50%,41.07%,41.07%,41.07%,41.07%,39.29%
2022-09,44,6.82%,11.36%,18.18%,20.45%,29.55%,27.27%,25.00%,25.00%,27.27%,22.73%,22.73%,22.73%
2022-10,41,4.88%,4.88%,7.32%,14.63%,21.95%,19.51%,17.07%,19.51%,24.39%,19.51%,21.95%,19.51%
2022-11,43,6.98%,11.63%,9.30%,11.63%,11.63%,20.93%,23.26%,18.60%,18.60%,18.60%,18.60%,18.60%
2022-12,66,1.52%,7.58%,12.12%,19.70%,18.18%,18.18%,24.24%,25.76%,27.27%,25.76%,25.76%,24.24%
2023-01,221,4.98%,10.86%,14.48%,15.84%,19.00%,24.43%,24.43%,27.60%,29.86%,30.77%,30.32%,29.41%
2023-02,332,6.93%,10.54%,14.76%,16.87%,19.88%,21.39%,22.89%,24.70%,23.80%,24.10%,23.49%,23.19%
2023-03,316,4.43%,10.76%,13.92%,18.99%,21.84%,23.73%,24.68%,25.63%,28.48%,30.38%,28.80%,28.16%
2023-04,634,3.31%,7.26%,11.99%,14.98%,17.67%,20.50%,21.61%,23.34%,24.61%,25.24%,25.87%,25.24%


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-09,5,nan%,20.00%,20.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%
2022-10,10,10.00%,nan%,10.00%,10.00%,10.00%,20.00%,10.00%,10.00%,10.00%,10.00%,10.00%,10.00%
2022-11,10,nan%,10.00%,10.00%,10.00%,10.00%,10.00%,30.00%,20.00%,20.00%,20.00%,20.00%,20.00%
2022-12,2,nan%,nan%,nan%,nan%,nan%,nan%,50.00%,nan%,nan%,nan%,nan%,nan%
2023-01,18,nan%,nan%,11.11%,11.11%,11.11%,22.22%,22.22%,22.22%,27.78%,27.78%,27.78%,27.78%
2023-02,38,nan%,2.63%,2.63%,2.63%,5.26%,7.89%,5.26%,7.89%,5.26%,5.26%,5.26%,5.26%
2023-03,61,1.64%,1.64%,4.92%,6.56%,11.48%,9.84%,13.11%,16.39%,16.39%,18.03%,18.03%,18.03%
2023-04,240,2.92%,5.83%,8.75%,10.42%,11.67%,16.25%,16.67%,18.33%,18.75%,19.58%,19.58%,19.58%
2023-05,145,nan%,3.45%,7.59%,7.59%,8.97%,8.97%,15.17%,17.24%,19.31%,16.55%,19.31%,18.62%
2023-06,263,1.90%,4.94%,9.89%,12.93%,15.21%,17.49%,21.29%,22.43%,24.33%,26.62%,25.48%,25.48%


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,7,14.29%,14.29%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,57.14%,42.86%,42.86%,42.86%
2022-08,22,13.64%,18.18%,22.73%,22.73%,22.73%,31.82%,36.36%,45.45%,36.36%,36.36%,36.36%,36.36%
2022-09,8,37.50%,50.00%,62.50%,62.50%,62.50%,62.50%,50.00%,62.50%,62.50%,62.50%,62.50%,62.50%
2022-10,6,nan%,16.67%,16.67%,33.33%,33.33%,16.67%,16.67%,16.67%,16.67%,16.67%,16.67%,16.67%
2022-11,10,30.00%,40.00%,30.00%,40.00%,40.00%,50.00%,40.00%,50.00%,50.00%,50.00%,50.00%,50.00%
2022-12,37,2.70%,10.81%,16.22%,21.62%,24.32%,24.32%,27.03%,32.43%,29.73%,29.73%,29.73%,27.03%
2023-01,134,5.22%,12.69%,16.42%,15.67%,17.16%,23.13%,23.13%,26.12%,27.61%,29.10%,28.36%,26.87%
2023-02,200,9.50%,13.50%,18.00%,20.50%,23.00%,26.00%,28.50%,30.00%,29.00%,29.00%,28.50%,28.50%
2023-03,150,7.33%,17.33%,22.00%,26.00%,29.33%,32.00%,32.00%,32.00%,34.67%,36.67%,34.67%,34.67%
2023-04,191,3.14%,8.38%,15.71%,17.80%,19.90%,19.90%,23.56%,26.18%,28.80%,28.27%,30.89%,29.32%


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9
2024-04,4,nan%,nan%,25.00%,25.00%,25.00%,25.00%,50.00%,50.00%,50.00%
2024-05,3,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-06,49,14.29%,26.53%,24.49%,36.73%,40.82%,42.86%,46.94%,nan%,nan%
2024-07,109,12.84%,24.77%,33.94%,39.45%,39.45%,43.12%,nan%,nan%,nan%
2024-08,65,27.69%,38.46%,36.92%,29.23%,27.69%,nan%,nan%,nan%,nan%
2024-09,8,25.00%,37.50%,50.00%,62.50%,nan%,nan%,nan%,nan%,nan%
2024-11,3,nan%,33.33%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-12,52,13.46%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2025-01,51,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,2,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,50.00%,nan%,nan%,nan%
2022-08,4,nan%,nan%,nan%,nan%,nan%,nan%,25.00%,25.00%,50.00%,50.00%,25.00%,25.00%
2022-09,9,nan%,11.11%,44.44%,33.33%,33.33%,33.33%,22.22%,33.33%,44.44%,33.33%,33.33%,33.33%
2022-10,9,nan%,nan%,nan%,nan%,nan%,11.11%,11.11%,11.11%,22.22%,11.11%,22.22%,11.11%
2022-11,9,nan%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%
2022-12,3,nan%,nan%,33.33%,nan%,33.33%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-01,17,5.88%,5.88%,11.76%,5.88%,11.76%,17.65%,11.76%,17.65%,29.41%,23.53%,29.41%,23.53%
2023-02,30,10.00%,10.00%,13.33%,20.00%,16.67%,23.33%,26.67%,30.00%,30.00%,33.33%,26.67%,26.67%
2023-03,44,2.27%,11.36%,15.91%,20.45%,25.00%,31.82%,29.55%,29.55%,31.82%,31.82%,31.82%,29.55%
2023-04,74,1.35%,4.05%,5.41%,12.16%,12.16%,12.16%,13.51%,9.46%,10.81%,12.16%,13.51%,10.81%


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-10,1,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2022-12,1,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-02,3,nan%,nan%,33.33%,33.33%,33.33%,33.33%,33.33%,33.33%,33.33%,33.33%,33.33%,33.33%
2023-03,2,nan%,50.00%,50.00%,50.00%,50.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%
2023-04,1,nan%,nan%,nan%,nan%,nan%,100.00%,100.00%,nan%,nan%,nan%,nan%,nan%
2023-06,8,nan%,nan%,nan%,12.50%,25.00%,25.00%,25.00%,37.50%,50.00%,50.00%,37.50%,37.50%
2023-07,6,nan%,nan%,nan%,16.67%,33.33%,33.33%,50.00%,33.33%,33.33%,33.33%,33.33%,33.33%
2023-08,10,20.00%,20.00%,20.00%,20.00%,30.00%,30.00%,40.00%,50.00%,50.00%,40.00%,40.00%,40.00%
2023-09,17,5.88%,5.88%,11.76%,17.65%,23.53%,17.65%,23.53%,29.41%,35.29%,35.29%,35.29%,35.29%
2023-10,32,nan%,6.25%,15.62%,18.75%,28.12%,28.12%,31.25%,37.50%,31.25%,28.12%,28.12%,28.12%


## 4_14.vintage 15+ 订单 口径

In [167]:
# note 20241127 df_mon 改为 df_mon2，分母保持一致
# （1）总体
vintage_df_d15 = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1) & (df_mob_total.归属渠道!='抖音渠道')],index='下单月份',columns='MOB',values='overdue_days_15',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d15 = vintage_df_d15[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
# vintage_df_d15
vintage_divide_d15 = vintage_df_d15.divide(df_mon2["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d15['订单量'] = df_mon2["order_number"]
vintage_divide_d15[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （2）蚂蚁租物
vintage_df_d15_zm = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1) & (df_mob_total.归属渠道=='芝麻租物')],index='下单月份',columns='MOB',values='overdue_days_15',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d15_zm = vintage_df_d15_zm[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',

vintage_divide_d15_zm = vintage_df_d15_zm.divide(df_mon_zm["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d15_zm['订单量'] = df_mon_zm["order_number"]
vintage_divide_d15_zm[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （3）搜素渠道
vintage_df_d15_ss = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1) & (df_mob_total.归属渠道=='搜索渠道')],index='下单月份',columns='MOB',values='overdue_days_15',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d15_ss = vintage_df_d15_ss[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',

vintage_divide_d15_ss = vintage_df_d15_ss.divide(df_mon_ss["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d15_ss['订单量'] = df_mon_ss["order_number"]
vintage_divide_d15_ss[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （4）抖音渠道
vintage_df_d15_dy = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1) & (df_mob_total.归属渠道=='抖音渠道')],index='下单月份',columns='MOB',values='overdue_days_15',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d15_dy = vintage_df_d15_dy[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9']]  #'下单月份','mob_0',
# vintage_df_d15_dy = vintage_df_d15_dy[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d15_dy = vintage_df_d15_dy.divide(df_mon_dy["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d15_dy['订单量'] = df_mon_dy["order_number"]
# vintage_divide_d15_dy[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]
vintage_divide_d15_dy[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9']]

# （5）二手非抖音
vintage_df_d15_old = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1)&(df_mob_total["是否二手"]=="二手")&(df_mob_total.归属渠道!='抖音渠道')],index='下单月份',columns='MOB',values='overdue_days_15',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d15_old = vintage_df_d15_old[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d15_old = vintage_df_d15_old.divide(df_mon_old["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d15_old['订单量'] = df_mon_old["order_number"]
vintage_divide_d15_old[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （6）单人会话
vintage_df_d15_drhh = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1)&(df_mob_total.归属渠道=='单人聊天会话中的小程序消息卡片（分享）')],index='下单月份',columns='MOB',values='overdue_days_15',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d15_drhh = vintage_df_d15_drhh[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d15_drhh = vintage_df_d15_drhh.divide(df_mon_drhh["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d15_drhh['订单量'] = df_mon_drhh["order_number"]
vintage_divide_d15_drhh[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,22,nan%,9.09%,27.27%,27.27%,27.27%,31.82%,36.36%,36.36%,36.36%,36.36%,36.36%,36.36%
2022-08,56,5.36%,14.29%,19.64%,23.21%,28.57%,33.93%,37.50%,39.29%,39.29%,41.07%,41.07%,39.29%
2022-09,44,2.27%,9.09%,18.18%,18.18%,27.27%,27.27%,25.00%,25.00%,25.00%,22.73%,22.73%,22.73%
2022-10,41,4.88%,4.88%,7.32%,14.63%,19.51%,19.51%,17.07%,19.51%,24.39%,19.51%,21.95%,19.51%
2022-11,43,4.65%,11.63%,9.30%,11.63%,11.63%,18.60%,23.26%,18.60%,18.60%,18.60%,18.60%,18.60%
2022-12,66,nan%,7.58%,7.58%,19.70%,16.67%,16.67%,22.73%,21.21%,27.27%,25.76%,25.76%,24.24%
2023-01,221,4.07%,9.95%,11.76%,15.38%,18.55%,23.98%,23.98%,26.70%,29.41%,28.96%,30.32%,29.41%
2023-02,332,4.22%,8.73%,13.55%,14.16%,18.98%,19.88%,21.08%,23.80%,23.80%,23.80%,23.49%,23.19%
2023-03,316,2.85%,8.54%,11.71%,16.46%,17.72%,22.15%,22.78%,23.73%,27.22%,29.43%,28.80%,28.16%
2023-04,634,2.21%,5.68%,10.09%,13.25%,15.62%,19.09%,20.50%,21.77%,23.66%,23.19%,25.39%,25.24%


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-09,5,nan%,nan%,20.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%
2022-10,10,10.00%,nan%,10.00%,10.00%,10.00%,20.00%,10.00%,10.00%,10.00%,10.00%,10.00%,10.00%
2022-11,10,nan%,10.00%,10.00%,10.00%,10.00%,10.00%,30.00%,20.00%,20.00%,20.00%,20.00%,20.00%
2022-12,2,nan%,nan%,nan%,nan%,nan%,nan%,50.00%,nan%,nan%,nan%,nan%,nan%
2023-01,18,nan%,nan%,11.11%,11.11%,11.11%,22.22%,22.22%,22.22%,27.78%,27.78%,27.78%,27.78%
2023-02,38,nan%,2.63%,2.63%,2.63%,5.26%,7.89%,5.26%,7.89%,5.26%,5.26%,5.26%,5.26%
2023-03,61,nan%,1.64%,3.28%,4.92%,6.56%,9.84%,13.11%,14.75%,14.75%,18.03%,18.03%,18.03%
2023-04,240,2.08%,3.75%,7.08%,9.17%,10.83%,14.58%,15.83%,16.67%,18.75%,18.75%,19.58%,19.58%
2023-05,145,nan%,2.07%,7.59%,7.59%,8.97%,8.28%,15.17%,15.86%,17.24%,15.86%,19.31%,18.62%
2023-06,263,1.14%,4.56%,8.37%,12.17%,14.83%,16.73%,20.53%,21.67%,23.57%,25.48%,25.48%,25.48%


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,7,nan%,14.29%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%
2022-08,22,9.09%,9.09%,13.64%,13.64%,22.73%,31.82%,36.36%,40.91%,36.36%,36.36%,36.36%,36.36%
2022-09,8,12.50%,50.00%,62.50%,62.50%,62.50%,62.50%,50.00%,62.50%,62.50%,62.50%,62.50%,62.50%
2022-10,6,nan%,16.67%,16.67%,33.33%,33.33%,16.67%,16.67%,16.67%,16.67%,16.67%,16.67%,16.67%
2022-11,10,20.00%,40.00%,30.00%,40.00%,40.00%,50.00%,40.00%,50.00%,50.00%,50.00%,50.00%,50.00%
2022-12,37,nan%,10.81%,10.81%,21.62%,21.62%,21.62%,24.32%,24.32%,29.73%,29.73%,29.73%,27.03%
2023-01,134,3.73%,11.19%,11.94%,15.67%,16.42%,22.39%,22.39%,24.63%,26.87%,26.12%,28.36%,26.87%
2023-02,200,5.00%,11.50%,17.00%,17.00%,22.50%,24.00%,26.50%,29.50%,29.00%,28.50%,28.50%,28.50%
2023-03,150,5.33%,13.33%,18.00%,24.00%,24.67%,28.67%,28.67%,29.33%,34.00%,36.00%,34.67%,34.67%
2023-04,191,2.09%,7.33%,12.04%,14.66%,16.23%,18.85%,21.99%,23.56%,26.70%,24.08%,29.32%,29.32%


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9
2024-04,4,nan%,nan%,nan%,25.00%,25.00%,25.00%,50.00%,50.00%,50.00%
2024-05,3,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-06,49,2.04%,22.45%,24.49%,30.61%,36.73%,40.82%,44.90%,nan%,nan%
2024-07,109,9.17%,24.77%,33.94%,36.70%,39.45%,42.20%,nan%,nan%,nan%
2024-08,65,18.46%,36.92%,36.92%,26.15%,26.15%,nan%,nan%,nan%,nan%
2024-09,8,12.50%,25.00%,37.50%,37.50%,nan%,nan%,nan%,nan%,nan%
2024-11,3,nan%,33.33%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-12,52,7.69%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2025-01,51,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,2,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2022-08,4,nan%,nan%,nan%,nan%,nan%,nan%,25.00%,25.00%,50.00%,50.00%,25.00%,25.00%
2022-09,9,nan%,11.11%,44.44%,33.33%,33.33%,33.33%,22.22%,33.33%,33.33%,33.33%,33.33%,33.33%
2022-10,9,nan%,nan%,nan%,nan%,nan%,11.11%,11.11%,11.11%,22.22%,11.11%,22.22%,11.11%
2022-11,9,nan%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%
2022-12,3,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-01,17,5.88%,5.88%,11.76%,5.88%,11.76%,17.65%,11.76%,17.65%,29.41%,23.53%,29.41%,23.53%
2023-02,30,3.33%,10.00%,13.33%,16.67%,16.67%,20.00%,23.33%,30.00%,30.00%,33.33%,26.67%,26.67%
2023-03,44,nan%,6.82%,9.09%,18.18%,20.45%,27.27%,27.27%,25.00%,29.55%,31.82%,31.82%,29.55%
2023-04,74,1.35%,1.35%,4.05%,10.81%,9.46%,10.81%,10.81%,9.46%,9.46%,8.11%,12.16%,10.81%


MOB,订单量,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-10,1,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2022-12,1,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-02,3,nan%,nan%,nan%,nan%,33.33%,nan%,nan%,33.33%,33.33%,33.33%,33.33%,33.33%
2023-03,2,nan%,50.00%,50.00%,50.00%,50.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%
2023-04,1,nan%,nan%,nan%,nan%,nan%,100.00%,100.00%,nan%,nan%,nan%,nan%,nan%
2023-06,8,nan%,nan%,nan%,12.50%,25.00%,25.00%,25.00%,37.50%,50.00%,50.00%,37.50%,37.50%
2023-07,6,nan%,nan%,nan%,16.67%,33.33%,33.33%,50.00%,33.33%,33.33%,33.33%,33.33%,33.33%
2023-08,10,nan%,10.00%,20.00%,20.00%,30.00%,30.00%,40.00%,50.00%,50.00%,40.00%,40.00%,40.00%
2023-09,17,nan%,5.88%,5.88%,11.76%,23.53%,17.65%,23.53%,29.41%,29.41%,35.29%,35.29%,35.29%
2023-10,32,nan%,6.25%,12.50%,12.50%,25.00%,28.12%,31.25%,34.38%,31.25%,28.12%,28.12%,28.12%


## 4_15.vintage 31+ 订单 口径

In [168]:
# note 20241127 df_mon 改为 df_mon2，分母保持一致
# （1）总体
vintage_df_d31 = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1) & (df_mob_total.归属渠道!='抖音渠道')],index='下单月份',columns='MOB',values='overdue_days_31',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d31 = vintage_df_d31[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
# vintage_df_d31
vintage_divide_d31 = vintage_df_d31.divide(df_mon2["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d31['订单量'] = df_mon2["order_number"]
vintage_divide_d31[['订单量','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'mob_1',

# （2）蚂蚁租物
vintage_df_d31_zm = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1) & (df_mob_total.归属渠道=='芝麻租物')],index='下单月份',columns='MOB',values='overdue_days_31',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d31_zm = vintage_df_d31_zm[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',

vintage_divide_d31_zm = vintage_df_d31_zm.divide(df_mon_zm["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d31_zm['订单量'] = df_mon_zm["order_number"]
vintage_divide_d31_zm[['订单量','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']] # ,'mob_1'

# （3）搜素渠道
vintage_df_d31_ss = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1) & (df_mob_total.归属渠道=='搜索渠道')],index='下单月份',columns='MOB',values='overdue_days_31',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_df_d31_ss = vintage_df_d31_ss[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0','mob_1',

vintage_divide_d31_ss = vintage_df_d31_ss.divide(df_mon_ss["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d31_ss['订单量'] = df_mon_ss["order_number"]
vintage_divide_d31_ss[['订单量','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （4）抖音渠道
vintage_df_d31_dy = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1) & (df_mob_total.归属渠道=='抖音渠道')],index='下单月份',columns='MOB',values='overdue_days_31',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d31_dy = vintage_df_d31_dy[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9']]  #'下单月份','mob_0',
# vintage_df_d31_dy = vintage_df_d31_dy[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d31_dy = vintage_df_d31_dy.divide(df_mon_dy["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d31_dy['订单量'] = df_mon_dy["order_number"]
# vintage_divide_d31_dy[['订单量','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]
vintage_divide_d31_dy[['订单量','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9']]

# （5）二手非抖音
vintage_df_d31_old = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1)&(df_mob_total["是否二手"]=="二手")&(df_mob_total.归属渠道!='抖音渠道')],index='下单月份',columns='MOB',values='overdue_days_31',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d31_old = vintage_df_d31_old[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d31_old = vintage_df_d31_old.divide(df_mon_old["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d31_old['订单量'] = df_mon_old["order_number"]
vintage_divide_d31_old[['订单量','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （6）单人会话
vintage_df_d31_drhh = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1)&(df_mob_total.归属渠道=='单人聊天会话中的小程序消息卡片（分享）')],index='下单月份',columns='MOB',values='overdue_days_31',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_df_d31_drhh = vintage_df_d31_drhh[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_d31_drhh = vintage_df_d31_drhh.divide(df_mon_drhh["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_d31_drhh['订单量'] = df_mon_drhh["order_number"]
vintage_divide_d31_drhh[['订单量','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

MOB,订单量,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,22,9.09%,27.27%,27.27%,27.27%,31.82%,36.36%,36.36%,36.36%,36.36%,36.36%,36.36%
2022-08,56,12.50%,17.86%,21.43%,26.79%,28.57%,32.14%,37.50%,35.71%,41.07%,39.29%,39.29%
2022-09,44,6.82%,11.36%,15.91%,25.00%,25.00%,25.00%,22.73%,25.00%,22.73%,22.73%,22.73%
2022-10,41,4.88%,4.88%,9.76%,14.63%,17.07%,12.20%,17.07%,17.07%,19.51%,19.51%,19.51%
2022-11,43,9.30%,9.30%,11.63%,11.63%,13.95%,16.28%,18.60%,18.60%,18.60%,18.60%,18.60%
2022-12,66,4.55%,6.06%,15.15%,16.67%,16.67%,21.21%,21.21%,25.76%,25.76%,25.76%,24.24%
2023-01,221,5.43%,9.50%,12.22%,14.48%,20.36%,20.81%,23.08%,27.15%,27.60%,29.86%,29.41%
2023-02,332,6.02%,9.64%,12.65%,16.87%,16.87%,19.58%,21.69%,22.59%,22.89%,23.19%,23.19%
2023-03,316,6.33%,10.13%,14.24%,17.41%,20.25%,20.89%,22.78%,26.58%,27.85%,27.53%,28.16%
2023-04,634,3.15%,6.31%,10.25%,12.78%,15.77%,18.30%,19.72%,22.24%,22.08%,23.97%,25.24%


MOB,订单量,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-09,5,nan%,20.00%,20.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%,40.00%
2022-10,10,nan%,nan%,10.00%,10.00%,10.00%,nan%,10.00%,nan%,10.00%,10.00%,10.00%
2022-11,10,nan%,10.00%,10.00%,10.00%,10.00%,10.00%,20.00%,20.00%,20.00%,20.00%,20.00%
2022-12,2,nan%,nan%,nan%,nan%,nan%,50.00%,nan%,nan%,nan%,nan%,nan%
2023-01,18,nan%,11.11%,11.11%,11.11%,22.22%,22.22%,22.22%,27.78%,27.78%,27.78%,27.78%
2023-02,38,2.63%,2.63%,2.63%,5.26%,5.26%,5.26%,5.26%,5.26%,5.26%,5.26%,5.26%
2023-03,61,1.64%,1.64%,3.28%,6.56%,9.84%,11.48%,13.11%,14.75%,16.39%,16.39%,18.03%
2023-04,240,2.50%,5.00%,7.92%,9.17%,11.67%,13.33%,15.42%,17.92%,17.50%,18.75%,19.58%
2023-05,145,nan%,2.76%,6.21%,6.90%,7.59%,11.03%,12.41%,15.86%,15.86%,17.24%,18.62%
2023-06,263,2.28%,5.32%,9.51%,12.17%,14.83%,17.49%,19.77%,21.67%,23.95%,25.10%,25.48%


MOB,订单量,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,7,14.29%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%,42.86%
2022-08,22,9.09%,13.64%,13.64%,22.73%,22.73%,31.82%,36.36%,31.82%,36.36%,36.36%,36.36%
2022-09,8,37.50%,50.00%,62.50%,62.50%,50.00%,50.00%,50.00%,62.50%,62.50%,62.50%,62.50%
2022-10,6,16.67%,16.67%,16.67%,33.33%,16.67%,16.67%,16.67%,16.67%,16.67%,16.67%,16.67%
2022-11,10,40.00%,30.00%,40.00%,40.00%,40.00%,40.00%,50.00%,50.00%,50.00%,50.00%,50.00%
2022-12,37,5.41%,8.11%,16.22%,21.62%,21.62%,24.32%,24.32%,29.73%,29.73%,29.73%,27.03%
2023-01,134,5.22%,8.96%,12.69%,13.43%,18.66%,19.40%,22.39%,24.63%,24.63%,27.61%,26.87%
2023-02,200,8.00%,11.50%,15.50%,20.00%,20.50%,24.50%,26.50%,27.50%,27.50%,28.00%,28.50%
2023-03,150,10.00%,16.67%,22.67%,24.00%,27.33%,26.00%,28.67%,33.33%,34.67%,33.33%,34.67%
2023-04,191,3.66%,6.81%,12.57%,15.18%,16.23%,18.85%,20.94%,25.13%,24.08%,27.23%,29.32%


MOB,订单量,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9
2024-04,4,nan%,nan%,25.00%,25.00%,25.00%,50.00%,50.00%,50.00%
2024-05,3,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-06,49,16.33%,22.45%,26.53%,30.61%,38.78%,42.86%,nan%,nan%
2024-07,109,14.68%,24.77%,31.19%,35.78%,37.61%,nan%,nan%,nan%
2024-08,65,30.77%,36.92%,24.62%,26.15%,nan%,nan%,nan%,nan%
2024-09,8,25.00%,25.00%,37.50%,nan%,nan%,nan%,nan%,nan%
2024-11,3,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-12,52,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2025-01,51,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,订单量,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,2,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2022-08,4,nan%,nan%,nan%,nan%,nan%,nan%,25.00%,25.00%,50.00%,25.00%,25.00%
2022-09,9,nan%,11.11%,33.33%,33.33%,22.22%,22.22%,22.22%,33.33%,33.33%,33.33%,33.33%
2022-10,9,nan%,nan%,nan%,nan%,nan%,nan%,11.11%,nan%,11.11%,11.11%,11.11%
2022-11,9,nan%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%,11.11%
2022-12,3,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-01,17,5.88%,5.88%,5.88%,5.88%,17.65%,11.76%,11.76%,17.65%,23.53%,23.53%,23.53%
2023-02,30,10.00%,10.00%,13.33%,16.67%,16.67%,20.00%,26.67%,26.67%,33.33%,26.67%,26.67%
2023-03,44,4.55%,6.82%,13.64%,20.45%,27.27%,27.27%,25.00%,29.55%,29.55%,25.00%,29.55%
2023-04,74,1.35%,1.35%,9.46%,8.11%,8.11%,6.76%,8.11%,6.76%,8.11%,8.11%,10.81%


MOB,订单量,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-10,1,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2022-12,1,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-02,3,nan%,nan%,nan%,33.33%,nan%,nan%,33.33%,33.33%,33.33%,33.33%,33.33%
2023-03,2,50.00%,50.00%,50.00%,50.00%,50.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%
2023-04,1,nan%,nan%,nan%,nan%,100.00%,100.00%,nan%,nan%,nan%,nan%,nan%
2023-06,8,nan%,nan%,12.50%,12.50%,25.00%,25.00%,25.00%,37.50%,50.00%,37.50%,37.50%
2023-07,6,nan%,nan%,nan%,33.33%,33.33%,50.00%,33.33%,33.33%,33.33%,33.33%,33.33%
2023-08,10,10.00%,20.00%,20.00%,20.00%,30.00%,30.00%,40.00%,40.00%,40.00%,40.00%,40.00%
2023-09,17,5.88%,5.88%,11.76%,17.65%,17.65%,23.53%,23.53%,29.41%,35.29%,35.29%,35.29%
2023-10,32,nan%,6.25%,6.25%,21.88%,25.00%,28.12%,28.12%,28.12%,28.12%,28.12%,28.12%


## 4_16.vintage 4+ 金额 口径

In [75]:
df_mob_total['unpaid_money_mob'] = df_mob_total['purchase_amount'] - df_mob_total['paid_money_mob']

In [76]:
# df_mob_total.shape
# df_mob_total[:1]
# df_mob_total[df_mob_total.order_number == 'A20241103155713186'][['order_number', 'paid_money_mob', 'overdue_days_mob', '当前应付租金','MOB','purchase_amount','paid_money_mob','unpaid_money_mob']]

In [77]:
# df_mob_total[(df_mob_total['unpaid_money_mob'] < 0) ].shape
# df_mob_total[(df_mob_total['unpaid_money_mob'] < 0) & (df_mob_total.purchase_amount > 0)].shape
# df_mob_total[(df_mob_total['unpaid_money_mob'] < 0) & (df_mob_total.purchase_amount > 0)][:3]
# df_mob_total[df_mob_total.order_number == 'A202207131230485']
# df_mob_total[df_mob_total.order_number == 'A202207131230485']
# df_fq_concat[df_fq_concat.order_number == 'A2024060516202368']
# A2024060516202368
# df_mob_total[:3]

# df_mob_total[(df_mob_total['unpaid_money_mob'] < 0) ]['order_number'].nunique()
# df_mob_total[(df_mob_total['unpaid_money_mob'] < 0)&(df_mob_total.overdue_days_1==1) ]['order_number'].nunique()


In [169]:
# note 20241127 df_mon 改为 df_mon2，分母保持一致
# note 20241127 增加条件 (df_mob_total.unpaid_money_mob > 0)   
# （1）总体
# vintage_money_d1 =  pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_1==1)& (df_mob_total.unpaid_money_mob > 0)  & (df_mob_total.归属渠道!='抖音渠道')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

# vintage_money_d1 = vintage_money_d1[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',

# vintage_divide_money_d1 = vintage_money_d1.divide(df_mon["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
# vintage_divide_money_d1['采购金额'] = df_mon["purchase_amount"]
# vintage_divide_money_d1[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

vintage_money_d4 =  pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1)& (df_mob_total.unpaid_money_mob > 0)  & (df_mob_total.归属渠道!='抖音渠道')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d4 = vintage_money_d4[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
# vintage_money_d4['采购金额'] = df_mon['采购金额']
# vintage_money_d4
vintage_divide_money_d4 = vintage_money_d4.divide(df_mon2["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d4['采购金额'] = df_mon2["purchase_amount"]
vintage_divide_money_d4[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （2）芝麻租物
vintage_money_d4_zm = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1)& (df_mob_total.unpaid_money_mob > 0)  & (df_mob_total.归属渠道=='芝麻租物')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d4_zm = vintage_money_d4_zm[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0','mob_1',

vintage_divide_money_d4_zm = vintage_money_d4_zm.divide(df_mon_zm["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d4_zm['采购金额'] = df_mon_zm["purchase_amount"]
vintage_divide_money_d4_zm[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （3）搜索渠道
vintage_money_d4_ss = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1)& (df_mob_total.unpaid_money_mob > 0)  & (df_mob_total.归属渠道=='搜索渠道')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d4_ss = vintage_money_d4_ss[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0','mob_1',

vintage_divide_money_d4_ss = vintage_money_d4_ss.divide(df_mon_ss["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d4_ss['采购金额'] = df_mon_ss["purchase_amount"]
vintage_divide_money_d4_ss[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （4）抖音
vintage_money_d4_dy = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1)& (df_mob_total.unpaid_money_mob > 0)  & (df_mob_total.归属渠道=='抖音渠道')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d4_dy = vintage_money_d4_dy[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9']]  #'下单月份','mob_0','mob_1',,'mob_2',

vintage_divide_money_d4_dy = vintage_money_d4_dy.divide(df_mon_dy["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d4_dy['采购金额'] = df_mon_dy["purchase_amount"]
vintage_divide_money_d4_dy[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9']]  #,'mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12'

# （5）二手非抖音
vintage_money_d4_old = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1)& (df_mob_total.unpaid_money_mob > 0)&(df_mob_total["是否二手"]=="二手")&(df_mob_total.归属渠道!='抖音渠道')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d4_old = vintage_money_d4_old[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_money_d4_old = vintage_money_d4_old.divide(df_mon_old["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d4_old['采购金额'] = df_mon_old["purchase_amount"]
vintage_divide_money_d4_old[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （6）单人会话
vintage_money_d4_drhh = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_4==1)& (df_mob_total.unpaid_money_mob > 0)&(df_mob_total.归属渠道=='单人聊天会话中的小程序消息卡片（分享）')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d4_drhh = vintage_money_d4_drhh[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_money_d4_drhh = vintage_money_d4_drhh.divide(df_mon_drhh["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d4_drhh['采购金额'] = df_mon_drhh["purchase_amount"]
vintage_divide_money_d4_drhh[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,125405.00,8.57%,13.39%,26.63%,28.31%,29.82%,29.57%,29.57%,29.57%,30.14%,29.57%,29.57%,29.57%
2022-08,358451.00,12.39%,19.81%,24.38%,24.76%,25.57%,27.19%,26.45%,28.10%,27.12%,26.93%,27.46%,26.76%
2022-09,277982.75,7.73%,12.04%,15.15%,18.43%,21.42%,19.59%,18.61%,18.87%,18.63%,15.98%,15.98%,15.98%
2022-10,249543.09,4.33%,5.47%,7.75%,12.33%,18.57%,16.13%,14.42%,16.43%,19.37%,15.67%,17.03%,15.04%
2022-11,344258.81,6.44%,9.70%,8.05%,11.72%,10.45%,16.50%,15.52%,12.47%,13.24%,12.93%,12.12%,12.12%
2022-12,566276.59,2.36%,6.21%,9.10%,14.62%,12.97%,12.56%,15.51%,15.58%,16.29%,15.37%,15.27%,13.89%
2023-01,1533459.00,4.38%,8.36%,9.92%,10.62%,13.12%,14.40%,14.05%,14.56%,14.77%,14.54%,14.38%,14.24%
2023-02,2181552.82,6.80%,10.84%,13.69%,14.52%,16.69%,16.92%,16.82%,17.39%,16.30%,16.19%,15.69%,15.52%
2023-03,2083021.00,5.91%,10.37%,12.55%,16.12%,17.16%,18.82%,18.50%,17.96%,18.21%,18.41%,17.81%,17.71%
2023-04,4317651.50,3.49%,7.37%,11.54%,13.16%,14.83%,16.07%,16.54%,16.77%,16.82%,16.54%,16.27%,16.07%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-09,24742.68,nan%,22.84%,22.84%,26.95%,26.95%,26.95%,26.95%,26.95%,26.95%,26.95%,26.95%,26.95%
2022-10,48835.00,5.01%,nan%,4.02%,4.02%,4.02%,14.93%,10.12%,10.12%,8.53%,8.53%,7.74%,7.74%
2022-11,68730.22,nan%,8.09%,8.09%,8.09%,8.09%,8.09%,16.13%,15.16%,14.22%,14.10%,14.10%,14.10%
2022-12,7657.00,nan%,nan%,nan%,nan%,nan%,nan%,32.52%,nan%,nan%,nan%,nan%,nan%
2023-01,110070.00,nan%,5.38%,3.34%,3.34%,3.34%,9.28%,9.85%,9.28%,10.08%,10.08%,9.53%,9.53%
2023-02,206619.00,1.35%,4.84%,6.32%,4.58%,5.07%,6.68%,5.65%,6.73%,5.07%,5.07%,5.07%,5.07%
2023-03,374912.00,1.15%,1.15%,3.18%,3.85%,8.56%,8.46%,10.44%,10.86%,10.39%,11.28%,12.00%,12.00%
2023-04,1628122.00,3.00%,5.88%,8.43%,9.03%,10.09%,13.12%,13.67%,14.22%,14.39%,14.45%,14.27%,14.24%
2023-05,905764.00,nan%,3.43%,6.75%,6.09%,8.21%,7.44%,11.22%,12.39%,13.03%,11.89%,12.58%,12.15%
2023-06,1710725.00,2.98%,5.77%,9.89%,12.43%,13.47%,14.69%,16.23%,17.48%,17.33%,18.54%,18.06%,17.89%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,46581.00,8.14%,8.14%,33.82%,33.82%,33.82%,33.82%,33.82%,33.82%,35.35%,33.82%,33.82%,33.82%
2022-08,145114.00,11.28%,15.80%,19.12%,18.50%,18.50%,20.89%,22.67%,26.76%,22.29%,21.85%,21.85%,21.85%
2022-09,67126.34,32.03%,41.44%,46.66%,46.66%,46.27%,45.48%,41.44%,44.70%,44.70%,44.70%,44.70%,44.70%
2022-10,44442.63,nan%,11.88%,11.88%,25.69%,25.69%,11.88%,11.88%,11.88%,11.88%,11.88%,11.88%,11.88%
2022-11,88153.59,25.14%,31.57%,25.14%,30.54%,30.54%,33.44%,28.04%,31.38%,31.38%,30.87%,30.87%,30.87%
2022-12,324164.59,1.99%,8.73%,10.08%,15.14%,15.38%,16.56%,17.58%,18.68%,17.85%,17.85%,17.85%,15.43%
2023-01,944943.00,4.71%,8.91%,11.21%,10.39%,12.53%,13.74%,13.67%,14.16%,14.05%,13.71%,13.51%,13.29%
2023-02,1369414.82,8.86%,13.30%,15.79%,17.11%,19.26%,20.40%,20.63%,20.64%,19.60%,19.25%,18.45%,18.45%
2023-03,1002057.00,9.36%,15.98%,19.74%,23.07%,23.07%,24.71%,23.86%,22.94%,22.91%,23.12%,22.22%,22.19%
2023-04,1275193.00,3.95%,9.65%,14.98%,15.72%,17.11%,15.92%,18.19%,17.63%,18.15%,17.70%,17.52%,17.12%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9
2024-04,15930.0,nan%,nan%,24.71%,24.71%,24.71%,28.83%,28.83%,28.83%,28.83%
2024-05,12690.0,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-06,220205.0,14.35%,18.77%,19.06%,25.17%,25.84%,27.42%,28.84%,nan%,nan%
2024-07,443358.0,12.57%,19.48%,25.12%,28.13%,27.78%,29.00%,nan%,nan%,nan%
2024-08,271022.0,17.48%,26.44%,25.52%,26.51%,24.67%,nan%,nan%,nan%,nan%
2024-09,30173.0,15.33%,24.41%,34.84%,40.23%,nan%,nan%,nan%,nan%,nan%
2024-11,9865.0,nan%,70.42%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-12,116135.0,26.26%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2025-01,125816.0,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,8350.00,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,8.53%,nan%,nan%,nan%
2022-08,19130.00,nan%,nan%,nan%,nan%,nan%,nan%,0.25%,0.25%,3.70%,3.45%,3.45%,3.45%
2022-09,48327.68,nan%,13.07%,30.94%,26.79%,26.24%,25.15%,19.54%,29.68%,28.30%,23.14%,23.14%,23.14%
2022-10,62825.00,nan%,nan%,nan%,nan%,nan%,8.48%,7.87%,9.68%,15.00%,6.63%,13.00%,6.01%
2022-11,69285.22,nan%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%
2022-12,13685.00,nan%,nan%,9.70%,nan%,5.54%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-01,89642.00,4.65%,4.65%,9.18%,4.28%,11.74%,11.56%,10.69%,12.43%,12.26%,10.38%,10.99%,9.71%
2023-02,147754.00,12.57%,12.39%,14.27%,20.30%,15.78%,21.39%,22.39%,22.11%,20.24%,20.94%,17.77%,17.77%
2023-03,226883.00,4.74%,9.50%,12.07%,17.02%,19.04%,23.24%,21.70%,21.63%,21.66%,21.60%,20.65%,19.92%
2023-04,336845.50,1.24%,3.09%,5.01%,9.63%,10.68%,9.73%,9.16%,6.79%,6.44%,6.62%,6.44%,5.55%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-10,1620.0,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2022-12,10508.0,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-02,24698.0,nan%,nan%,26.17%,24.01%,24.01%,19.69%,17.53%,17.53%,17.53%,17.53%,17.53%,17.53%
2023-03,13995.0,32.63%,32.63%,32.63%,32.63%,32.63%,54.22%,54.22%,54.22%,54.22%,54.22%,54.22%,54.22%
2023-04,6840.0,nan%,nan%,nan%,nan%,nan%,24.00%,11.33%,nan%,nan%,nan%,nan%,nan%
2023-06,45910.0,nan%,nan%,nan%,14.02%,18.20%,18.20%,18.20%,22.82%,27.20%,27.20%,22.82%,22.82%
2023-07,41960.0,nan%,nan%,nan%,7.91%,21.90%,31.62%,31.62%,21.90%,21.90%,21.90%,21.90%,21.90%
2023-08,84769.0,16.88%,17.68%,17.68%,17.68%,22.56%,22.56%,27.47%,31.62%,30.92%,27.47%,26.07%,26.07%
2023-09,140380.0,4.94%,4.94%,9.52%,10.91%,14.59%,13.06%,13.06%,15.27%,16.97%,16.97%,16.97%,16.97%
2023-10,279724.0,2.46%,7.48%,14.69%,14.31%,22.41%,21.79%,21.24%,23.42%,19.80%,18.63%,18.63%,18.63%


## 4_17.vintage 7+ 金额 口径

In [182]:
# note 20241127 df_mon 改为 df_mon2，分母保持一致
# （1）总体
vintage_money_d7 = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1)& (df_mob_total.unpaid_money_mob > 0)  & (df_mob_total.归属渠道!='抖音渠道')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d7 = vintage_money_d7[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
# vintage_money_d7
vintage_divide_money_d7 = vintage_money_d7.divide(df_mon2["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d7['采购金额'] = df_mon2["purchase_amount"]
vintage_divide_money_d7[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （2）蚂蚁租物
vintage_money_d7_zm = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1)& (df_mob_total.unpaid_money_mob > 0)  & (df_mob_total.归属渠道=='芝麻租物')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d7_zm = vintage_money_d7_zm[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0','mob_1',

vintage_divide_money_d7_zm = vintage_money_d7_zm.divide(df_mon_zm["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d7_zm['采购金额'] = df_mon_zm["purchase_amount"]
vintage_divide_money_d7_zm[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （3）搜索渠道
vintage_money_d7_ss = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1)& (df_mob_total.unpaid_money_mob > 0)  & (df_mob_total.归属渠道=='搜索渠道')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d7_ss = vintage_money_d7_ss[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0','mob_1',

vintage_divide_money_d7_ss = vintage_money_d7_ss.divide(df_mon_ss["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d7_ss['采购金额'] = df_mon_ss["purchase_amount"]
vintage_divide_money_d7_ss[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （4）抖音
vintage_money_d7_dy = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1)& (df_mob_total.unpaid_money_mob > 0)  & (df_mob_total.归属渠道=='抖音渠道')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d7_dy = vintage_money_d7_dy[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9']]  #'下单月份','mob_0','mob_1',,'mob_2',

vintage_divide_money_d7_dy = vintage_money_d7_dy.divide(df_mon_dy["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d7_dy['采购金额'] = df_mon_dy["purchase_amount"]
vintage_divide_money_d7_dy[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9']]  #,'mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12'

# （5）二手非抖音
vintage_money_d7_old = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1)& (df_mob_total.unpaid_money_mob > 0)&(df_mob_total["是否二手"]=="二手")&(df_mob_total.归属渠道!='抖音渠道')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d7_old = vintage_money_d7_old[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_money_d7_old = vintage_money_d7_old.divide(df_mon_old["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d7_old['采购金额'] = df_mon_old["purchase_amount"]
vintage_divide_money_d7_old[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （6）单人会话
vintage_money_d7_drhh = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_7==1)& (df_mob_total.unpaid_money_mob > 0)&(df_mob_total.归属渠道=='单人聊天会话中的小程序消息卡片（分享）')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d7_drhh = vintage_money_d7_drhh[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_money_d7_drhh = vintage_money_d7_drhh.divide(df_mon_drhh["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d7_drhh['采购金额'] = df_mon_drhh["purchase_amount"]
vintage_divide_money_d7_drhh[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,125405.00,3.02%,13.39%,26.63%,28.31%,29.82%,29.57%,29.57%,29.57%,30.14%,29.57%,29.57%,29.57%
2022-08,358451.00,9.77%,19.81%,24.38%,24.76%,25.57%,25.99%,26.45%,28.10%,27.12%,26.93%,27.46%,26.76%
2022-09,277982.75,7.73%,12.04%,15.15%,16.55%,21.42%,19.59%,18.61%,17.90%,18.63%,15.98%,15.98%,15.98%
2022-10,249543.09,4.33%,5.47%,6.09%,12.33%,18.57%,16.13%,14.42%,15.98%,19.37%,15.67%,17.03%,15.04%
2022-11,344258.81,6.44%,9.70%,8.05%,9.44%,9.44%,14.48%,15.52%,12.47%,12.28%,12.12%,12.12%,12.12%
2022-12,566276.59,1.14%,6.21%,8.10%,14.62%,11.97%,12.56%,15.51%,15.58%,16.29%,15.37%,15.27%,13.89%
2023-01,1533459.00,3.97%,7.97%,9.92%,10.62%,12.05%,14.29%,13.41%,14.27%,14.77%,14.54%,14.38%,14.24%
2023-02,2181552.82,6.41%,10.05%,13.17%,14.43%,16.28%,16.45%,16.77%,17.22%,16.15%,16.08%,15.69%,15.52%
2023-03,2083021.00,4.06%,9.54%,11.16%,15.50%,16.61%,17.60%,17.68%,17.58%,18.21%,18.41%,17.81%,17.71%
2023-04,4317651.50,3.19%,6.62%,10.75%,12.69%,14.42%,15.89%,16.16%,16.55%,16.69%,16.43%,16.27%,16.07%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-09,24742.68,nan%,22.84%,22.84%,26.95%,26.95%,26.95%,26.95%,26.95%,26.95%,26.95%,26.95%,26.95%
2022-10,48835.00,5.01%,nan%,4.02%,4.02%,4.02%,14.93%,10.12%,10.12%,8.53%,8.53%,7.74%,7.74%
2022-11,68730.22,nan%,8.09%,8.09%,8.09%,8.09%,8.09%,16.13%,15.16%,14.22%,14.10%,14.10%,14.10%
2022-12,7657.00,nan%,nan%,nan%,nan%,nan%,nan%,32.52%,nan%,nan%,nan%,nan%,nan%
2023-01,110070.00,nan%,nan%,3.34%,3.34%,3.34%,9.28%,9.28%,9.28%,10.08%,10.08%,9.53%,9.53%
2023-02,206619.00,nan%,3.62%,3.62%,3.62%,5.07%,5.98%,5.07%,6.73%,5.07%,5.07%,5.07%,5.07%
2023-03,374912.00,1.15%,1.15%,3.18%,3.85%,8.56%,6.89%,9.00%,10.86%,10.39%,11.28%,12.00%,12.00%
2023-04,1628122.00,3.00%,5.45%,7.78%,9.03%,10.09%,13.12%,13.35%,14.22%,14.39%,14.45%,14.27%,14.24%
2023-05,905764.00,nan%,3.43%,6.69%,6.09%,7.34%,7.44%,11.22%,11.99%,13.03%,11.61%,12.39%,12.15%
2023-06,1710725.00,2.07%,4.58%,9.15%,11.52%,12.92%,14.15%,16.23%,16.61%,17.33%,18.39%,17.91%,17.89%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,46581.00,8.14%,8.14%,33.82%,33.82%,33.82%,33.82%,33.82%,33.82%,35.35%,33.82%,33.82%,33.82%
2022-08,145114.00,11.28%,15.80%,19.12%,18.50%,18.50%,20.89%,22.67%,26.76%,22.29%,21.85%,21.85%,21.85%
2022-09,67126.34,32.03%,41.44%,46.66%,46.66%,46.27%,45.48%,41.44%,44.70%,44.70%,44.70%,44.70%,44.70%
2022-10,44442.63,nan%,11.88%,11.88%,25.69%,25.69%,11.88%,11.88%,11.88%,11.88%,11.88%,11.88%,11.88%
2022-11,88153.59,25.14%,31.57%,25.14%,30.54%,30.54%,33.44%,28.04%,31.38%,31.38%,30.87%,30.87%,30.87%
2022-12,324164.59,1.99%,8.73%,10.08%,15.14%,15.38%,16.56%,17.58%,18.68%,17.85%,17.85%,17.85%,15.43%
2023-01,944943.00,4.03%,8.91%,11.21%,10.39%,10.79%,13.57%,12.70%,13.69%,14.05%,13.71%,13.51%,13.29%
2023-02,1369414.82,8.45%,12.22%,15.37%,17.11%,18.60%,19.75%,20.63%,20.64%,19.60%,19.07%,18.45%,18.45%
2023-03,1002057.00,6.61%,15.22%,18.14%,21.88%,22.51%,24.22%,23.44%,22.49%,22.91%,23.12%,22.22%,22.19%
2023-04,1275193.00,2.92%,7.64%,14.08%,14.81%,16.10%,15.41%,17.31%,17.40%,18.15%,17.52%,17.52%,17.12%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9
2024-04,15930.0,nan%,nan%,24.71%,24.71%,24.71%,24.71%,28.83%,28.83%,28.83%
2024-05,12690.0,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-06,220205.0,9.15%,18.77%,17.93%,24.14%,25.84%,26.60%,28.13%,nan%,nan%
2024-07,443358.0,9.71%,19.48%,25.12%,28.13%,27.78%,29.00%,nan%,nan%,nan%
2024-08,271022.0,15.54%,25.42%,23.22%,25.07%,24.67%,nan%,nan%,nan%,nan%
2024-09,30173.0,15.33%,24.41%,34.84%,40.23%,nan%,nan%,nan%,nan%,nan%
2024-11,9865.0,nan%,70.42%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-12,116135.0,20.93%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2025-01,125816.0,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,8350.00,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,8.53%,nan%,nan%,nan%
2022-08,19130.00,nan%,nan%,nan%,nan%,nan%,nan%,0.25%,0.25%,3.70%,3.45%,3.45%,3.45%
2022-09,48327.68,nan%,13.07%,30.94%,26.79%,26.24%,25.15%,19.54%,24.07%,28.30%,23.14%,23.14%,23.14%
2022-10,62825.00,nan%,nan%,nan%,nan%,nan%,8.48%,7.87%,7.87%,15.00%,6.63%,13.00%,6.01%
2022-11,69285.22,nan%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%
2022-12,13685.00,nan%,nan%,9.70%,nan%,5.54%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-01,89642.00,4.65%,4.65%,9.18%,4.28%,7.36%,11.56%,7.18%,9.36%,12.26%,10.38%,10.99%,9.71%
2023-02,147754.00,10.68%,10.68%,12.75%,18.95%,15.78%,20.40%,21.58%,22.11%,20.24%,20.94%,17.77%,17.77%
2023-03,226883.00,1.90%,9.50%,12.07%,17.02%,19.04%,23.24%,21.70%,21.55%,21.66%,21.60%,20.65%,19.92%
2023-04,336845.50,1.24%,3.09%,4.42%,9.63%,10.68%,9.73%,9.16%,6.79%,6.44%,6.62%,6.44%,5.55%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-10,1620.0,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2022-12,10508.0,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-02,24698.0,nan%,nan%,26.17%,24.01%,24.01%,19.69%,17.53%,17.53%,17.53%,17.53%,17.53%,17.53%
2023-03,13995.0,nan%,32.63%,32.63%,32.63%,32.63%,54.22%,54.22%,54.22%,54.22%,54.22%,54.22%,54.22%
2023-04,6840.0,nan%,nan%,nan%,nan%,nan%,24.00%,11.33%,nan%,nan%,nan%,nan%,nan%
2023-06,45910.0,nan%,nan%,nan%,14.02%,18.20%,18.20%,18.20%,22.82%,27.20%,27.20%,22.82%,22.82%
2023-07,41960.0,nan%,nan%,nan%,7.91%,21.90%,21.90%,31.62%,21.90%,21.90%,21.90%,21.90%,21.90%
2023-08,84769.0,16.88%,17.68%,17.68%,17.68%,22.56%,22.56%,27.47%,31.62%,30.92%,27.47%,26.07%,26.07%
2023-09,140380.0,4.94%,4.94%,9.52%,10.91%,14.59%,10.00%,13.06%,15.27%,16.97%,16.97%,16.97%,16.97%
2023-10,279724.0,nan%,5.23%,12.66%,14.31%,20.45%,20.41%,21.24%,23.42%,19.80%,18.63%,18.63%,18.63%


## 4_18.vintage 15+ 金额 口径

In [183]:
# note 20241127 df_mon 改为 df_mon2，分母保持一致
# （1）总体
vintage_money_d15 = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1)& (df_mob_total.unpaid_money_mob > 0)  & (df_mob_total.归属渠道!='抖音渠道')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d15 = vintage_money_d15[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份',
# vintage_money_d15
vintage_divide_money_d15 = vintage_money_d15.divide(df_mon2["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d15['采购金额'] = df_mon2["purchase_amount"]
vintage_divide_money_d15[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （2）芝麻租物
vintage_money_d15_zm = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1)& (df_mob_total.unpaid_money_mob > 0)  & (df_mob_total.归属渠道=='芝麻租物')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d15_zm = vintage_money_d15_zm[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0','mob_1',

vintage_divide_money_d15_zm = vintage_money_d15_zm.divide(df_mon_zm["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d15_zm['采购金额'] = df_mon_zm["purchase_amount"]
vintage_divide_money_d15_zm[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （3）搜索渠道
vintage_money_d15_ss = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1)& (df_mob_total.unpaid_money_mob > 0)  & (df_mob_total.归属渠道=='搜索渠道')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d15_ss = vintage_money_d15_ss[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0','mob_1',

vintage_divide_money_d15_ss = vintage_money_d15_ss.divide(df_mon_ss["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d15_ss['采购金额'] = df_mon_ss["purchase_amount"]
vintage_divide_money_d15_ss[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （4）抖音
vintage_money_d15_dy = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1)& (df_mob_total.unpaid_money_mob > 0)  & (df_mob_total.归属渠道=='抖音渠道')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d15_dy = vintage_money_d15_dy[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9']]  #'下单月份','mob_0','mob_1',,'mob_2',

vintage_divide_money_d15_dy = vintage_money_d15_dy.divide(df_mon_dy["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d15_dy['采购金额'] = df_mon_dy["purchase_amount"]
vintage_divide_money_d15_dy[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9']]  #,'mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12'

# （5）二手非抖音
vintage_money_d15_old = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1)& (df_mob_total.unpaid_money_mob > 0)&(df_mob_total["是否二手"]=="二手")&(df_mob_total.归属渠道!='抖音渠道')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d15_old = vintage_money_d15_old[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_money_d15_old = vintage_money_d15_old.divide(df_mon_old["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d15_old['采购金额'] = df_mon_old["purchase_amount"]
vintage_divide_money_d15_old[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （6）单人会话
vintage_money_d15_drhh = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_15==1)& (df_mob_total.unpaid_money_mob > 0)&(df_mob_total.归属渠道=='单人聊天会话中的小程序消息卡片（分享）')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d15_drhh = vintage_money_d15_drhh[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_money_d15_drhh = vintage_money_d15_drhh.divide(df_mon_drhh["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d15_drhh['采购金额'] = df_mon_drhh["purchase_amount"]
vintage_divide_money_d15_drhh[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,125405.00,nan%,8.57%,26.63%,26.63%,26.63%,28.38%,29.57%,29.57%,29.57%,29.57%,29.57%,29.57%
2022-08,358451.00,5.45%,13.09%,18.50%,21.03%,24.15%,25.99%,26.45%,27.19%,26.35%,26.93%,27.46%,26.76%
2022-09,277982.75,2.27%,10.01%,15.15%,14.79%,19.98%,19.59%,18.61%,17.90%,17.74%,15.98%,15.98%,15.98%
2022-10,249543.09,4.33%,5.47%,6.09%,12.33%,16.70%,16.13%,14.42%,15.98%,19.37%,15.67%,17.03%,15.04%
2022-11,344258.81,4.44%,9.70%,8.05%,9.44%,9.44%,12.78%,15.52%,12.47%,12.28%,12.12%,12.12%,12.12%
2022-12,566276.59,nan%,6.21%,6.12%,14.62%,11.84%,11.75%,14.92%,14.37%,16.29%,15.37%,15.27%,13.89%
2023-01,1533459.00,3.35%,7.29%,8.22%,10.33%,11.78%,14.03%,13.17%,14.07%,14.61%,14.28%,14.38%,14.24%
2023-02,2181552.82,3.97%,8.36%,12.25%,12.48%,15.78%,15.47%,15.92%,16.81%,16.15%,15.98%,15.69%,15.52%
2023-03,2083021.00,2.85%,7.78%,9.69%,13.25%,14.33%,16.73%,16.89%,17.01%,17.61%,18.13%,17.81%,17.71%
2023-04,4317651.50,2.18%,5.31%,9.31%,11.40%,13.08%,14.99%,15.50%,15.76%,16.42%,15.80%,16.21%,16.07%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-09,24742.68,nan%,nan%,22.84%,26.95%,26.95%,26.95%,26.95%,26.95%,26.95%,26.95%,26.95%,26.95%
2022-10,48835.00,5.01%,nan%,4.02%,4.02%,4.02%,14.93%,10.12%,10.12%,8.53%,8.53%,7.74%,7.74%
2022-11,68730.22,nan%,8.09%,8.09%,8.09%,8.09%,8.09%,16.13%,15.16%,14.22%,14.10%,14.10%,14.10%
2022-12,7657.00,nan%,nan%,nan%,nan%,nan%,nan%,32.52%,nan%,nan%,nan%,nan%,nan%
2023-01,110070.00,nan%,nan%,3.34%,3.34%,3.34%,9.28%,9.28%,9.28%,10.08%,10.08%,9.53%,9.53%
2023-02,206619.00,nan%,3.62%,3.62%,3.62%,5.07%,5.98%,5.07%,6.73%,5.07%,5.07%,5.07%,5.07%
2023-03,374912.00,nan%,1.15%,2.22%,3.01%,4.50%,6.89%,9.00%,10.18%,9.83%,11.28%,12.00%,12.00%
2023-04,1628122.00,2.16%,3.88%,6.73%,7.91%,9.50%,11.94%,12.80%,13.13%,14.39%,14.12%,14.27%,14.24%
2023-05,905764.00,nan%,2.19%,6.69%,6.09%,7.34%,7.21%,11.22%,11.44%,12.19%,11.32%,12.39%,12.15%
2023-06,1710725.00,1.28%,4.23%,7.83%,10.88%,12.74%,13.83%,15.76%,16.23%,17.07%,17.79%,17.91%,17.89%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,46581.00,nan%,8.14%,33.82%,33.82%,33.82%,33.82%,33.82%,33.82%,33.82%,33.82%,33.82%,33.82%
2022-08,145114.00,8.23%,8.23%,12.97%,12.97%,18.50%,20.89%,22.67%,24.50%,22.29%,21.85%,21.85%,21.85%
2022-09,67126.34,9.40%,41.44%,46.66%,46.66%,46.27%,45.48%,41.44%,44.70%,44.70%,44.70%,44.70%,44.70%
2022-10,44442.63,nan%,11.88%,11.88%,25.69%,25.69%,11.88%,11.88%,11.88%,11.88%,11.88%,11.88%,11.88%
2022-11,88153.59,17.33%,31.57%,25.14%,30.54%,30.54%,33.44%,28.04%,31.38%,31.38%,30.87%,30.87%,30.87%
2022-12,324164.59,nan%,8.73%,8.33%,15.14%,15.14%,15.14%,16.56%,16.56%,17.85%,17.85%,17.85%,15.43%
2023-01,944943.00,3.04%,7.81%,8.46%,10.39%,10.35%,13.14%,12.32%,13.37%,13.78%,13.29%,13.51%,13.29%
2023-02,1369414.82,4.57%,10.43%,14.83%,14.83%,18.52%,18.54%,19.89%,20.50%,19.60%,18.91%,18.45%,18.45%
2023-03,1002057.00,5.11%,12.34%,15.46%,19.97%,20.60%,22.41%,22.21%,21.94%,22.54%,22.78%,22.22%,22.19%
2023-04,1275193.00,1.99%,6.83%,11.06%,12.78%,13.73%,14.79%,16.47%,16.42%,17.58%,16.31%,17.32%,17.12%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9
2024-04,15930.0,nan%,nan%,nan%,24.71%,24.71%,24.71%,28.83%,28.83%,28.83%
2024-05,12690.0,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-06,220205.0,1.12%,15.62%,17.93%,21.05%,23.98%,25.49%,27.42%,nan%,nan%
2024-07,443358.0,6.71%,19.48%,25.12%,26.59%,27.78%,28.58%,nan%,nan%,nan%
2024-08,271022.0,7.41%,23.75%,23.22%,23.60%,23.91%,nan%,nan%,nan%,nan%
2024-09,30173.0,7.93%,15.33%,26.51%,26.51%,nan%,nan%,nan%,nan%,nan%
2024-11,9865.0,nan%,70.42%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-12,116135.0,11.83%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2025-01,125816.0,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,8350.00,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2022-08,19130.00,nan%,nan%,nan%,nan%,nan%,nan%,0.25%,0.25%,3.70%,3.45%,3.45%,3.45%
2022-09,48327.68,nan%,13.07%,30.94%,26.79%,26.24%,25.15%,19.54%,24.07%,23.14%,23.14%,23.14%,23.14%
2022-10,62825.00,nan%,nan%,nan%,nan%,nan%,8.48%,7.87%,7.87%,15.00%,6.63%,13.00%,6.01%
2022-11,69285.22,nan%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%
2022-12,13685.00,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-01,89642.00,4.65%,4.65%,9.18%,4.28%,7.36%,11.56%,7.18%,9.36%,12.26%,10.38%,10.99%,9.71%
2023-02,147754.00,3.99%,10.68%,12.75%,15.78%,15.78%,17.88%,19.38%,22.11%,20.24%,20.94%,17.77%,17.77%
2023-03,226883.00,nan%,6.55%,7.79%,14.54%,16.88%,21.21%,19.84%,18.77%,20.74%,21.60%,20.65%,19.92%
2023-04,336845.50,1.24%,1.24%,4.27%,8.50%,8.58%,9.05%,7.29%,6.79%,5.89%,5.06%,6.12%,5.55%


MOB,采购金额,mob_1,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-10,1620.0,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2022-12,10508.0,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-02,24698.0,nan%,nan%,nan%,nan%,24.01%,nan%,nan%,17.53%,17.53%,17.53%,17.53%,17.53%
2023-03,13995.0,nan%,32.63%,32.63%,32.63%,32.63%,54.22%,54.22%,54.22%,54.22%,54.22%,54.22%,54.22%
2023-04,6840.0,nan%,nan%,nan%,nan%,nan%,24.00%,11.33%,nan%,nan%,nan%,nan%,nan%
2023-06,45910.0,nan%,nan%,nan%,14.02%,18.20%,18.20%,18.20%,22.82%,27.20%,27.20%,22.82%,22.82%
2023-07,41960.0,nan%,nan%,nan%,7.91%,21.90%,21.90%,31.62%,21.90%,21.90%,21.90%,21.90%,21.90%
2023-08,84769.0,nan%,9.93%,17.68%,17.68%,22.56%,22.56%,27.47%,31.62%,30.92%,27.47%,26.07%,26.07%
2023-09,140380.0,nan%,4.94%,4.94%,9.52%,14.59%,10.00%,13.06%,15.27%,15.27%,16.97%,16.97%,16.97%
2023-10,279724.0,nan%,5.23%,10.17%,9.77%,18.65%,20.41%,21.24%,22.21%,19.80%,18.63%,18.63%,18.63%


## 4_19.vintage 31+ 金额 口径

In [184]:
# note 20241127 df_mon 改为 df_mon2，分母保持一致
# （1）总体
vintage_money_d31 = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1)& (df_mob_total.unpaid_money_mob > 0)  & (df_mob_total.归属渠道!='抖音渠道')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d31 = vintage_money_d31[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_1',
# vintage_money_d31
vintage_divide_money_d31 = vintage_money_d31.divide(df_mon2["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d31['采购金额'] = df_mon2["purchase_amount"]
vintage_divide_money_d31[['采购金额','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （2）芝麻租物
vintage_money_d31_zm = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1)& (df_mob_total.unpaid_money_mob > 0)  & (df_mob_total.归属渠道=='芝麻租物')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d31_zm = vintage_money_d31_zm[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0','mob_1',

vintage_divide_money_d31_zm = vintage_money_d31_zm.divide(df_mon_zm["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d31_zm['采购金额'] = df_mon_zm["purchase_amount"]
vintage_divide_money_d31_zm[['采购金额','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （3）搜索渠道
vintage_money_d31_ss = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1)& (df_mob_total.unpaid_money_mob > 0)  & (df_mob_total.归属渠道=='搜索渠道')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d31_ss = vintage_money_d31_ss[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0','mob_1',

vintage_divide_money_d31_ss = vintage_money_d31_ss.divide(df_mon_ss["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d31_ss['采购金额'] = df_mon_ss["purchase_amount"]
vintage_divide_money_d31_ss[['采购金额','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （4）抖音
vintage_money_d31_dy = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1)& (df_mob_total.unpaid_money_mob > 0)  & (df_mob_total.归属渠道=='抖音渠道')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_money_d31_dy = vintage_money_d31_dy[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9']]  #'下单月份','mob_0','mob_1',,'mob_2',

vintage_divide_money_d31_dy = vintage_money_d31_dy.divide(df_mon_dy["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d31_dy['采购金额'] = df_mon_dy["purchase_amount"]
vintage_divide_money_d31_dy[['采购金额','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9']]  #,'mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12'

# （5）二手非抖音
vintage_money_d31_old = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1)& (df_mob_total.unpaid_money_mob > 0)&(df_mob_total["是否二手"]=="二手")&(df_mob_total.归属渠道!='抖音渠道')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d31_old = vintage_money_d31_old[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_money_d31_old = vintage_money_d31_old.divide(df_mon_old["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d31_old['采购金额'] = df_mon_old["purchase_amount"]
vintage_divide_money_d31_old[['采购金额','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

# （6）单人会话
vintage_money_d31_drhh = pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_31==1)& (df_mob_total.unpaid_money_mob > 0)&(df_mob_total.归属渠道=='单人聊天会话中的小程序消息卡片（分享）')],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True
vintage_money_d31_drhh = vintage_money_d31_drhh[['mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',
vintage_divide_money_d31_drhh = vintage_money_d31_drhh.divide(df_mon_drhh["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_money_d31_drhh['采购金额'] = df_mon_drhh["purchase_amount"]
vintage_divide_money_d31_drhh[['采购金额','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

MOB,采购金额,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,125405.00,8.57%,26.63%,26.63%,26.63%,28.38%,29.57%,29.57%,29.57%,29.57%,29.57%,29.57%
2022-08,358451.00,11.16%,16.74%,20.26%,23.27%,23.85%,25.22%,26.45%,25.51%,26.93%,26.93%,26.76%
2022-09,277982.75,7.73%,12.04%,14.42%,18.34%,18.61%,18.61%,17.11%,17.74%,15.98%,15.98%,15.98%
2022-10,249543.09,5.47%,5.30%,7.75%,12.33%,14.00%,10.83%,14.42%,14.00%,15.67%,15.27%,15.04%
2022-11,344258.81,8.08%,8.05%,9.44%,9.44%,10.45%,12.51%,12.47%,12.28%,12.12%,12.12%,12.12%
2022-12,566276.59,3.85%,5.20%,11.13%,11.84%,11.75%,13.89%,14.37%,15.74%,15.37%,15.27%,13.89%
2023-01,1533459.00,4.21%,6.53%,8.33%,9.55%,12.84%,12.41%,13.07%,14.20%,14.11%,14.30%,14.24%
2023-02,2181552.82,5.71%,9.32%,11.63%,14.46%,14.33%,15.31%,16.04%,15.99%,15.56%,15.61%,15.52%
2023-03,2083021.00,5.95%,8.84%,11.80%,14.21%,15.66%,15.84%,16.89%,17.37%,17.71%,17.32%,17.71%
2023-04,4317651.50,3.00%,5.96%,8.99%,11.16%,13.00%,14.27%,15.00%,15.88%,15.54%,15.95%,16.07%


MOB,采购金额,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-09,24742.68,nan%,22.84%,22.84%,26.95%,26.95%,26.95%,26.95%,26.95%,26.95%,26.95%,26.95%
2022-10,48835.00,nan%,nan%,4.02%,4.02%,4.02%,nan%,10.12%,nan%,8.53%,7.74%,7.74%
2022-11,68730.22,nan%,8.09%,8.09%,8.09%,8.09%,8.09%,15.16%,14.22%,14.10%,14.10%,14.10%
2022-12,7657.00,nan%,nan%,nan%,nan%,nan%,32.52%,nan%,nan%,nan%,nan%,nan%
2023-01,110070.00,nan%,3.34%,3.34%,3.34%,9.28%,9.28%,9.28%,10.08%,10.08%,9.53%,9.53%
2023-02,206619.00,3.62%,3.62%,3.62%,5.07%,5.07%,5.07%,5.07%,5.07%,5.07%,5.07%,5.07%
2023-03,374912.00,1.15%,1.15%,2.22%,4.50%,6.89%,8.46%,9.91%,9.83%,10.39%,10.91%,12.00%
2023-04,1628122.00,2.59%,5.12%,6.93%,8.34%,9.98%,11.05%,12.33%,13.92%,13.57%,14.00%,14.24%
2023-05,905764.00,nan%,2.69%,5.05%,5.67%,6.66%,8.70%,9.78%,11.55%,11.32%,11.47%,12.15%
2023-06,1710725.00,2.52%,4.99%,8.92%,10.84%,12.46%,13.90%,15.07%,16.17%,17.14%,17.68%,17.89%


MOB,采购金额,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,46581.00,8.14%,33.82%,33.82%,33.82%,33.82%,33.82%,33.82%,33.82%,33.82%,33.82%,33.82%
2022-08,145114.00,8.23%,12.97%,12.97%,18.50%,18.50%,20.89%,22.67%,20.67%,21.85%,21.85%,21.85%
2022-09,67126.34,32.03%,41.44%,46.66%,46.27%,41.44%,41.44%,41.44%,44.70%,44.70%,44.70%,44.70%
2022-10,44442.63,11.88%,11.88%,11.88%,25.69%,11.88%,11.88%,11.88%,11.88%,11.88%,11.88%,11.88%
2022-11,88153.59,31.57%,25.14%,30.54%,30.54%,30.54%,28.04%,31.38%,31.38%,30.87%,30.87%,30.87%
2022-12,324164.59,4.60%,6.73%,11.41%,15.14%,15.14%,16.56%,16.56%,17.85%,17.85%,17.85%,15.43%
2023-01,944943.00,4.00%,6.19%,8.55%,8.65%,11.65%,11.26%,12.53%,13.16%,13.03%,13.39%,13.29%
2023-02,1369414.82,7.12%,10.81%,13.85%,16.94%,17.17%,18.94%,19.52%,19.35%,18.46%,18.32%,18.45%
2023-03,1002057.00,9.44%,14.59%,19.20%,20.35%,21.77%,20.66%,21.94%,22.11%,22.31%,21.78%,22.19%
2023-04,1275193.00,3.42%,6.18%,11.39%,13.33%,13.54%,14.93%,15.67%,16.97%,16.31%,17.05%,17.12%


MOB,采购金额,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9
2024-04,15930.0,nan%,nan%,24.71%,24.71%,24.71%,28.83%,28.83%,28.83%
2024-05,12690.0,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-06,220205.0,11.88%,16.50%,19.07%,20.24%,24.82%,26.60%,nan%,nan%
2024-07,443358.0,11.84%,19.33%,23.37%,25.88%,26.61%,nan%,nan%,nan%
2024-08,271022.0,18.03%,23.22%,22.73%,23.91%,nan%,nan%,nan%,nan%
2024-09,30173.0,15.33%,15.33%,26.51%,nan%,nan%,nan%,nan%,nan%
2024-11,9865.0,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-12,116135.0,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2025-01,125816.0,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,采购金额,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-07,8350.00,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2022-08,19130.00,nan%,nan%,nan%,nan%,nan%,nan%,0.25%,0.25%,3.45%,3.45%,3.45%
2022-09,48327.68,nan%,13.07%,26.79%,26.24%,19.54%,19.54%,19.54%,23.14%,23.14%,23.14%,23.14%
2022-10,62825.00,nan%,nan%,nan%,nan%,nan%,nan%,7.87%,nan%,6.63%,6.01%,6.01%
2022-11,69285.22,nan%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%,8.02%
2022-12,13685.00,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-01,89642.00,4.65%,4.28%,4.28%,4.28%,11.56%,7.18%,7.18%,9.88%,10.38%,9.71%,9.71%
2023-02,147754.00,10.68%,9.11%,12.75%,15.78%,15.78%,17.60%,21.58%,19.92%,20.94%,17.77%,17.77%
2023-03,226883.00,4.74%,6.23%,11.87%,16.88%,21.21%,19.84%,18.77%,20.74%,20.54%,17.99%,19.92%
2023-04,336845.50,1.24%,1.68%,7.24%,7.39%,7.39%,5.54%,6.07%,4.38%,5.06%,5.06%,5.55%


MOB,采购金额,mob_2,mob_3,mob_4,mob_5,mob_6,mob_7,mob_8,mob_9,mob_10,mob_11,mob_12
2022-10,1620.0,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2022-12,10508.0,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-02,24698.0,nan%,nan%,nan%,24.01%,nan%,nan%,17.53%,17.53%,17.53%,17.53%,17.53%
2023-03,13995.0,32.63%,32.63%,32.63%,32.63%,32.63%,54.22%,54.22%,54.22%,54.22%,54.22%,54.22%
2023-04,6840.0,nan%,nan%,nan%,nan%,24.00%,11.33%,nan%,nan%,nan%,nan%,nan%
2023-06,45910.0,nan%,nan%,14.02%,14.02%,18.20%,18.20%,18.20%,22.82%,27.20%,22.82%,22.82%
2023-07,41960.0,nan%,nan%,nan%,21.90%,21.90%,31.62%,21.90%,21.90%,21.90%,21.90%,21.90%
2023-08,84769.0,9.93%,17.68%,17.68%,17.68%,22.56%,22.56%,27.47%,27.47%,27.47%,26.07%,26.07%
2023-09,140380.0,4.94%,4.94%,9.52%,10.49%,10.00%,13.06%,13.06%,15.27%,16.97%,16.97%,16.97%
2023-10,279724.0,nan%,5.35%,4.92%,16.63%,18.60%,19.75%,19.43%,18.63%,18.63%,18.63%,18.63%


# 5.续租订单

<!-- 注意 20241205 20241206 修改非续租的内容 -->
<!-- code未修正 -->

## 5_11.数据处理

In [82]:
# （1）续租数据
df_relet = df_xz_concat[df_xz_concat.is_relet>=1]
df_relet.shape

(39396, 160)

In [83]:
# （2）每个mob对应的月底时间，有些订单mob1不用还，故 n 放到 24
from pandas.tseries.offsets import MonthEnd
def mob_date(df):
    df['下单日期'] = pd.to_datetime(df['下单日期'])
    df['实付日期new'] = pd.to_datetime(df['实付日期new'])
    for n in range(13,25):  
        # note 20241209 修改 月末的的跳级了，所以不要加1，mob对齐
        # df[f'mob_date_{n}'] = np.where(df["下单日期"].dt.day.isin([28,29,30,31]), df['下单日期'] + MonthEnd(n), df['下单日期'] + MonthEnd(n+1))
        df[f'mob_date_{n}'] = np.where((df["下单日期"].dt.day == 31)|(df["下单日期"].isin(['2023-02-28','2024-02-29','2025-02-28']))|((df["下单日期"].dt.month.isin([4,6,9,11]))&(df["下单日期"].dt.day == 30)), df['下单日期'] + MonthEnd(n), df['下单日期'] + MonthEnd(n+1))
        # df[f'mob_date_{n}'] = df['下单日期'] + MonthEnd(n+1) 
        df[f'mob_date_{n}'] = pd.to_datetime(df[f'mob_date_{n}'])
        df[f'paid_date_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],pd.NaT,df['实付日期new'])
        df[f'paid_money_mob{n}'] = np.where(df['实付日期new']  > df[f'mob_date_{n}'],np.nan,df['实付金额'])
        df[f'paid_date_mob{n}'] = pd.to_datetime(df[f'paid_date_mob{n}']) 
    return df

df_relet = mob_date(df_relet)
df_relet[['下单日期','mob_date_15','mob_date_21','mob_date_20','mob_date_13','实付金额','paid_money_mob13','paid_date_mob15','paid_money_mob18']][:2]

,下单日期,mob_date_15,mob_date_21,mob_date_20,mob_date_13,实付金额,paid_money_mob13,paid_date_mob15,paid_money_mob18
12,2022-07-05,2023-10-31,2024-04-30,2024-03-31,2023-08-31,354.0,354.0,2023-07-07,354.0
13,2022-07-05,2023-10-31,2024-04-30,2024-03-31,2023-08-31,354.0,354.0,2023-08-07,354.0


In [84]:
# （3）每个mob对应的逾期天数  
# note 20241209 更新 应付日期，提前还款的统计在内
df_relet['应付日期_new'] = np.where((df_relet['实付日期new'].notna())&((df_relet['应付日期'] - df_relet['实付日期new']).dt.days >= 0),df_relet['实付日期new'],df_relet['应付日期'])
df_mob_list = []

for i in range(12,25): 
    # note 20241209 增加 =
    df_mob = df_relet[((df_relet[f'mob_date_{i}'] - df_relet['应付日期_new']).dt.days >= 0) 
                      & ((df_relet['date_str'] - df_relet['应付日期']).dt.days > 0) 
                      & ((df_relet['date_str'] - df_relet[f'mob_date_{i}']).dt.days > 0)
                      ]
   
    conditions = [ ( ( df_mob['逾期天数']==0) | (pd.notnull(df_mob[f'paid_date_mob{i}']))),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days > 0)  & (pd.isnull(df_mob[f'paid_date_mob{i}'])) ),  
                  (((df_mob[f'mob_date_{i}'] - df_mob['date_str']).dt.days <= 0) & ( pd.isnull(df_mob[f'paid_date_mob{i}'])))]  
  
    choices = [0,(df_mob['date_str'] - df_mob['应付日期']).dt.days,(df_mob[f'mob_date_{i}'] - df_mob['应付日期']).dt.days]  
  
    # 使用numpy.where来应用条件  
    df_mob['overdue_days_mob'] = np.select(conditions, choices, default = df_mob['逾期天数'])

    df_mob['paid_money_mob'] = df_mob[f'paid_money_mob{i}']
    # df_mob

    # df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    # note 20241210 修改 提前支付的剔除未到期应收
    df_mob_group = df_mob.groupby(by=['order_number','下单日期']).agg({'paid_money_mob':'sum','overdue_days_mob':'max'}).reset_index()
    df_mob_group_1 = df_relet[((df_relet[f'mob_date_{i}'] - df_relet['应付日期']).dt.days >= 0) 
                      & ((df_relet['date_str'] - df_relet['应付日期']).dt.days > 0) 
                      & ((df_relet['date_str'] - df_relet[f'mob_date_{i}']).dt.days > 0)
                      ].groupby(by=['order_number']).agg({'当前应付租金':'sum'}).reset_index()
    df_mob_group = df_mob_group.merge(df_mob_group_1, on = 'order_number', how = 'left')


    df_mob_group['MOB'] = f'mob_{i}'

    df_mob_list.append(df_mob_group)

df_mob_relet = pd.concat(df_mob_list, ignore_index=True)
df_mob_relet[:2]

# 数据检查
# order_number order_id
# A202207050851373    8924     已买断，买断金额没有体现在实付金额中，买断后剩余期数的实付金额为0；2023-07-05、2023-10-3的还款记录在系统没有找到
# A2023073114301076   508404  只有mob13-16，mob12在非续租那里了，2023.8不用还款，隔月了，故非续租为13个月长度
# A2023070210395232   432132  只有mob12-16，其他的未到期
# A20231026115543149  726194  只有mob12、mob13，其他的未到期

# df_mob_relet[df_mob_relet.order_number == 'A202207050851373']
# 有mob12但无mob13
# A2023110100321259  738560  23
# 有mob13但无mob12
# A202208291922283    18163  26  2023.2-2023.5 的账单没有还，但2023.6-2023.8的账单还了
# A202207282047042    11630  25

# order_number_param = 'A202207282047042'
# var = ['mob_date_0','mob_date_11','mob_date_12','mob_date_13','mob_date_21','mob_date_23']
# var1 = ['mob_date_0','mob_date_1','mob_date_2','mob_date_3','mob_date_4','mob_date_11','mob_date_12']
# df_xz_concat[df_xz_concat.order_number == order_number_param][['order_number','下单日期','应付日期','实付日期new','实付金额] + var1]
# df_relet[df_relet.order_number == order_number_param][['order_number','order_id','下单日期','状态','应付日期','当前应付租金','实付日期new','实付金额','逾期天数'] + var]
# df_mob_relet[df_mob_relet.order_number == order_number_param][['order_number','下单日期','当前应付租金','paid_money_mob','overdue_days_mob','MOB']]

,order_number,下单日期,paid_money_mob,overdue_days_mob,当前应付租金,MOB
0,A202207050851373,2022-07-05,354.0,0,354.0,mob_12
1,A2022072220482114,2022-07-22,0.0,9,495.0,mob_12


In [85]:
# （4）
df_mob_relet['overdue_days_1'] = np.where(df_mob_relet['overdue_days_mob']>= 1,1,0)
df_mob_relet['overdue_days_4'] = np.where(df_mob_relet['overdue_days_mob']>= 4,1,0)
df_mob_relet['overdue_days_7'] = np.where(df_mob_relet['overdue_days_mob']>= 7,1,0)
df_mob_relet['overdue_days_15'] = np.where(df_mob_relet['overdue_days_mob']>= 15,1,0)
df_mob_relet['overdue_days_31'] = np.where(df_mob_relet['overdue_days_mob']>= 31,1,0)
df_mob_relet['下单月份'] = df_mob_relet['下单日期'].dt.strftime('%Y-%m')  

In [86]:
# （5）
# note 20241210
# 非续租跨月的有mob0_mob12，正常的情况下只有mob0-mob11，故用实付金额（已把优惠金额计算在内），首租还完了才可以续租
# 正常续租&逾期续租，逾期续租的话，续租时间不确定，可能会导致分母变化，分母分子统一用续租的，对首租的逾期状态不延伸，首租逾期的不统计在续租范围内（如续租期间还首租的），因为续租的时候肯定是首租已经还清了的
# df_mob_total['采购金额'] 
df_mob_relet = df_mob_relet.merge(dfzh1[['order_number','search_time','purchase_amount','归属渠道']],on = 'order_number',how='left')
df_sz_sh = df_xz_concat[df_xz_concat.is_relet == 0].groupby(by=['order_number']).agg({'实付金额':'sum'}).reset_index()
df_mob_relet.shape
# df_mob_relet[:2]

df_relet_mob10 = df_xz_concat[((df_xz_concat[f'mob_date_10'] - df_xz_concat['应付日期']).dt.days >= 0) 
& ((df_xz_concat['date_str'] - df_xz_concat['应付日期']).dt.days > 0) 
& ((df_xz_concat['date_str'] - df_xz_concat[f'mob_date_10']).dt.days > 0)].groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob10':'sum'}).reset_index()

df_relet_mob11 = df_xz_concat[((df_xz_concat[f'mob_date_11'] - df_xz_concat['应付日期']).dt.days >= 0) 
& ((df_xz_concat['date_str'] - df_xz_concat['应付日期']).dt.days > 0) 
& ((df_xz_concat['date_str'] - df_xz_concat[f'mob_date_11']).dt.days > 0)].groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob11':'sum'}).reset_index()

df_relet_mob12 = df_xz_concat[((df_xz_concat[f'mob_date_12'] - df_xz_concat['应付日期']).dt.days >= 0) 
& ((df_xz_concat['date_str'] - df_xz_concat['应付日期']).dt.days > 0) 
& ((df_xz_concat['date_str'] - df_xz_concat[f'mob_date_12']).dt.days > 0)].groupby(by=['order_number','下单日期']).agg({'当前应付租金':'sum','paid_money_mob12':'sum'}).reset_index()

df_mob_relet = df_mob_relet.merge(df_sz_sh,on = 'order_number',how='left')
df_mob_relet = df_mob_relet.merge(df_relet_mob10[['order_number','paid_money_mob10']],on = 'order_number',how='left')
df_mob_relet = df_mob_relet.merge(df_relet_mob11[['order_number','paid_money_mob11']],on = 'order_number',how='left')
df_mob_relet = df_mob_relet.merge(df_relet_mob12[['order_number','paid_money_mob12']],on = 'order_number',how='left')
df_mob_relet[['order_number','下单日期','当前应付租金','paid_money_mob','overdue_days_mob','MOB','purchase_amount','paid_money_mob10','paid_money_mob11','paid_money_mob12','实付金额']][:2]

(18670, 15)

,order_number,下单日期,当前应付租金,paid_money_mob,overdue_days_mob,MOB,purchase_amount,paid_money_mob10,paid_money_mob11,paid_money_mob12,实付金额
0,A202207050851373,2022-07-05,354.0,354.0,0,mob_12,5150.0,3937.45,4295.4,4649.4,4295.4
1,A2022072220482114,2022-07-22,495.0,0.0,9,mob_12,5890.0,3963.30,4323.6,4323.6,4323.6


In [87]:
# （6）数据检查  首租还完了才可以续租
# order_number order_id
# A202207050851373    8924     已买断，买断金额没有体现在实付金额中，买断后剩余期数的实付金额为0；2023-07-05、2023-10-3的还款记录在系统没有找到
# A2023073114301076   508404  只有mob13-16，mob12在非续租那里了，2023.8不用还款，隔月了，故非续租为13个月长度
# A2023070210395232   432132  只有mob12-16，其他的未到期
# A20231026115543149  726194  只有mob12、mob13，其他的未到期

# df_mob_relet[df_mob_relet.order_number == 'A202207050851373']
# 有mob12但无mob13
# A2023110100321259  738560  23
# 有mob13但无mob12
# A202208291922283  18163  26  2023.2-2023.5 的账单没有还，但2023.6-2023.8的账单还了
# A202207282047042  11630  2022-07-28 首次应付日期和下单日期一致 续租表现期已满
# A2022072220482114   2022-07-22 续租11期
# A20231008165443277  续租首月就逾期
# A20231008003444120 续租首月就逾期，首租的末期在续租的第一期还

# A2024072414134141
order_number_param = 'A2023070210395232'
var = ['mob_date_0','mob_date_11','mob_date_12','mob_date_13','mob_date_21','mob_date_23']
var1 = ['mob_date_0','mob_date_1','mob_date_2','mob_date_3','mob_date_4','mob_date_11','mob_date_12']
df_xz_concat[df_xz_concat.order_number == order_number_param][['order_number','下单日期','应付日期','实付日期new','当前应付租金','实付金额','purchase_amount'] + var1]
df_relet[df_relet.order_number == order_number_param][['order_number','order_id','下单日期','状态','应付日期','当前应付租金','实付日期new','实付金额','逾期天数'] + var]
df_mob_relet[df_mob_relet.order_number == order_number_param][['order_number','下单日期','当前应付租金','paid_money_mob','overdue_days_mob','MOB','purchase_amount','paid_money_mob10','paid_money_mob11','paid_money_mob12','实付金额']]

,order_number,下单日期,应付日期,实付日期new,当前应付租金,实付金额,purchase_amount,mob_date_0,mob_date_1,mob_date_2,mob_date_3,mob_date_4,mob_date_11,mob_date_12
42080,A2023070210395232,2023-07-02,2023-07-05,2023-07-02,1.00,1.00,7240.0,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2024-06-30,2024-07-31
42081,A2023070210395232,2023-07-02,2023-08-05,2023-08-07,539.80,539.80,7240.0,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2024-06-30,2024-07-31
42082,A2023070210395232,2023-07-02,2023-09-05,2023-09-15,539.80,539.80,7240.0,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2024-06-30,2024-07-31
42083,A2023070210395232,2023-07-02,2023-10-05,2023-11-03,539.80,539.80,7240.0,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2024-06-30,2024-07-31
42084,A2023070210395232,2023-07-02,2023-11-05,2023-12-04,539.80,539.80,7240.0,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2024-06-30,2024-07-31
42085,A2023070210395232,2023-07-02,2023-12-05,2023-12-25,539.80,539.80,7240.0,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2024-06-30,2024-07-31
42086,A2023070210395232,2023-07-02,2024-01-05,2024-02-04,539.80,539.80,7240.0,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2024-06-30,2024-07-31
42087,A2023070210395232,2023-07-02,2024-02-05,2024-03-04,539.80,539.80,7240.0,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2024-06-30,2024-07-31
42088,A2023070210395232,2023-07-02,2024-03-05,2024-03-04,539.80,539.80,7240.0,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2024-06-30,2024-07-31
42089,A2023070210395232,2023-07-02,2024-04-05,2024-04-12,539.80,539.80,7240.0,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2024-06-30,2024-07-31


,order_number,order_id,下单日期,状态,应付日期,当前应付租金,实付日期new,实付金额,逾期天数,mob_date_0,mob_date_11,mob_date_12,mob_date_13,mob_date_21,mob_date_23
42092,A2023070210395232,432132,2023-07-02,已逾期支付,2024-07-04,583.88,2024-07-10,583.88,6,2023-07-31,2024-06-30,2024-07-31,2024-08-31,2025-04-30,2025-06-30
42093,A2023070210395232,432132,2023-07-02,已逾期支付,2024-08-04,583.88,2024-08-17,583.88,13,2023-07-31,2024-06-30,2024-07-31,2024-08-31,2025-04-30,2025-06-30
42094,A2023070210395232,432132,2023-07-02,已逾期支付,2024-09-04,583.88,2024-10-02,583.88,28,2023-07-31,2024-06-30,2024-07-31,2024-08-31,2025-04-30,2025-06-30
42095,A2023070210395232,432132,2023-07-02,已逾期支付,2024-10-04,583.88,2024-10-20,583.88,16,2023-07-31,2024-06-30,2024-07-31,2024-08-31,2025-04-30,2025-06-30
42096,A2023070210395232,432132,2023-07-02,已逾期支付,2024-11-04,583.88,2024-11-16,583.88,12,2023-07-31,2024-06-30,2024-07-31,2024-08-31,2025-04-30,2025-06-30
42097,A2023070210395232,432132,2023-07-02,已逾期支付,2024-12-04,583.88,2024-12-13,583.88,9,2023-07-31,2024-06-30,2024-07-31,2024-08-31,2025-04-30,2025-06-30
42098,A2023070210395232,432132,2023-07-02,已逾期支付,2025-01-04,583.88,2025-01-18,583.88,14,2023-07-31,2024-06-30,2024-07-31,2024-08-31,2025-04-30,2025-06-30
42099,A2023070210395232,432132,2023-07-02,已逾期,2025-02-04,583.88,NaT,0.00,1,2023-07-31,2024-06-30,2024-07-31,2024-08-31,2025-04-30,2025-06-30
42100,A2023070210395232,432132,2023-07-02,未到还款日,2025-03-04,583.88,NaT,0.00,0,2023-07-31,2024-06-30,2024-07-31,2024-08-31,2025-04-30,2025-06-30
42101,A2023070210395232,432132,2023-07-02,未到还款日,2025-04-04,583.88,NaT,0.00,0,2023-07-31,2024-06-30,2024-07-31,2024-08-31,2025-04-30,2025-06-30


,order_number,下单日期,当前应付租金,paid_money_mob,overdue_days_mob,MOB,purchase_amount,paid_money_mob10,paid_money_mob11,paid_money_mob12,实付金额
747,A2023070210395232,2023-07-02,583.88,583.88,0,mob_12,7240.0,4859.2,5400.4,6524.08,5940.2
4172,A2023070210395232,2023-07-02,1167.76,1167.76,0,mob_13,7240.0,4859.2,5400.4,6524.08,5940.2
7468,A2023070210395232,2023-07-02,1751.64,1167.76,26,mob_14,7240.0,4859.2,5400.4,6524.08,5940.2
10361,A2023070210395232,2023-07-02,2335.52,2335.52,0,mob_15,7240.0,4859.2,5400.4,6524.08,5940.2
12630,A2023070210395232,2023-07-02,2919.40,2919.40,0,mob_16,7240.0,4859.2,5400.4,6524.08,5940.2
14441,A2023070210395232,2023-07-02,3503.28,3503.28,0,mob_17,7240.0,4859.2,5400.4,6524.08,5940.2
15893,A2023070210395232,2023-07-02,4087.16,4087.16,0,mob_18,7240.0,4859.2,5400.4,6524.08,5940.2


## 5_12.每月对应的续租用户及采购金额

In [88]:
# （1）
# 废弃这个代码
# df_relet1 = df_relet[df_relet.当前期数==13].merge(df_relet_mob10[['order_number','paid_money_mob10']],on = 'order_number',how='left')
# df_relet_mon = df_relet1.groupby(by=['search_time']).agg({'order_number': 'count', '实付金额': 'sum','purchase_amount':'sum','paid_money_mob10_y':'sum'})
# df_relet_mon['relet_principal'] = df_relet_mon['purchase_amount'] - df_relet_mon['paid_money_mob10_y']
# df_relet_mon

# # A2024072414134141 的 relet_principal 不对

In [89]:
# （2）数据检查
# note 20241210 修改
# 续租有些mob12开始，有些mob13开始

# 数据检查，跑数日期20241210，续租的第一期还款日期>=202412的未统计在内
# tmp = df_relet[df_relet.当前期数==13][['order_number','下单日期','search_time']]
# tmp1 = df_mob_relet.drop_duplicates(subset = 'order_number')[['order_number']]
# tmp1['label'] = 1
# tmp.shape
# tmp1.shape

# tmp2 = tmp.merge(tmp1, on = 'order_number', how = 'left')
# tmp2[tmp2.label.isna()].shape
# # A2023112013061723
# # A20231122095801122
# # A2024040211571596
# # A2024072414134141
# tmp2[tmp2.label.isna()].下单日期.min()
# tmp2[tmp2.label.isna()].下单日期.max()
# tmp2[tmp2.label.isna()].search_time.value_counts()
# tmp2[tmp2.label.isna()].tail()


In [90]:
# （3）拆分续租期数
relet_tmp = df_xz_concat[df_xz_concat.is_relet>=1][['order_number','应付日期']]
relet_tmp = relet_tmp.drop_duplicates(subset = ['order_number','应付日期'])
relet_tmp_1 = relet_tmp.groupby(['order_number']).agg({'应付日期': 'count'}).reset_index()
relet_tmp_1.shape
relet_tmp_1 = relet_tmp_1.rename(columns = {'应付日期': 'relet_periods'})
relet_tmp_1.tail(2)
relet_tmp_1.relet_periods.min()
relet_tmp_1.relet_periods.max()
relet_tmp_1[relet_tmp_1.relet_periods <= 6].shape
relet_tmp_1[relet_tmp_1.relet_periods > 6].shape

# 以下是数据检查，租后监控表的 续租期数 不能用，用资方分期表加工续租期数
# sql = '''-- 租后应收监控  
# SELECT  tprm.order_number, tprm.order_id, tprm.relet_periods
# from db_digua_business.t_postlease_receivables_monitoring   tprm 
# where tprm.order_number = 'A202404291003438'
# '''
# query(sql)

# A202207050851373   资方分期表显示12期，租后监控表显示 续租期数是11
# A2022072220482114  资方分期表显示11期，租后监控表显示 续租期数是11
# A2024040211571596  续租提前买断，6期，租后监控表显示 续租期数是0
# A202404291003438   资方分期表为 8 期，租后监控表 3期
# A2024072414134141  资方分期表为 6 期，租后监控表 0期
# df_xz_concat[(df_xz_concat.order_number == 'A202404291003438')][['order_number','下单日期','应付日期','实付日期new','实付金额','is_relet']]
# df_xz_concat[(df_xz_concat.is_relet>=1)][['order_number']].tail(20)
# relet_tmp_1[relet_tmp_1.order_number == 'A2024072414134141']

df_mob_relet = df_mob_relet.merge(relet_tmp_1, on = 'order_number', how = 'left')
df_mob_relet.shape

(3723, 2)

,order_number,relet_periods
3721,A20240620152351112,10
3722,A2024072414134141,6


3

24

(362, 2)

(3361, 2)

(18670, 20)

In [91]:
# （4）
df_relet_mon_tmp = df_mob_relet.sort_values(by = 'MOB', ascending = True)
df_relet_mon_tmp = df_relet_mon_tmp.drop_duplicates(subset = 'order_number')
df_relet_mon_tmp[df_relet_mon_tmp.实付金额 > df_relet_mon_tmp.purchase_amount].shape
df_relet_mon_tmp.shape
# 部份分布在mob_13
df_relet_mon_tmp['MOB'].value_counts()

# relet_principal：采购金额-首租已还租金=续租本金
# 续租总体
df_relet_mon = df_relet_mon_tmp.groupby(by=['search_time']).agg({'order_number': 'count', '实付金额': 'sum','purchase_amount':'sum'})
df_relet_mon['relet_principal'] = np.where((df_relet_mon['purchase_amount'] - df_relet_mon['实付金额'])>0,df_relet_mon['purchase_amount'] - df_relet_mon['实付金额'],0)
df_relet_mon = df_relet_mon.rename(columns = {'实付金额': '首租已付租金'})
df_relet_mon

# 续租6期及以内
df_relet_mon_6= df_relet_mon_tmp[df_relet_mon_tmp.relet_periods <= 6].groupby(by=['search_time']).agg({'order_number': 'count', '实付金额': 'sum','purchase_amount':'sum'})
df_relet_mon_6['relet_principal'] = np.where((df_relet_mon_6['purchase_amount'] - df_relet_mon_6['实付金额'])>0,df_relet_mon_6['purchase_amount'] - df_relet_mon_6['实付金额'],0)
df_relet_mon_6 = df_relet_mon_6.rename(columns = {'实付金额': '首租已付租金'})
df_relet_mon_6

# 续租6期以上
df_relet_mon_12= df_relet_mon_tmp[df_relet_mon_tmp.relet_periods > 6].groupby(by=['search_time']).agg({'order_number': 'count', '实付金额': 'sum','purchase_amount':'sum'})
df_relet_mon_12['relet_principal'] = np.where((df_relet_mon_12['purchase_amount'] - df_relet_mon_12['实付金额'])>0,df_relet_mon_12['purchase_amount'] - df_relet_mon_12['实付金额'],0)
df_relet_mon_12 = df_relet_mon_12.rename(columns = {'实付金额': '首租已付租金'})
df_relet_mon_12

(82, 20)

(3596, 20)

MOB
mob_12    3366
mob_13     230
Name: count, dtype: int64

,order_number,首租已付租金,purchase_amount,relet_principal
search_time,,,,
2022-07,5,22561.42,29187.00,6625.58
2022-08,15,72075.07,98008.00,25932.93
2022-09,14,76425.12,92730.85,16305.73
2022-10,15,71732.50,87371.00,15638.50
2022-11,17,92272.70,132745.00,40472.30
2022-12,21,129087.10,177135.00,48047.90
2023-01,61,304912.80,391326.00,86413.20
2023-02,72,356934.95,457244.00,100309.05
2023-03,69,336251.37,472351.00,136099.63


,order_number,首租已付租金,purchase_amount,relet_principal
search_time,,,,
2023-01,59,297922.00,380406.0,82484.00
2023-02,70,347883.00,444794.0,96911.00
2023-03,68,332402.59,464309.0,131906.41
2023-04,109,524505.81,742049.0,217543.19
2023-05,15,70356.82,96510.0,26153.18
2023-07,1,5874.20,7420.0,1545.80
2024-01,9,52101.90,64430.0,12328.10


,order_number,首租已付租金,purchase_amount,relet_principal
search_time,,,,
2022-07,5,22561.42,29187.00,6625.58
2022-08,15,72075.07,98008.00,25932.93
2022-09,14,76425.12,92730.85,16305.73
2022-10,15,71732.50,87371.00,15638.50
2022-11,17,92272.70,132745.00,40472.30
2022-12,21,129087.10,177135.00,48047.90
2023-01,2,6990.80,10920.00,3929.20
2023-02,2,9051.95,12450.00,3398.05
2023-03,1,3848.78,8042.00,4193.22


## 5_13.续租订单 vintage 4+ 订单口径

In [92]:
# （1）续租总体
vintage_relet_d4 = pd.pivot_table(df_mob_relet[df_mob_relet.overdue_days_4==1],index='下单月份',columns='MOB',values='overdue_days_4',aggfunc='sum') 
vintage_relet_d4 
 #.reset_index()  #,margins=True

vintage_relet_d4 = vintage_relet_d4[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]  #'下单月份','mob_0',,'mob_23'
# vintage_df_d4
vintage_divide_relet_d4 = vintage_relet_d4.divide(df_relet_mon["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_relet_d4['订单量'] = df_relet_mon["order_number"]
vintage_divide_relet_d4[['订单量','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]  #  ,'mob_23'

# （2）续租<=6期
conditions_relet = (df_mob_relet.relet_periods <= 6)
vintage_relet_d4 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_4==1) & conditions_relet],index='下单月份',columns='MOB',values='overdue_days_4',aggfunc='sum') 
# vintage_relet_d4 

# vintage_relet_d4 = vintage_relet_d4[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]
vintage_relet_d4 = vintage_relet_d4[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
vintage_divide_relet_d4 = vintage_relet_d4.divide(df_relet_mon_6["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_relet_d4['订单量'] = df_relet_mon_6["order_number"]
# vintage_divide_relet_d4[['订单量','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]
vintage_divide_relet_d4[['订单量','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]


# （3）续租>6期
conditions_relet = (df_mob_relet.relet_periods > 6)
vintage_relet_d4 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_4==1) & conditions_relet],index='下单月份',columns='MOB',values='overdue_days_4',aggfunc='sum') 
# vintage_relet_d4 

# vintage_relet_d4 = vintage_relet_d4[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]
vintage_relet_d4 = vintage_relet_d4[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
vintage_divide_relet_d4 = vintage_relet_d4.divide(df_relet_mon_12["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_relet_d4['订单量'] = df_relet_mon_12["order_number"]
# vintage_divide_relet_d4[['订单量','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]
vintage_divide_relet_d4[['订单量','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]

MOB,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23,mob_24
下单月份,,,,,,,,,,,,,
2022-07,1.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,NaN,2.0,1.0
2022-08,2.0,2.0,3.0,4.0,4.0,3.0,5.0,4.0,4.0,4.0,4.0,4.0,4.0
2022-09,5.0,2.0,3.0,4.0,5.0,6.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
2022-10,NaN,NaN,2.0,3.0,4.0,5.0,4.0,5.0,5.0,4.0,3.0,3.0,3.0
2022-11,NaN,1.0,2.0,3.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,4.0,4.0
2022-12,2.0,2.0,4.0,4.0,3.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
2023-01,3.0,8.0,14.0,15.0,17.0,18.0,17.0,13.0,13.0,13.0,14.0,14.0,14.0
2023-02,7.0,13.0,15.0,20.0,22.0,26.0,20.0,18.0,18.0,18.0,18.0,18.0,NaN
2023-03,6.0,9.0,13.0,11.0,17.0,20.0,16.0,16.0,15.0,15.0,15.0,NaN,NaN


MOB,订单量,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
下单月份,,,,,,,,,,,,,
2022-07,5,20.00%,40.00%,40.00%,20.00%,20.00%,20.00%,40.00%,40.00%,40.00%,20.00%,nan%,40.00%
2022-08,15,13.33%,13.33%,20.00%,26.67%,26.67%,20.00%,33.33%,26.67%,26.67%,26.67%,26.67%,26.67%
2022-09,14,35.71%,14.29%,21.43%,28.57%,35.71%,42.86%,35.71%,35.71%,35.71%,35.71%,35.71%,35.71%
2022-10,15,nan%,nan%,13.33%,20.00%,26.67%,33.33%,26.67%,33.33%,33.33%,26.67%,20.00%,20.00%
2022-11,17,nan%,5.88%,11.76%,17.65%,23.53%,23.53%,23.53%,23.53%,29.41%,23.53%,23.53%,23.53%
2022-12,21,9.52%,9.52%,19.05%,19.05%,14.29%,14.29%,19.05%,19.05%,19.05%,19.05%,19.05%,19.05%
2023-01,61,4.92%,13.11%,22.95%,24.59%,27.87%,29.51%,27.87%,21.31%,21.31%,21.31%,22.95%,22.95%
2023-02,72,9.72%,18.06%,20.83%,27.78%,30.56%,36.11%,27.78%,25.00%,25.00%,25.00%,25.00%,25.00%
2023-03,69,8.70%,13.04%,18.84%,15.94%,24.64%,28.99%,23.19%,23.19%,21.74%,21.74%,21.74%,nan%


MOB,订单量,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
2023-01,59,5.08%,13.56%,23.73%,25.42%,27.12%,28.81%,27.12%,20.34%,20.34%,20.34%,20.34%,20.34%
2023-02,70,8.57%,17.14%,20.00%,27.14%,28.57%,34.29%,25.71%,24.29%,24.29%,22.86%,22.86%,22.86%
2023-03,68,8.82%,13.24%,19.12%,16.18%,25.00%,29.41%,23.53%,23.53%,22.06%,22.06%,22.06%,nan%
2023-04,109,9.17%,17.43%,22.02%,28.44%,33.94%,34.86%,31.19%,29.36%,28.44%,27.52%,nan%,nan%
2023-05,15,6.67%,20.00%,26.67%,33.33%,26.67%,33.33%,26.67%,26.67%,26.67%,nan%,nan%,nan%
2023-07,1,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,nan%,nan%,nan%,nan%,nan%
2024-01,9,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,订单量,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
2022-07,5,20.00%,40.00%,40.00%,20.00%,20.00%,20.00%,40.00%,40.00%,40.00%,20.00%,nan%,40.00%
2022-08,15,13.33%,13.33%,20.00%,26.67%,26.67%,20.00%,33.33%,26.67%,26.67%,26.67%,26.67%,26.67%
2022-09,14,35.71%,14.29%,21.43%,28.57%,35.71%,42.86%,35.71%,35.71%,35.71%,35.71%,35.71%,35.71%
2022-10,15,nan%,nan%,13.33%,20.00%,26.67%,33.33%,26.67%,33.33%,33.33%,26.67%,20.00%,20.00%
2022-11,17,nan%,5.88%,11.76%,17.65%,23.53%,23.53%,23.53%,23.53%,29.41%,23.53%,23.53%,23.53%
2022-12,21,9.52%,9.52%,19.05%,19.05%,14.29%,14.29%,19.05%,19.05%,19.05%,19.05%,19.05%,19.05%
2023-01,2,nan%,nan%,nan%,nan%,50.00%,50.00%,50.00%,50.00%,50.00%,50.00%,100.00%,100.00%
2023-02,2,50.00%,50.00%,50.00%,50.00%,100.00%,100.00%,100.00%,50.00%,50.00%,100.00%,100.00%,100.00%
2023-03,1,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-04,28,17.86%,14.29%,17.86%,17.86%,14.29%,28.57%,28.57%,28.57%,32.14%,35.71%,nan%,nan%


## 5_14.续租订单 vintage 7+ 订单口径

In [93]:
# （1）续租总体
vintage_relet_d7 = pd.pivot_table(df_mob_relet[df_mob_relet.overdue_days_7==1],index='下单月份',columns='MOB',values='overdue_days_7',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_relet_d7 = vintage_relet_d7[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]  #'下单月份','mob_0',,'mob_23'
# vintage_df_d4
vintage_divide_relet_d7 = vintage_relet_d7.divide(df_relet_mon["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_relet_d7['订单量'] = df_relet_mon["order_number"]
vintage_divide_relet_d7[['订单量','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]  #  ,'mob_23'

# （2）续租<=6期
conditions_relet = (df_mob_relet.relet_periods <= 6)
vintage_relet_d7 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_7==1) & conditions_relet],index='下单月份',columns='MOB',values='overdue_days_7',aggfunc='sum') 
# vintage_relet_d7 

# vintage_relet_d7 = vintage_relet_d7[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]
vintage_relet_d7 = vintage_relet_d7[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
vintage_divide_relet_d7 = vintage_relet_d7.divide(df_relet_mon_6["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_relet_d7['订单量'] = df_relet_mon_6["order_number"]
# vintage_divide_relet_d7[['订单量','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]
vintage_divide_relet_d7[['订单量','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]


# （3）续租>6期
conditions_relet = (df_mob_relet.relet_periods > 6)
vintage_relet_d7 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_7==1) & conditions_relet],index='下单月份',columns='MOB',values='overdue_days_7',aggfunc='sum') 
# vintage_relet_d7 

# vintage_relet_d7 = vintage_relet_d7[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]
vintage_relet_d7 = vintage_relet_d7[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
vintage_divide_relet_d7 = vintage_relet_d7.divide(df_relet_mon_12["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_relet_d7['订单量'] = df_relet_mon_12["order_number"]
# vintage_divide_relet_d7[['订单量','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]
vintage_divide_relet_d7[['订单量','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]

MOB,订单量,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
下单月份,,,,,,,,,,,,,
2022-07,5,20.00%,20.00%,40.00%,20.00%,20.00%,20.00%,40.00%,40.00%,40.00%,20.00%,nan%,40.00%
2022-08,15,13.33%,6.67%,20.00%,26.67%,26.67%,20.00%,33.33%,26.67%,26.67%,26.67%,26.67%,26.67%
2022-09,14,7.14%,14.29%,14.29%,14.29%,28.57%,35.71%,35.71%,35.71%,35.71%,35.71%,35.71%,35.71%
2022-10,15,nan%,nan%,13.33%,20.00%,26.67%,33.33%,26.67%,33.33%,33.33%,26.67%,20.00%,20.00%
2022-11,17,nan%,nan%,11.76%,17.65%,17.65%,23.53%,23.53%,23.53%,29.41%,23.53%,23.53%,23.53%
2022-12,21,nan%,9.52%,14.29%,19.05%,14.29%,14.29%,14.29%,19.05%,19.05%,19.05%,19.05%,19.05%
2023-01,61,4.92%,13.11%,21.31%,24.59%,27.87%,27.87%,27.87%,21.31%,21.31%,21.31%,22.95%,22.95%
2023-02,72,8.33%,16.67%,18.06%,25.00%,26.39%,33.33%,27.78%,25.00%,25.00%,25.00%,25.00%,25.00%
2023-03,69,7.25%,11.59%,17.39%,14.49%,23.19%,28.99%,23.19%,23.19%,21.74%,21.74%,21.74%,nan%


MOB,订单量,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
2023-01,59,5.08%,13.56%,22.03%,25.42%,27.12%,27.12%,27.12%,20.34%,20.34%,20.34%,20.34%,20.34%
2023-02,70,7.14%,15.71%,17.14%,24.29%,24.29%,31.43%,25.71%,24.29%,24.29%,22.86%,22.86%,22.86%
2023-03,68,7.35%,11.76%,17.65%,14.71%,23.53%,29.41%,23.53%,23.53%,22.06%,22.06%,22.06%,nan%
2023-04,109,8.26%,17.43%,21.10%,28.44%,33.94%,33.94%,31.19%,29.36%,28.44%,27.52%,nan%,nan%
2023-05,15,6.67%,20.00%,26.67%,33.33%,26.67%,33.33%,26.67%,26.67%,26.67%,nan%,nan%,nan%
2023-07,1,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,nan%,nan%,nan%,nan%,nan%
2024-01,9,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,订单量,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
2022-07,5,20.00%,20.00%,40.00%,20.00%,20.00%,20.00%,40.00%,40.00%,40.00%,20.00%,nan%,40.00%
2022-08,15,13.33%,6.67%,20.00%,26.67%,26.67%,20.00%,33.33%,26.67%,26.67%,26.67%,26.67%,26.67%
2022-09,14,7.14%,14.29%,14.29%,14.29%,28.57%,35.71%,35.71%,35.71%,35.71%,35.71%,35.71%,35.71%
2022-10,15,nan%,nan%,13.33%,20.00%,26.67%,33.33%,26.67%,33.33%,33.33%,26.67%,20.00%,20.00%
2022-11,17,nan%,nan%,11.76%,17.65%,17.65%,23.53%,23.53%,23.53%,29.41%,23.53%,23.53%,23.53%
2022-12,21,nan%,9.52%,14.29%,19.05%,14.29%,14.29%,14.29%,19.05%,19.05%,19.05%,19.05%,19.05%
2023-01,2,nan%,nan%,nan%,nan%,50.00%,50.00%,50.00%,50.00%,50.00%,50.00%,100.00%,100.00%
2023-02,2,50.00%,50.00%,50.00%,50.00%,100.00%,100.00%,100.00%,50.00%,50.00%,100.00%,100.00%,100.00%
2023-03,1,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-04,28,17.86%,14.29%,17.86%,17.86%,14.29%,28.57%,28.57%,28.57%,32.14%,35.71%,nan%,nan%


## 5_15.续租订单 vintage 15+ 订单口径

In [94]:
# （1）续租总体
vintage_relet_d15 = pd.pivot_table(df_mob_relet[df_mob_relet.overdue_days_15==1],index='下单月份',columns='MOB',values='overdue_days_15',aggfunc='sum')
vintage_relet_d15
# vintage_relet_d15 = pd.pivot_table(df_mob_relet[df_mob_relet.overdue_days_15==1],index='下单月份',columns='MOB',values='overdue_days_15',aggfunc='sum')  #.reset_index()  #,margins=True

vintage_relet_d15 = vintage_relet_d15[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]  #'下单月份','mob_0',,'mob_23'
# vintage_df_d4
vintage_divide_relet_d15 = vintage_relet_d15.divide(df_relet_mon["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_relet_d15['订单量'] = df_relet_mon["order_number"]
vintage_divide_relet_d15[['订单量','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]  #  ,'mob_23'

# （2）续租<=6期
conditions_relet = (df_mob_relet.relet_periods <= 6)
vintage_relet_d15 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_15==1) & conditions_relet],index='下单月份',columns='MOB',values='overdue_days_15',aggfunc='sum') 
# vintage_relet_d15 

# vintage_relet_d15 = vintage_relet_d15[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]
vintage_relet_d15 = vintage_relet_d15[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
vintage_divide_relet_d15 = vintage_relet_d15.divide(df_relet_mon_6["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_relet_d15['订单量'] = df_relet_mon_6["order_number"]
# vintage_divide_relet_d15[['订单量','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]
vintage_divide_relet_d15[['订单量','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]


# （3）续租>6期
conditions_relet = (df_mob_relet.relet_periods > 6)
vintage_relet_d15 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_15==1) & conditions_relet],index='下单月份',columns='MOB',values='overdue_days_15',aggfunc='sum') 
# vintage_relet_d15 

# vintage_relet_d15 = vintage_relet_d15[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]
vintage_relet_d15 = vintage_relet_d15[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
vintage_divide_relet_d15 = vintage_relet_d15.divide(df_relet_mon_12["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_relet_d15['订单量'] = df_relet_mon_12["order_number"]
# vintage_divide_relet_d15[['订单量','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]
vintage_divide_relet_d15[['订单量','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]

MOB,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23,mob_24
下单月份,,,,,,,,,,,,,
2022-07,NaN,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN
2022-08,1.0,1.0,3.0,4.0,4.0,3.0,4.0,3.0,4.0,4.0,4.0,4.0,4.0
2022-09,NaN,1.0,2.0,2.0,4.0,5.0,4.0,4.0,5.0,5.0,5.0,5.0,5.0
2022-10,NaN,NaN,NaN,3.0,2.0,5.0,4.0,5.0,4.0,3.0,3.0,3.0,3.0
2022-11,NaN,NaN,2.0,2.0,3.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
2022-12,NaN,2.0,2.0,4.0,3.0,3.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0
2023-01,1.0,6.0,11.0,15.0,16.0,15.0,17.0,13.0,13.0,13.0,14.0,14.0,14.0
2023-02,4.0,9.0,13.0,16.0,18.0,23.0,20.0,18.0,18.0,18.0,18.0,18.0,NaN
2023-03,3.0,5.0,9.0,8.0,13.0,18.0,16.0,16.0,15.0,15.0,15.0,NaN,NaN


MOB,订单量,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
下单月份,,,,,,,,,,,,,
2022-07,5,nan%,20.00%,40.00%,20.00%,20.00%,20.00%,20.00%,40.00%,20.00%,nan%,nan%,nan%
2022-08,15,6.67%,6.67%,20.00%,26.67%,26.67%,20.00%,26.67%,20.00%,26.67%,26.67%,26.67%,26.67%
2022-09,14,nan%,7.14%,14.29%,14.29%,28.57%,35.71%,28.57%,28.57%,35.71%,35.71%,35.71%,35.71%
2022-10,15,nan%,nan%,nan%,20.00%,13.33%,33.33%,26.67%,33.33%,26.67%,20.00%,20.00%,20.00%
2022-11,17,nan%,nan%,11.76%,11.76%,17.65%,17.65%,23.53%,23.53%,23.53%,23.53%,23.53%,23.53%
2022-12,21,nan%,9.52%,9.52%,19.05%,14.29%,14.29%,14.29%,19.05%,19.05%,19.05%,19.05%,19.05%
2023-01,61,1.64%,9.84%,18.03%,24.59%,26.23%,24.59%,27.87%,21.31%,21.31%,21.31%,22.95%,22.95%
2023-02,72,5.56%,12.50%,18.06%,22.22%,25.00%,31.94%,27.78%,25.00%,25.00%,25.00%,25.00%,25.00%
2023-03,69,4.35%,7.25%,13.04%,11.59%,18.84%,26.09%,23.19%,23.19%,21.74%,21.74%,21.74%,nan%


MOB,订单量,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
2023-01,59,1.69%,10.17%,18.64%,25.42%,27.12%,25.42%,27.12%,20.34%,20.34%,20.34%,20.34%,20.34%
2023-02,70,4.29%,11.43%,17.14%,21.43%,22.86%,30.00%,25.71%,24.29%,24.29%,22.86%,22.86%,22.86%
2023-03,68,4.41%,7.35%,13.24%,11.76%,19.12%,26.47%,23.53%,23.53%,22.06%,22.06%,22.06%,nan%
2023-04,109,4.59%,15.60%,18.35%,20.18%,28.44%,32.11%,31.19%,29.36%,28.44%,27.52%,nan%,nan%
2023-05,15,6.67%,13.33%,26.67%,33.33%,26.67%,33.33%,26.67%,26.67%,26.67%,nan%,nan%,nan%
2023-07,1,nan%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,nan%,nan%,nan%,nan%,nan%
2024-01,9,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,订单量,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
2022-07,5,nan%,20.00%,40.00%,20.00%,20.00%,20.00%,20.00%,40.00%,20.00%,nan%,nan%,nan%
2022-08,15,6.67%,6.67%,20.00%,26.67%,26.67%,20.00%,26.67%,20.00%,26.67%,26.67%,26.67%,26.67%
2022-09,14,nan%,7.14%,14.29%,14.29%,28.57%,35.71%,28.57%,28.57%,35.71%,35.71%,35.71%,35.71%
2022-10,15,nan%,nan%,nan%,20.00%,13.33%,33.33%,26.67%,33.33%,26.67%,20.00%,20.00%,20.00%
2022-11,17,nan%,nan%,11.76%,11.76%,17.65%,17.65%,23.53%,23.53%,23.53%,23.53%,23.53%,23.53%
2022-12,21,nan%,9.52%,9.52%,19.05%,14.29%,14.29%,14.29%,19.05%,19.05%,19.05%,19.05%,19.05%
2023-01,2,nan%,nan%,nan%,nan%,nan%,nan%,50.00%,50.00%,50.00%,50.00%,100.00%,100.00%
2023-02,2,50.00%,50.00%,50.00%,50.00%,100.00%,100.00%,100.00%,50.00%,50.00%,100.00%,100.00%,100.00%
2023-03,1,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-04,28,10.71%,10.71%,17.86%,14.29%,14.29%,17.86%,28.57%,28.57%,32.14%,32.14%,nan%,nan%


## 5_16.续租订单 vintage 31+ 订单口径

In [95]:
# （1）
vintage_relet_d31 = pd.pivot_table(df_mob_relet[df_mob_relet.overdue_days_31==1],index='下单月份',columns='MOB',values='overdue_days_31',aggfunc='sum') 
vintage_relet_d31

 #.reset_index()  #,margins=True

vintage_relet_d31 = vintage_relet_d31[['mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]  #'下单月份','mob_0',,'mob_23''mob_11',
# vintage_df_d4
vintage_divide_relet_d31 = vintage_relet_d31.divide(df_relet_mon["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_relet_d31['订单量'] = df_relet_mon["order_number"]
vintage_divide_relet_d31[['订单量','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]  #  ,'mob_23'

# （2）续租<=6期
conditions_relet = (df_mob_relet.relet_periods <= 6)
vintage_relet_d31 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_31==1) & conditions_relet],index='下单月份',columns='MOB',values='overdue_days_31',aggfunc='sum') 
# vintage_relet_d31 

vintage_relet_d31 = vintage_relet_d31[['mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
# vintage_relet_d31 = vintage_relet_d31[['mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
vintage_divide_relet_d31 = vintage_relet_d31.divide(df_relet_mon_6["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_relet_d31['订单量'] = df_relet_mon_6["order_number"]
vintage_divide_relet_d31[['订单量','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
# vintage_divide_relet_d31[['订单量','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]


# （3）续租>6期
conditions_relet = (df_mob_relet.relet_periods > 6)
vintage_relet_d31 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_31==1) & conditions_relet],index='下单月份',columns='MOB',values='overdue_days_31',aggfunc='sum') 
# vintage_relet_d31 

# vintage_relet_d31 = vintage_relet_d31[['mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]
vintage_relet_d31 = vintage_relet_d31[['mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
vintage_divide_relet_d31 = vintage_relet_d31.divide(df_relet_mon_12["order_number"],axis=0).applymap(lambda x:format(x,".2%"))
vintage_divide_relet_d31['订单量'] = df_relet_mon_12["order_number"]
# vintage_divide_relet_d31[['订单量','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]
vintage_divide_relet_d31[['订单量','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]

MOB,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23,mob_24
下单月份,,,,,,,,,,,,
2022-07,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN
2022-08,1.0,2.0,3.0,3.0,3.0,3.0,3.0,4.0,4.0,4.0,4.0,4.0
2022-09,NaN,2.0,2.0,4.0,5.0,4.0,4.0,5.0,5.0,5.0,5.0,5.0
2022-10,NaN,NaN,1.0,2.0,4.0,4.0,3.0,3.0,3.0,3.0,3.0,3.0
2022-11,NaN,1.0,2.0,2.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
2022-12,2.0,2.0,3.0,2.0,3.0,3.0,3.0,3.0,4.0,4.0,4.0,4.0
2023-01,1.0,7.0,12.0,12.0,13.0,16.0,13.0,13.0,13.0,13.0,14.0,14.0
2023-02,6.0,10.0,13.0,16.0,22.0,20.0,18.0,18.0,17.0,18.0,18.0,NaN
2023-03,3.0,8.0,7.0,10.0,17.0,16.0,16.0,15.0,15.0,15.0,NaN,NaN


MOB,订单量,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
2022-07,5,20.00%,40.00%,20.00%,20.00%,20.00%,20.00%,40.00%,20.00%,nan%,nan%,nan%
2022-08,15,6.67%,13.33%,20.00%,20.00%,20.00%,20.00%,20.00%,26.67%,26.67%,26.67%,26.67%
2022-09,14,nan%,14.29%,14.29%,28.57%,35.71%,28.57%,28.57%,35.71%,35.71%,35.71%,35.71%
2022-10,15,nan%,nan%,6.67%,13.33%,26.67%,26.67%,20.00%,20.00%,20.00%,20.00%,20.00%
2022-11,17,nan%,5.88%,11.76%,11.76%,17.65%,23.53%,23.53%,23.53%,23.53%,23.53%,23.53%
2022-12,21,9.52%,9.52%,14.29%,9.52%,14.29%,14.29%,14.29%,14.29%,19.05%,19.05%,19.05%
2023-01,61,1.64%,11.48%,19.67%,19.67%,21.31%,26.23%,21.31%,21.31%,21.31%,21.31%,22.95%
2023-02,72,8.33%,13.89%,18.06%,22.22%,30.56%,27.78%,25.00%,25.00%,23.61%,25.00%,25.00%
2023-03,69,4.35%,11.59%,10.14%,14.49%,24.64%,23.19%,23.19%,21.74%,21.74%,21.74%,nan%
2023-04,137,8.03%,13.14%,16.06%,21.90%,26.28%,29.93%,29.20%,28.47%,27.74%,nan%,nan%


MOB,订单量,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
2023-01,59,1.69%,11.86%,20.34%,20.34%,22.03%,25.42%,20.34%,20.34%,20.34%,20.34%,20.34%
2023-02,70,7.14%,12.86%,17.14%,21.43%,28.57%,25.71%,24.29%,24.29%,22.86%,22.86%,22.86%
2023-03,68,4.41%,11.76%,10.29%,14.71%,25.00%,23.53%,23.53%,22.06%,22.06%,22.06%,nan%
2023-04,109,7.34%,11.93%,16.51%,23.85%,28.44%,30.28%,29.36%,28.44%,27.52%,nan%,nan%
2023-05,15,13.33%,20.00%,26.67%,26.67%,26.67%,26.67%,26.67%,26.67%,nan%,nan%,nan%
2023-07,1,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,nan%,nan%,nan%,nan%,nan%
2024-01,9,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,订单量,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
2022-07,5,20.00%,40.00%,20.00%,20.00%,20.00%,20.00%,40.00%,20.00%,nan%,nan%,nan%
2022-08,15,6.67%,13.33%,20.00%,20.00%,20.00%,20.00%,20.00%,26.67%,26.67%,26.67%,26.67%
2022-09,14,nan%,14.29%,14.29%,28.57%,35.71%,28.57%,28.57%,35.71%,35.71%,35.71%,35.71%
2022-10,15,nan%,nan%,6.67%,13.33%,26.67%,26.67%,20.00%,20.00%,20.00%,20.00%,20.00%
2022-11,17,nan%,5.88%,11.76%,11.76%,17.65%,23.53%,23.53%,23.53%,23.53%,23.53%,23.53%
2022-12,21,9.52%,9.52%,14.29%,9.52%,14.29%,14.29%,14.29%,14.29%,19.05%,19.05%,19.05%
2023-01,2,nan%,nan%,nan%,nan%,nan%,50.00%,50.00%,50.00%,50.00%,50.00%,100.00%
2023-02,2,50.00%,50.00%,50.00%,50.00%,100.00%,100.00%,50.00%,50.00%,50.00%,100.00%,100.00%
2023-03,1,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-04,28,10.71%,17.86%,14.29%,14.29%,17.86%,28.57%,28.57%,28.57%,28.57%,nan%,nan%


## 5_17.续租订单 vintage 4+ 金额口径

In [96]:
# （1）续租总体
# df_mob_relet['unpaid_money_mob'] = df_mob_relet['purchase_amount'] - df_mob_relet['paid_money_mob'] - df_mob_relet['paid_money_mob10']
# note 20241210
df_mob_relet['unpaid_money_mob'] = df_mob_relet['purchase_amount'] - df_mob_relet['实付金额'] - df_mob_relet['paid_money_mob']
# money_relet_d4 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_4==1)],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum') 
money_relet_d4 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_4==1)&(df_mob_relet.unpaid_money_mob>0)],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum') 

# money_relet_d4 = pd.pivot_table(df_mob_relet[df_mob_relet.overdue_days_4==1],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

money_relet_d4 = money_relet_d4[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]  #'下单月份','mob_0', ,'mob_23'
money_divide_relet_d4 = money_relet_d4.divide(df_relet_mon["relet_principal"],axis=0).applymap(lambda x:format(x,".2%"))
money_divide_relet_d4['relet_principal'] = df_relet_mon["relet_principal"]
money_divide_relet_d4[['relet_principal','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
# money_relet_d4
# （2）续租<=6期
conditions_relet = (df_mob_relet.relet_periods <= 6)
money_relet_d4 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_4==1)&(df_mob_relet.unpaid_money_mob>0)&conditions_relet],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum') 
money_relet_d4 = money_relet_d4[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']] 
# money_relet_d4 = money_relet_d4[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']] 
money_divide_relet_d4 = money_relet_d4.divide(df_relet_mon_6["relet_principal"],axis=0).applymap(lambda x:format(x,".2%"))
money_divide_relet_d4['relet_principal'] = df_relet_mon_6["relet_principal"]
money_divide_relet_d4[['relet_principal','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
# money_divide_relet_d4[['relet_principal','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
# money_relet_d4
# （3）续租>6期
conditions_relet = (df_mob_relet.relet_periods > 6)
money_relet_d4 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_4==1)&(df_mob_relet.unpaid_money_mob>0)&conditions_relet],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum') 
money_relet_d4 = money_relet_d4[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']] 
money_divide_relet_d4 = money_relet_d4.divide(df_relet_mon_12["relet_principal"],axis=0).applymap(lambda x:format(x,".2%"))
money_divide_relet_d4['relet_principal'] = df_relet_mon_12["relet_principal"]
money_divide_relet_d4[['relet_principal','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
# money_relet_d4


MOB,relet_principal,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
下单月份,,,,,,,,,,,,,
2022-07,6625.58,23.64%,45.06%,45.06%,23.64%,23.64%,23.64%,16.17%,16.17%,16.17%,nan%,nan%,nan%
2022-08,25932.93,8.14%,1.43%,14.94%,16.54%,15.12%,14.94%,14.94%,14.94%,14.94%,12.98%,12.98%,12.98%
2022-09,16305.73,38.04%,18.66%,18.66%,18.66%,18.66%,18.66%,18.66%,18.66%,15.94%,15.94%,15.94%,15.94%
2022-10,15638.50,nan%,nan%,3.64%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%
2022-11,40472.30,nan%,10.79%,10.79%,13.47%,12.06%,8.21%,8.21%,8.21%,8.21%,8.21%,8.21%,8.21%
2022-12,48047.90,13.94%,13.94%,19.62%,18.24%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%
2023-01,86413.20,5.25%,6.98%,7.95%,6.51%,5.19%,4.26%,4.26%,4.26%,4.26%,3.18%,3.18%,3.18%
2023-02,100309.05,9.05%,9.44%,8.78%,7.70%,7.59%,7.59%,4.96%,4.56%,4.56%,4.56%,4.56%,4.56%
2023-03,136099.63,8.21%,9.97%,8.92%,5.87%,5.87%,5.87%,5.87%,5.87%,5.87%,5.87%,5.87%,nan%


MOB,relet_principal,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
2023-01,82484.00,5.50%,7.31%,8.33%,6.82%,5.44%,4.46%,4.46%,4.46%,4.46%,3.34%,3.34%,3.34%
2023-02,96911.00,7.37%,7.77%,7.09%,5.97%,5.97%,5.97%,3.77%,3.77%,3.77%,3.77%,3.77%,3.77%
2023-03,131906.41,8.47%,10.29%,9.20%,6.06%,6.06%,6.06%,6.06%,6.06%,6.06%,6.06%,6.06%,nan%
2023-04,217543.19,9.17%,9.90%,8.39%,8.37%,7.59%,6.74%,6.74%,6.64%,6.64%,6.64%,nan%,nan%
2023-05,26153.18,7.42%,13.88%,13.88%,13.88%,9.35%,7.59%,7.59%,7.59%,7.59%,nan%,nan%,nan%
2023-07,1545.80,100.00%,100.00%,100.00%,100.00%,23.18%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-01,12328.10,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,relet_principal,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
2022-07,6625.58,23.64%,45.06%,45.06%,23.64%,23.64%,23.64%,16.17%,16.17%,16.17%,nan%,nan%,nan%
2022-08,25932.93,8.14%,1.43%,14.94%,16.54%,15.12%,14.94%,14.94%,14.94%,14.94%,12.98%,12.98%,12.98%
2022-09,16305.73,38.04%,18.66%,18.66%,18.66%,18.66%,18.66%,18.66%,18.66%,15.94%,15.94%,15.94%,15.94%
2022-10,15638.50,nan%,nan%,3.64%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%
2022-11,40472.30,nan%,10.79%,10.79%,13.47%,12.06%,8.21%,8.21%,8.21%,8.21%,8.21%,8.21%,8.21%
2022-12,48047.90,13.94%,13.94%,19.62%,18.24%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%
2023-01,3929.20,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-02,3398.05,56.94%,56.94%,56.94%,56.94%,53.64%,53.64%,38.64%,26.94%,26.94%,26.94%,26.94%,26.94%
2023-03,4193.22,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-04,55956.12,22.87%,18.86%,21.24%,13.39%,11.66%,11.66%,10.27%,10.27%,10.27%,10.27%,nan%,nan%


## 5_18.续租订单 vintage 7+ 金额口径

In [97]:
# （1）
money_relet_d7 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_7==1)&(df_mob_relet.unpaid_money_mob>0)],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

money_relet_d7 = money_relet_d7[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]  #'下单月份','mob_0', ,'mob_23'
# vintage_df_d4
money_divide_relet_d7 = money_relet_d7.divide(df_relet_mon["relet_principal"],axis=0).applymap(lambda x:format(x,".2%"))
money_divide_relet_d7['relet_principal'] = df_relet_mon["relet_principal"]
money_divide_relet_d7[['relet_principal','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]

# （2）续租<=6期
conditions_relet = (df_mob_relet.relet_periods <= 6)
money_relet_d7 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_7==1)&(df_mob_relet.unpaid_money_mob>0)&conditions_relet],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum') 
money_relet_d7 = money_relet_d7[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']] 
# money_relet_d7 = money_relet_d7[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']] 
money_divide_relet_d7 = money_relet_d7.divide(df_relet_mon_6["relet_principal"],axis=0).applymap(lambda x:format(x,".2%"))
money_divide_relet_d7['relet_principal'] = df_relet_mon_6["relet_principal"]
money_divide_relet_d7[['relet_principal','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
# money_divide_relet_d7[['relet_principal','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
# money_relet_d7
# （3）续租>6期
conditions_relet = (df_mob_relet.relet_periods > 6)
money_relet_d7 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_7==1)&(df_mob_relet.unpaid_money_mob>0)&conditions_relet],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum') 
money_relet_d7 = money_relet_d7[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']] 
money_divide_relet_d7 = money_relet_d7.divide(df_relet_mon_12["relet_principal"],axis=0).applymap(lambda x:format(x,".2%"))
money_divide_relet_d7['relet_principal'] = df_relet_mon_12["relet_principal"]
money_divide_relet_d7[['relet_principal','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]

MOB,relet_principal,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
下单月份,,,,,,,,,,,,,
2022-07,6625.58,23.64%,23.64%,45.06%,23.64%,23.64%,23.64%,16.17%,16.17%,16.17%,nan%,nan%,nan%
2022-08,25932.93,8.14%,nan%,14.94%,16.54%,15.12%,14.94%,14.94%,14.94%,14.94%,12.98%,12.98%,12.98%
2022-09,16305.73,5.82%,18.66%,18.66%,18.66%,18.66%,18.66%,18.66%,18.66%,15.94%,15.94%,15.94%,15.94%
2022-10,15638.50,nan%,nan%,3.64%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%
2022-11,40472.30,nan%,nan%,10.79%,13.47%,11.97%,8.21%,8.21%,8.21%,8.21%,8.21%,8.21%,8.21%
2022-12,48047.90,nan%,13.94%,17.36%,18.24%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%
2023-01,86413.20,5.25%,6.98%,7.95%,6.51%,5.19%,4.26%,4.26%,4.26%,4.26%,3.18%,3.18%,3.18%
2023-02,100309.05,7.45%,8.98%,8.78%,7.70%,7.59%,7.59%,4.96%,4.56%,4.56%,4.56%,4.56%,4.56%
2023-03,136099.63,6.37%,8.27%,8.92%,5.87%,5.87%,5.87%,5.87%,5.87%,5.87%,5.87%,5.87%,nan%


MOB,relet_principal,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
2023-01,82484.00,5.50%,7.31%,8.33%,6.82%,5.44%,4.46%,4.46%,4.46%,4.46%,3.34%,3.34%,3.34%
2023-02,96911.00,5.71%,7.30%,7.09%,5.97%,5.97%,5.97%,3.77%,3.77%,3.77%,3.77%,3.77%,3.77%
2023-03,131906.41,6.57%,8.54%,9.20%,6.06%,6.06%,6.06%,6.06%,6.06%,6.06%,6.06%,6.06%,nan%
2023-04,217543.19,8.10%,9.90%,8.39%,8.37%,7.59%,6.74%,6.74%,6.64%,6.64%,6.64%,nan%,nan%
2023-05,26153.18,7.42%,13.88%,13.88%,13.88%,9.35%,7.59%,7.59%,7.59%,7.59%,nan%,nan%,nan%
2023-07,1545.80,100.00%,100.00%,100.00%,100.00%,23.18%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-01,12328.10,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,relet_principal,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
2022-07,6625.58,23.64%,23.64%,45.06%,23.64%,23.64%,23.64%,16.17%,16.17%,16.17%,nan%,nan%,nan%
2022-08,25932.93,8.14%,nan%,14.94%,16.54%,15.12%,14.94%,14.94%,14.94%,14.94%,12.98%,12.98%,12.98%
2022-09,16305.73,5.82%,18.66%,18.66%,18.66%,18.66%,18.66%,18.66%,18.66%,15.94%,15.94%,15.94%,15.94%
2022-10,15638.50,nan%,nan%,3.64%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%
2022-11,40472.30,nan%,nan%,10.79%,13.47%,11.97%,8.21%,8.21%,8.21%,8.21%,8.21%,8.21%,8.21%
2022-12,48047.90,nan%,13.94%,17.36%,18.24%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%
2023-01,3929.20,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-02,3398.05,56.94%,56.94%,56.94%,56.94%,53.64%,53.64%,38.64%,26.94%,26.94%,26.94%,26.94%,26.94%
2023-03,4193.22,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-04,55956.12,22.87%,18.86%,21.24%,13.39%,11.66%,11.66%,10.27%,10.27%,10.27%,10.27%,nan%,nan%


## 5_19.续租订单 vintage 15+ 金额口径

In [98]:
# （1）
money_relet_d15 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_15==1)&(df_mob_relet.unpaid_money_mob>0)],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum') 
money_relet_d15
#.reset_index()  #,margins=True

money_relet_d15 = money_relet_d15[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]   #'下单月份','mob_0', ,'mob_23'
# vintage_df_d4
money_divide_relet_d15 = money_relet_d15.divide(df_relet_mon["relet_principal"],axis=0).applymap(lambda x:format(x,".2%"))
money_divide_relet_d15['relet_principal'] = df_relet_mon["relet_principal"]
money_divide_relet_d15[['relet_principal','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]

# （2）续租<=6期
conditions_relet = (df_mob_relet.relet_periods <= 6)
money_relet_d15= pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_15==1)&(df_mob_relet.unpaid_money_mob>0)&conditions_relet],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum') 
money_relet_d15= money_relet_d15[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']] 
# money_relet_d15= money_relet_d15[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']] 
money_divide_relet_d15= money_relet_d15.divide(df_relet_mon_6["relet_principal"],axis=0).applymap(lambda x:format(x,".2%"))
money_divide_relet_d15['relet_principal'] = df_relet_mon_6["relet_principal"]
money_divide_relet_d15[['relet_principal','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
# money_divide_relet_d15[['relet_principal','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
# money_relet_d15
# （3）续租>6期
conditions_relet = (df_mob_relet.relet_periods > 6)
money_relet_d15= pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_15==1)&(df_mob_relet.unpaid_money_mob>0)&conditions_relet],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum') 
money_relet_d15= money_relet_d15[['mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']] 
money_divide_relet_d15= money_relet_d15.divide(df_relet_mon_12["relet_principal"],axis=0).applymap(lambda x:format(x,".2%"))
money_divide_relet_d15['relet_principal'] = df_relet_mon_12["relet_principal"]
money_divide_relet_d15[['relet_principal','mob_12','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]

MOB,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23,mob_24
下单月份,,,,,,,,,,,,,
2022-07,NaN,1566.40,2985.40,1566.40,1566.40,1566.40,1071.40,1071.40,1071.40,NaN,NaN,NaN,NaN
2022-08,2110.01,NaN,3875.24,4290.44,3922.04,3875.24,3875.24,3875.24,3875.24,3365.24,3365.24,3365.24,3365.24
2022-09,NaN,1145.00,3042.68,3042.68,3042.68,3042.68,3042.68,3042.68,2598.68,2598.68,2598.68,2598.68,2598.68
2022-10,NaN,NaN,NaN,1211.86,1211.86,1211.86,1211.86,1211.86,1211.86,1211.86,1211.86,1211.86,1211.86
2022-11,NaN,NaN,4366.90,4366.90,4843.30,2934.70,3324.40,3324.40,3324.40,3324.40,3324.40,3324.40,3324.40
2022-12,NaN,6698.50,6698.50,8764.30,7001.90,7001.90,7001.90,7001.90,7001.90,7001.90,7001.90,7001.90,7001.90
2023-01,1737.00,4915.40,6753.01,5624.61,4485.61,3681.61,3681.61,3681.61,3681.61,2751.61,2751.61,2751.61,2751.61
2023-02,4751.65,8132.54,8802.12,7724.12,7612.02,7612.02,4970.55,4572.95,4572.95,4572.95,4572.95,4572.95,NaN
2023-03,3057.50,8515.64,11261.47,7988.57,7988.57,7988.57,7988.57,7988.57,7988.57,7988.57,7988.57,NaN,NaN


MOB,relet_principal,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
下单月份,,,,,,,,,,,,,
2022-07,6625.58,nan%,23.64%,45.06%,23.64%,23.64%,23.64%,16.17%,16.17%,16.17%,nan%,nan%,nan%
2022-08,25932.93,8.14%,nan%,14.94%,16.54%,15.12%,14.94%,14.94%,14.94%,14.94%,12.98%,12.98%,12.98%
2022-09,16305.73,nan%,7.02%,18.66%,18.66%,18.66%,18.66%,18.66%,18.66%,15.94%,15.94%,15.94%,15.94%
2022-10,15638.50,nan%,nan%,nan%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%
2022-11,40472.30,nan%,nan%,10.79%,10.79%,11.97%,7.25%,8.21%,8.21%,8.21%,8.21%,8.21%,8.21%
2022-12,48047.90,nan%,13.94%,13.94%,18.24%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%
2023-01,86413.20,2.01%,5.69%,7.81%,6.51%,5.19%,4.26%,4.26%,4.26%,4.26%,3.18%,3.18%,3.18%
2023-02,100309.05,4.74%,8.11%,8.78%,7.70%,7.59%,7.59%,4.96%,4.56%,4.56%,4.56%,4.56%,4.56%
2023-03,136099.63,2.25%,6.26%,8.27%,5.87%,5.87%,5.87%,5.87%,5.87%,5.87%,5.87%,5.87%,nan%


MOB,relet_principal,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
2023-01,82484.00,2.11%,5.96%,8.19%,6.82%,5.44%,4.46%,4.46%,4.46%,4.46%,3.34%,3.34%,3.34%
2023-02,96911.00,2.91%,6.40%,7.09%,5.97%,5.97%,5.97%,3.77%,3.77%,3.77%,3.77%,3.77%,3.77%
2023-03,131906.41,2.32%,6.46%,8.54%,6.06%,6.06%,6.06%,6.06%,6.06%,6.06%,6.06%,6.06%,nan%
2023-04,217543.19,5.36%,9.45%,8.39%,8.37%,7.59%,6.74%,6.74%,6.64%,6.64%,6.64%,nan%,nan%
2023-05,26153.18,7.42%,13.72%,13.88%,13.88%,9.35%,7.59%,7.59%,7.59%,7.59%,nan%,nan%,nan%
2023-07,1545.80,nan%,100.00%,100.00%,100.00%,23.18%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-01,12328.10,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,relet_principal,mob_12,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
2022-07,6625.58,nan%,23.64%,45.06%,23.64%,23.64%,23.64%,16.17%,16.17%,16.17%,nan%,nan%,nan%
2022-08,25932.93,8.14%,nan%,14.94%,16.54%,15.12%,14.94%,14.94%,14.94%,14.94%,12.98%,12.98%,12.98%
2022-09,16305.73,nan%,7.02%,18.66%,18.66%,18.66%,18.66%,18.66%,18.66%,15.94%,15.94%,15.94%,15.94%
2022-10,15638.50,nan%,nan%,nan%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%
2022-11,40472.30,nan%,nan%,10.79%,10.79%,11.97%,7.25%,8.21%,8.21%,8.21%,8.21%,8.21%,8.21%
2022-12,48047.90,nan%,13.94%,13.94%,18.24%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%
2023-01,3929.20,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-02,3398.05,56.94%,56.94%,56.94%,56.94%,53.64%,53.64%,38.64%,26.94%,26.94%,26.94%,26.94%,26.94%
2023-03,4193.22,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-04,55956.12,14.77%,14.77%,21.24%,11.66%,11.66%,11.66%,10.27%,10.27%,10.27%,10.27%,nan%,nan%


## 5_20.续租订单 vintage 31+ 金额口径

In [99]:
# （1）
money_relet_d31 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_31==1)&(df_mob_relet.unpaid_money_mob>0)],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum') 
money_relet_d31

 #.reset_index()  #,margins=True

money_relet_d31 = money_relet_d31[['mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]  #'下单月份','mob_0', ,'mob_23''mob_11',
# vintage_df_d4
money_divide_relet_d31 = money_relet_d31.divide(df_relet_mon["relet_principal"],axis=0).applymap(lambda x:format(x,".2%"))
money_divide_relet_d31['relet_principal'] = df_relet_mon["relet_principal"]
money_divide_relet_d31[['relet_principal','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]

# （2）续租<=6期
conditions_relet = (df_mob_relet.relet_periods <= 6)
money_relet_d31 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_31==1)&(df_mob_relet.unpaid_money_mob>0)&conditions_relet],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum') 
# money_relet_d31 = money_relet_d31[['mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']] 
money_relet_d31 = money_relet_d31[['mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']] 
money_divide_relet_d31 = money_relet_d31.divide(df_relet_mon_6["relet_principal"],axis=0).applymap(lambda x:format(x,".2%"))
money_divide_relet_d31['relet_principal'] = df_relet_mon_6["relet_principal"]
# money_divide_relet_d31[['relet_principal','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22']]
money_divide_relet_d31[['relet_principal','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]
# money_relet_d31
# （3）续租>6期
conditions_relet = (df_mob_relet.relet_periods > 6)
money_relet_d31 = pd.pivot_table(df_mob_relet[(df_mob_relet.overdue_days_31==1)&(df_mob_relet.unpaid_money_mob>0)&conditions_relet],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum') 
money_relet_d31 = money_relet_d31[['mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']] 
money_divide_relet_d31 = money_relet_d31.divide(df_relet_mon_12["relet_principal"],axis=0).applymap(lambda x:format(x,".2%"))
money_divide_relet_d31['relet_principal'] = df_relet_mon_12["relet_principal"]
money_divide_relet_d31[['relet_principal','mob_13','mob_14','mob_15','mob_16','mob_17','mob_18','mob_19','mob_20','mob_21','mob_22','mob_23']]


MOB,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23,mob_24
下单月份,,,,,,,,,,,,
2022-07,1566.40,2985.40,1566.40,1566.40,1566.40,1071.40,1071.40,1071.40,NaN,NaN,NaN,NaN
2022-08,NaN,2647.23,3875.24,3875.24,3875.24,3875.24,3875.24,3875.24,3365.24,3365.24,3365.24,3365.24
2022-09,NaN,3042.68,3042.68,3042.68,3042.68,3042.68,3042.68,2598.68,2598.68,2598.68,2598.68,2598.68
2022-10,NaN,NaN,NaN,1211.86,1211.86,1211.86,1211.86,1211.86,1211.86,1211.86,1211.86,1211.86
2022-11,NaN,4366.90,4366.90,4366.90,2934.70,3324.40,3324.40,3324.40,3324.40,3324.40,3324.40,3324.40
2022-12,6698.50,6698.50,7782.70,6698.50,7001.90,7001.90,7001.90,7001.90,7001.90,7001.90,7001.90,7001.90
2023-01,1737.00,5845.40,5624.61,4485.61,3681.61,3681.61,3681.61,3681.61,2751.61,2751.61,2751.61,2751.61
2023-02,7961.42,8458.38,7724.12,7214.42,7612.02,4970.55,4572.95,4572.95,4572.95,4572.95,4572.95,NaN
2023-03,4275.80,11261.47,7988.57,7988.57,7988.57,7988.57,7988.57,7988.57,7988.57,7988.57,NaN,NaN


MOB,relet_principal,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
2022-07,6625.58,23.64%,45.06%,23.64%,23.64%,23.64%,16.17%,16.17%,16.17%,nan%,nan%,nan%
2022-08,25932.93,nan%,10.21%,14.94%,14.94%,14.94%,14.94%,14.94%,14.94%,12.98%,12.98%,12.98%
2022-09,16305.73,nan%,18.66%,18.66%,18.66%,18.66%,18.66%,18.66%,15.94%,15.94%,15.94%,15.94%
2022-10,15638.50,nan%,nan%,nan%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%
2022-11,40472.30,nan%,10.79%,10.79%,10.79%,7.25%,8.21%,8.21%,8.21%,8.21%,8.21%,8.21%
2022-12,48047.90,13.94%,13.94%,16.20%,13.94%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%
2023-01,86413.20,2.01%,6.76%,6.51%,5.19%,4.26%,4.26%,4.26%,4.26%,3.18%,3.18%,3.18%
2023-02,100309.05,7.94%,8.43%,7.70%,7.19%,7.59%,4.96%,4.56%,4.56%,4.56%,4.56%,4.56%
2023-03,136099.63,3.14%,8.27%,5.87%,5.87%,5.87%,5.87%,5.87%,5.87%,5.87%,5.87%,nan%
2023-04,273499.31,8.14%,10.91%,9.04%,8.42%,7.74%,7.46%,7.38%,7.38%,7.38%,nan%,nan%


MOB,relet_principal,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
2023-01,82484.00,2.11%,7.09%,6.82%,5.44%,4.46%,4.46%,4.46%,4.46%,3.34%,3.34%,3.34%
2023-02,96911.00,6.22%,6.73%,5.97%,5.97%,5.97%,3.77%,3.77%,3.77%,3.77%,3.77%,3.77%
2023-03,131906.41,3.24%,8.54%,6.06%,6.06%,6.06%,6.06%,6.06%,6.06%,6.06%,6.06%,nan%
2023-04,217543.19,6.44%,8.26%,8.37%,7.59%,6.74%,6.74%,6.64%,6.64%,6.64%,nan%,nan%
2023-05,26153.18,13.72%,13.88%,13.88%,9.35%,7.59%,7.59%,7.59%,7.59%,nan%,nan%,nan%
2023-07,1545.80,100.00%,100.00%,100.00%,23.18%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2024-01,12328.10,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


MOB,relet_principal,mob_13,mob_14,mob_15,mob_16,mob_17,mob_18,mob_19,mob_20,mob_21,mob_22,mob_23
2022-07,6625.58,23.64%,45.06%,23.64%,23.64%,23.64%,16.17%,16.17%,16.17%,nan%,nan%,nan%
2022-08,25932.93,nan%,10.21%,14.94%,14.94%,14.94%,14.94%,14.94%,14.94%,12.98%,12.98%,12.98%
2022-09,16305.73,nan%,18.66%,18.66%,18.66%,18.66%,18.66%,18.66%,15.94%,15.94%,15.94%,15.94%
2022-10,15638.50,nan%,nan%,nan%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%,7.75%
2022-11,40472.30,nan%,10.79%,10.79%,10.79%,7.25%,8.21%,8.21%,8.21%,8.21%,8.21%,8.21%
2022-12,48047.90,13.94%,13.94%,16.20%,13.94%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%,14.57%
2023-01,3929.20,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-02,3398.05,56.94%,56.94%,56.94%,41.94%,53.64%,38.64%,26.94%,26.94%,26.94%,26.94%,26.94%
2023-03,4193.22,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
2023-04,55956.12,14.77%,21.24%,11.66%,11.66%,11.66%,10.27%,10.27%,10.27%,10.27%,nan%,nan%


In [100]:
# df_mob_relet[df_mob_relet.下单月份 == '2023-01'].to_excel('tmp.xlsx', index = False)

# 6.每月还款状态

## 6_11.每月截面数据

In [101]:
# unrecovered_cost overdue_type purchase_amount 来源租后应收监控表 t_postlease_receivables_monitoring T+1更新，最新的数据
# 未回收成本 = 采购成本 - 已收总租金  -- 不成立
# note 20241227
# 未回收成本 = 采购金额 - 总已收（total_received） 
# 总已收（total_received） = 已收租金 + 溢价费 + ......  
# 未回收成本已经剔除了溢价费，但未剔除碎屏险，故减去碎屏险
 
dfzh3["下单月份"] = dfzh3['search_time']
dfzh3["未回收成本"] = dfzh3['unrecovered_cost']
dfzh3["逾期类型"] = dfzh3['overdue_type']
dfzh3["采购金额"] = dfzh3['purchase_amount']

In [102]:
# （1）碎屏险
# `price`： '价格'
# special_price`： '优惠价'
sql = ''' 
SELECT  distinct order_id,price
from db_digua_business.t_service_order
'''

df_s_o = query(sql)
df_s_o = df_s_o.fillna(0)
df_s_o.shape
df_s_o[df_s_o.price > 0].shape
# df_s_o[(df_s_o.price > 0)&(df_s_o.special_price > 0)].shape
# df_s_o[(df_s_o.price < df_s_o.special_price)].shape
df_s_o.drop_duplicates(subset=["order_id"],inplace=True)
df_s_o.shape
# df_s_o[(df_s_o.price>0)&(df_s_o.special_price>0)].tail()

dfzh3.shape
dfzh3 = dfzh3.merge(df_s_o, on = 'order_id', how = 'left')
dfzh3['price'] = dfzh3['price'].fillna(0)
dfzh3.shape

(218506, 2)

(218506, 2)

(217480, 2)

(31757, 86)

(31757, 87)

In [103]:
# （2）溢价费
# sql = ''' 
# SELECT  distinct product_id,commodity_premium
# from db_digua_business.t_product_sku
# '''
# df_p_s = query(sql)
# df_p_s.shape
# count(*)	count(distinct product_id)
# 	319176	6868
# df_p_s.product_id.value_counts()
# df_p_s[df_p_s.product_id == 4321]

# dfzh3['commodity_premium'] = dfzh3['commodity_premium'].fillna(0)

In [104]:
# 更新未回收成本
dfzh3["未回收成本"] = dfzh3.未回收成本 - dfzh3.price

In [105]:
# （1）总体
# M0：1-30
# M1:31-60
# M2：61-90
# M3:91-180
# M6：>180
# 逾期类型：订单当前逾期类型
# note 20241127 续租情况也包含在内 A2023052816303242

df_zt = dfzh3[dfzh3["未回收成本"]>0]    #.groupby(["下单月份","逾期类型"]).agg({"未回收成本":np.sum}).reset_index().pivot(index="下单月份",columns="逾期类型",values="未回收成本")

df_zt['M0'] = np.where(df_zt.逾期类型=='M0',df_zt["未回收成本"],0)
df_zt['M1'] = np.where(df_zt.逾期类型=='M1',df_zt["未回收成本"],0)
df_zt['M2'] = np.where(df_zt.逾期类型=='M2',df_zt["未回收成本"],0)
df_zt['M3'] = np.where(df_zt.逾期类型=='M3',df_zt["未回收成本"],0)
df_zt['M4'] = np.where(df_zt.逾期类型=='M4',df_zt["未回收成本"],0)
df_zt['M5'] = np.where(df_zt.逾期类型=='M5',df_zt["未回收成本"],0)
df_zt['M6'] = np.where(df_zt.逾期类型=='M6',df_zt["未回收成本"],0)
dfyq = df_zt.groupby(["下单月份"]).agg({'M0':'sum','M1':'sum','M2':'sum','M3':'sum','M4':'sum','M5':'sum','M6':'sum'})
# dfyq=dfzh3[dfzh3["未回收成本"]>0].groupby(["下单月份","逾期类型"]).agg({"未回收成本":np.sum}).reset_index().pivot(index="下单月份",columns="逾期类型",values="未回收成本")
dfyq=dfyq.fillna(0)
dfyq.loc[:,"合计逾期"] = dfyq.apply(lambda x:x.sum(),axis=1) 
dfyq1=pd.merge(dfyq,dfzh3.groupby(["下单月份"]).agg({"采购金额":np.sum}),left_on="下单月份",right_on="下单月份",how="left")
dfyq1.loc["总计"]=dfyq1.apply(lambda x:x.sum(),axis=0)
dfyq1["金额逾期率"]=dfyq1["合计逾期"]/dfyq1["采购金额"]
dfyq1["金额逾期率"]=dfyq1["金额逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M1+逾期率"]=(dfyq1["M6"]+dfyq1["M3"]+dfyq1["M2"]+dfyq1["M1"])/dfyq1["采购金额"]
dfyq1["M1+逾期率"]=dfyq1["M1+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M3+逾期率"]=(dfyq1["M6"]+dfyq1["M3"])/dfyq1["采购金额"]
dfyq1["M3+逾期率"]=dfyq1["M3+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M6+逾期率"]=dfyq1["M6"]/dfyq1["采购金额"]
dfyq1["M6+逾期率"]=dfyq1["M6+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1[['M0','M1','M2','M3','M6','合计逾期','采购金额','金额逾期率','M1+逾期率','M3+逾期率','M6+逾期率']]
# dfyq1.iloc[:,1:]#

,M0,M1,M2,M3,M6,合计逾期,采购金额,金额逾期率,M1+逾期率,M3+逾期率,M6+逾期率
下单月份,,,,,,,,,,,
2022-07,0.00,0.00,0.00,0.00,37996.24,37996.24,1.254050e+05,30.30%,30.30%,30.30%,30.30%
2022-08,0.00,0.00,0.00,0.00,104969.98,104969.98,3.584510e+05,29.28%,29.28%,29.28%,29.28%
2022-09,0.00,0.00,0.00,0.00,52385.30,52385.30,2.779828e+05,18.84%,18.84%,18.84%,18.84%
2022-10,0.00,0.00,0.00,0.00,43583.45,43583.45,2.495431e+05,17.47%,17.47%,17.47%,17.47%
2022-11,0.00,0.00,0.00,0.00,48077.70,48077.70,3.442588e+05,13.97%,13.97%,13.97%,13.97%
2022-12,0.00,0.00,0.00,0.00,94904.60,94904.60,5.662766e+05,16.76%,16.76%,16.76%,16.76%
2023-01,0.00,0.00,0.00,0.00,247669.25,247669.25,1.533459e+06,16.15%,16.15%,16.15%,16.15%
2023-02,0.00,0.00,0.00,0.00,354955.03,354955.03,2.181553e+06,16.27%,16.27%,16.27%,16.27%
2023-03,0.00,0.00,0.00,0.00,406344.16,406344.16,2.083021e+06,19.51%,19.51%,19.51%,19.51%


In [106]:
# 跟账龄对比
# df_mon = df_fq_concat[(df_fq_concat.当前期数==1) & (df_fq_concat.purchase_amount > 0)].groupby(by=['search_time']).agg({'order_number': 'count', 'contract_price': 'sum','total_receivable':'sum','purchase_amount':'sum'}) 
# vintage_money_d1 =  pd.pivot_table(df_mob_total[(df_mob_total.overdue_days_1==1)& (df_mob_total.unpaid_money_mob > 0)],index='下单月份',columns='MOB',values='unpaid_money_mob',aggfunc='sum')  #.reset_index()  #,margins=True

# vintage_money_d1 = vintage_money_d1[['mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]  #'下单月份','mob_0',

# vintage_divide_money_d1 = vintage_money_d1.divide(df_mon["purchase_amount"],axis=0).applymap(lambda x:format(x,".2%"))
# vintage_divide_money_d1['采购金额'] = df_mon["purchase_amount"]
# vintage_divide_money_d1[['采购金额','mob_1','mob_2','mob_3','mob_4','mob_5','mob_6','mob_7','mob_8','mob_9','mob_10','mob_11','mob_12']]

In [107]:
# （2）芝麻租物
df_zw = dfzh3[dfzh3["归属渠道"]=="芝麻租物"]
df_zw['M0'] = np.where(df_zw.逾期类型=='M0',df_zw["未回收成本"],0)
df_zw['M1'] = np.where(df_zw.逾期类型=='M1',df_zw["未回收成本"],0)
df_zw['M2'] = np.where(df_zw.逾期类型=='M2',df_zw["未回收成本"],0)
df_zw['M3'] = np.where(df_zw.逾期类型=='M3',df_zw["未回收成本"],0)
df_zw['M4'] = np.where(df_zw.逾期类型=='M4',df_zw["未回收成本"],0)
df_zw['M5'] = np.where(df_zw.逾期类型=='M5',df_zw["未回收成本"],0)
df_zw['M6'] = np.where(df_zw.逾期类型=='M6',df_zw["未回收成本"],0)

dfyq = df_zw[df_zw["未回收成本"]>0].groupby(["下单月份"]).agg({'M0':'sum','M1':'sum','M2':'sum','M3':'sum','M4':'sum','M5':'sum','M6':'sum'})
# dfyq=df_zw[df_zw["未回收成本"]>0].groupby(["下单月份","逾期类型"]).agg({"未回收成本":np.sum}).reset_index().pivot(index="下单月份",columns="逾期类型",values="未回收成本")
dfyq=dfyq.fillna(0)
dfyq.loc[:,"合计逾期"]=dfyq.apply(lambda x:x.sum(),axis=1)
dfyq1=pd.merge(dfyq,df_zw.groupby(["下单月份"]).agg({"采购金额":np.sum}),left_on="下单月份",right_on="下单月份",how="left")
dfyq1.loc["总计"]=dfyq1.apply(lambda x:x.sum(),axis=0)
dfyq1["金额逾期率"]=dfyq1["合计逾期"]/dfyq1["采购金额"]
dfyq1["金额逾期率"]=dfyq1["金额逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M1+逾期率"]=(dfyq1["M6"]+dfyq1["M3"]+dfyq1["M2"]+dfyq1["M1"])/dfyq1["采购金额"]
dfyq1["M1+逾期率"]=dfyq1["M1+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M3+逾期率"]=(dfyq1["M6"]+dfyq1["M3"])/dfyq1["采购金额"]
dfyq1["M3+逾期率"]=dfyq1["M3+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M6+逾期率"]=dfyq1["M6"]/dfyq1["采购金额"]
dfyq1["M6+逾期率"]=dfyq1["M6+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1[['M0','M1','M2','M3','M6','合计逾期','采购金额','金额逾期率','M1+逾期率','M3+逾期率','M6+逾期率']] #,'M4','M5'

# （3）搜索渠道
df_ss = dfzh3[dfzh3["归属渠道"]=="搜索渠道"]  
df_ss['M0'] = np.where(df_ss.逾期类型=='M0',df_ss["未回收成本"],0)
df_ss['M1'] = np.where(df_ss.逾期类型=='M1',df_ss["未回收成本"],0)
df_ss['M2'] = np.where(df_ss.逾期类型=='M2',df_ss["未回收成本"],0)
df_ss['M3'] = np.where(df_ss.逾期类型=='M3',df_ss["未回收成本"],0)
df_ss['M4'] = np.where(df_ss.逾期类型=='M4',df_ss["未回收成本"],0)
df_ss['M5'] = np.where(df_ss.逾期类型=='M5',df_ss["未回收成本"],0)
df_ss['M6'] = np.where(df_ss.逾期类型=='M6',df_ss["未回收成本"],0)

dfyq = df_ss[df_ss["未回收成本"]>0].groupby(["下单月份"]).agg({'M0':'sum','M1':'sum','M2':'sum','M3':'sum','M4':'sum','M5':'sum','M6':'sum'})
# dfyq=df_ss[df_ss["未回收成本"]>0].groupby(["下单月份","逾期类型"]).agg({"未回收成本":np.sum}).reset_index().pivot(index="下单月份",columns="逾期类型",values="未回收成本")
dfyq=dfyq.fillna(0)
dfyq.loc[:,"合计逾期"]=dfyq.apply(lambda x:x.sum(),axis=1)
dfyq1=pd.merge(dfyq,df_ss.groupby(["下单月份"]).agg({"采购金额":np.sum}),left_on="下单月份",right_on="下单月份",how="left")
dfyq1.loc["总计"]=dfyq1.apply(lambda x:x.sum(),axis=0)
dfyq1["金额逾期率"]=dfyq1["合计逾期"]/dfyq1["采购金额"]
dfyq1["金额逾期率"]=dfyq1["金额逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M1+逾期率"]=(dfyq1["M6"]+dfyq1["M3"]+dfyq1["M2"]+dfyq1["M1"])/dfyq1["采购金额"]
dfyq1["M1+逾期率"]=dfyq1["M1+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M3+逾期率"]=(dfyq1["M6"]+dfyq1["M3"])/dfyq1["采购金额"]
dfyq1["M3+逾期率"]=dfyq1["M3+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M6+逾期率"]=dfyq1["M6"]/dfyq1["采购金额"]
dfyq1["M6+逾期率"]=dfyq1["M6+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1[['M0','M1','M2','M3','M6','合计逾期','采购金额','金额逾期率','M1+逾期率','M3+逾期率','M6+逾期率']] #,'M4','M5'

# （4）抖音
df_dy = dfzh3[dfzh3["归属渠道"]=="抖音渠道"]  #df_dy=df[df["渠道名称"]=="芝麻租物"]#
df_dy['M0'] = np.where(df_dy.逾期类型=='M0',df_dy["未回收成本"],0)
df_dy['M1'] = np.where(df_dy.逾期类型=='M1',df_dy["未回收成本"],0)
df_dy['M2'] = np.where(df_dy.逾期类型=='M2',df_dy["未回收成本"],0)
df_dy['M3'] = np.where(df_dy.逾期类型=='M3',df_dy["未回收成本"],0)
df_dy['M4'] = np.where(df_dy.逾期类型=='M4',df_dy["未回收成本"],0)
df_dy['M5'] = np.where(df_dy.逾期类型=='M5',df_dy["未回收成本"],0)
df_dy['M6'] = np.where(df_dy.逾期类型=='M6',df_dy["未回收成本"],0)

dfyq = df_dy[df_dy["未回收成本"]>0].groupby(["下单月份"]).agg({'M0':'sum','M1':'sum','M2':'sum','M3':'sum','M4':'sum','M5':'sum','M6':'sum'})
# dfyq=df_dy[df_dy["未回收成本"]>0].groupby(["下单月份","逾期类型"]).agg({"未回收成本":np.sum}).reset_index().pivot(index="下单月份",columns="逾期类型",values="未回收成本")
dfyq=dfyq.fillna(0)
dfyq.loc[:,"合计逾期"]=dfyq.apply(lambda x:x.sum(),axis=1)
dfyq1=pd.merge(dfyq,df_dy.groupby(["下单月份"]).agg({"采购金额":np.sum}),left_on="下单月份",right_on="下单月份",how="left")
dfyq1.loc["总计"]=dfyq1.apply(lambda x:x.sum(),axis=0)
dfyq1["金额逾期率"]=dfyq1["合计逾期"]/dfyq1["采购金额"]
dfyq1["金额逾期率"]=dfyq1["金额逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M1+逾期率"]=(dfyq1["M6"]+dfyq1["M3"]+dfyq1["M2"]+dfyq1["M1"])/dfyq1["采购金额"]
dfyq1["M1+逾期率"]=dfyq1["M1+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M3+逾期率"]=(dfyq1["M6"]+dfyq1["M3"])/dfyq1["采购金额"]
dfyq1["M3+逾期率"]=dfyq1["M3+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M6+逾期率"]=dfyq1["M6"]/dfyq1["采购金额"]
dfyq1["M6+逾期率"]=dfyq1["M6+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1[['M0','M1','M2','M3','M6','合计逾期','采购金额','金额逾期率','M1+逾期率','M3+逾期率','M6+逾期率']] #,'M4','M5'

,M0,M1,M2,M3,M6,合计逾期,采购金额,金额逾期率,M1+逾期率,M3+逾期率,M6+逾期率
下单月份,,,,,,,,,,,
2022-09,0.00,0.00,0.00,0.00,6668.60,6668.60,2.474268e+04,26.95%,26.95%,26.95%,26.95%
2022-10,0.00,0.00,0.00,0.00,4295.60,4295.60,4.883500e+04,8.80%,8.80%,8.80%,8.80%
2022-11,0.00,0.00,0.00,0.00,9771.79,9771.79,6.873022e+04,14.22%,14.22%,14.22%,14.22%
2022-12,0.00,0.00,0.00,0.00,1255.20,1255.20,7.657000e+03,16.39%,16.39%,16.39%,16.39%
2023-01,0.00,0.00,0.00,0.00,10493.55,10493.55,1.100700e+05,9.53%,9.53%,9.53%,9.53%
2023-02,0.00,0.00,0.00,0.00,15759.76,15759.76,2.066190e+05,7.63%,7.63%,7.63%,7.63%
2023-03,0.00,0.00,0.00,0.00,49946.37,49946.37,3.749120e+05,13.32%,13.32%,13.32%,13.32%
2023-04,0.00,0.00,0.00,0.00,308875.96,308875.96,1.628122e+06,18.97%,18.97%,18.97%,18.97%
2023-05,0.00,0.00,0.00,30.53,137038.86,137069.39,9.057640e+05,15.13%,15.13%,15.13%,15.13%


,M0,M1,M2,M3,M6,合计逾期,采购金额,金额逾期率,M1+逾期率,M3+逾期率,M6+逾期率
下单月份,,,,,,,,,,,
2022-07,0.00,0.00,0.00,0.00,16668.37,16668.37,46581.00,35.78%,35.78%,35.78%,35.78%
2022-08,0.00,0.00,0.00,0.00,34705.96,34705.96,145114.00,23.92%,23.92%,23.92%,23.92%
2022-09,0.00,0.00,0.00,0.00,30435.80,30435.80,67126.34,45.34%,45.34%,45.34%,45.34%
2022-10,0.00,0.00,0.00,0.00,5281.00,5281.00,44442.63,11.88%,11.88%,11.88%,11.88%
2022-11,0.00,0.00,0.00,0.00,27591.41,27591.41,88153.59,31.30%,31.30%,31.30%,31.30%
2022-12,0.00,0.00,0.00,0.00,57221.20,57221.20,324164.59,17.65%,17.65%,17.65%,17.65%
2023-01,0.00,0.00,0.00,0.00,152200.98,152200.98,944943.00,16.11%,16.11%,16.11%,16.11%
2023-02,0.00,0.00,0.00,0.00,247199.03,247199.03,1369414.82,18.05%,18.05%,18.05%,18.05%
2023-03,0.00,0.00,0.00,0.00,241497.40,241497.40,1002057.00,24.10%,24.10%,24.10%,24.10%


,M0,M1,M2,M3,M6,合计逾期,采购金额,金额逾期率,M1+逾期率,M3+逾期率,M6+逾期率
下单月份,,,,,,,,,,,
2024-04,0.00,0.00,0.00,655.85,3932.0,4587.85,15930.0,28.80%,28.80%,28.80%,24.68%
2024-05,0.00,0.00,0.00,0.00,0.0,0.00,12690.0,0.00%,0.00%,0.00%,0.00%
2024-06,6800.93,4218.66,7133.78,17276.96,31691.0,67121.33,220205.0,30.48%,27.39%,22.24%,14.39%
2024-07,13438.75,12150.97,13679.27,83197.28,14676.0,137142.27,443358.0,30.93%,27.90%,22.08%,3.31%
2024-08,8366.82,3911.00,2377.00,58011.00,0.0,72665.82,271022.0,26.81%,23.72%,21.40%,0.00%
2024-09,4132.46,0.00,3374.00,4625.00,0.0,12131.46,30173.0,40.21%,26.51%,15.33%,0.00%
2024-11,6947.00,0.00,0.00,0.00,0.0,6947.00,9865.0,70.42%,0.00%,0.00%,0.00%
2024-12,40078.00,0.00,0.00,0.00,0.0,40078.00,116135.0,34.51%,0.00%,0.00%,0.00%
2025-01,0.00,0.00,0.00,0.00,0.0,0.00,125816.0,0.00%,0.00%,0.00%,0.00%


In [108]:
# （5）二手非抖音
# note 20241209 增加   抖音放安卓新机和苹果11-15二手
df_zw = dfzh3[(dfzh3["是否二手"]=="二手")&(dfzh3.归属渠道!='抖音渠道')]
df_zw['M0'] = np.where(df_zw.逾期类型=='M0',df_zw["未回收成本"],0)
df_zw['M1'] = np.where(df_zw.逾期类型=='M1',df_zw["未回收成本"],0)
df_zw['M2'] = np.where(df_zw.逾期类型=='M2',df_zw["未回收成本"],0)
df_zw['M3'] = np.where(df_zw.逾期类型=='M3',df_zw["未回收成本"],0)
df_zw['M4'] = np.where(df_zw.逾期类型=='M4',df_zw["未回收成本"],0)
df_zw['M5'] = np.where(df_zw.逾期类型=='M5',df_zw["未回收成本"],0)
df_zw['M6'] = np.where(df_zw.逾期类型=='M6',df_zw["未回收成本"],0)

dfyq = df_zw[df_zw["未回收成本"]>0].groupby(["下单月份"]).agg({'M0':'sum','M1':'sum','M2':'sum','M3':'sum','M4':'sum','M5':'sum','M6':'sum'})
# dfyq=df_zw[df_zw["未回收成本"]>0].groupby(["下单月份","逾期类型"]).agg({"未回收成本":np.sum}).reset_index().pivot(index="下单月份",columns="逾期类型",values="未回收成本")
dfyq=dfyq.fillna(0)
dfyq.loc[:,"合计逾期"]=dfyq.apply(lambda x:x.sum(),axis=1)
dfyq1=pd.merge(dfyq,df_zw.groupby(["下单月份"]).agg({"采购金额":np.sum}),left_on="下单月份",right_on="下单月份",how="left")
dfyq1.loc["总计"]=dfyq1.apply(lambda x:x.sum(),axis=0)
dfyq1["金额逾期率"]=dfyq1["合计逾期"]/dfyq1["采购金额"]
dfyq1["金额逾期率"]=dfyq1["金额逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M1+逾期率"]=(dfyq1["M6"]+dfyq1["M3"]+dfyq1["M2"]+dfyq1["M1"])/dfyq1["采购金额"]
dfyq1["M1+逾期率"]=dfyq1["M1+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M3+逾期率"]=(dfyq1["M6"]+dfyq1["M3"])/dfyq1["采购金额"]
dfyq1["M3+逾期率"]=dfyq1["M3+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M6+逾期率"]=dfyq1["M6"]/dfyq1["采购金额"]
dfyq1["M6+逾期率"]=dfyq1["M6+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1[['M0','M1','M2','M3','M6','合计逾期','采购金额','金额逾期率','M1+逾期率','M3+逾期率','M6+逾期率']] #,'M4','M5'

,M0,M1,M2,M3,M6,合计逾期,采购金额,金额逾期率,M1+逾期率,M3+逾期率,M6+逾期率
下单月份,,,,,,,,,,,
2022-08,0.00,0.00,0.00,0.00,660.00,660.00,19130.00,3.45%,3.45%,3.45%,3.45%
2022-09,0.00,0.00,0.00,0.00,11586.46,11586.46,48327.68,23.97%,23.97%,23.97%,23.97%
2022-10,0.00,0.00,0.00,0.00,7654.60,7654.60,62825.00,12.18%,12.18%,12.18%,12.18%
2022-11,0.00,0.00,0.00,0.00,5558.69,5558.69,69285.22,8.02%,8.02%,8.02%,8.02%
2022-12,0.00,0.00,0.00,0.00,0.00,0.00,13685.00,0.00%,0.00%,0.00%,0.00%
2023-01,0.00,0.00,0.00,0.00,9858.20,9858.20,89642.00,11.00%,11.00%,11.00%,11.00%
2023-02,0.00,0.00,0.00,0.00,29630.36,29630.36,147754.00,20.05%,20.05%,20.05%,20.05%
2023-03,0.00,0.00,0.00,0.00,50439.67,50439.67,226883.00,22.23%,22.23%,22.23%,22.23%
2023-04,0.00,0.00,0.00,0.00,23893.48,23893.48,336845.50,7.09%,7.09%,7.09%,7.09%


In [109]:
# （6）单人会话
# note 20241209 增加
df_zw = dfzh3[(dfzh3.归属渠道=='单人聊天会话中的小程序消息卡片（分享）')]
df_zw['M0'] = np.where(df_zw.逾期类型=='M0',df_zw["未回收成本"],0)
df_zw['M1'] = np.where(df_zw.逾期类型=='M1',df_zw["未回收成本"],0)
df_zw['M2'] = np.where(df_zw.逾期类型=='M2',df_zw["未回收成本"],0)
df_zw['M3'] = np.where(df_zw.逾期类型=='M3',df_zw["未回收成本"],0)
df_zw['M4'] = np.where(df_zw.逾期类型=='M4',df_zw["未回收成本"],0)
df_zw['M5'] = np.where(df_zw.逾期类型=='M5',df_zw["未回收成本"],0)
df_zw['M6'] = np.where(df_zw.逾期类型=='M6',df_zw["未回收成本"],0)

dfyq = df_zw[df_zw["未回收成本"]>0].groupby(["下单月份"]).agg({'M0':'sum','M1':'sum','M2':'sum','M3':'sum','M4':'sum','M5':'sum','M6':'sum'})
# dfyq=df_zw[df_zw["未回收成本"]>0].groupby(["下单月份","逾期类型"]).agg({"未回收成本":np.sum}).reset_index().pivot(index="下单月份",columns="逾期类型",values="未回收成本")
dfyq=dfyq.fillna(0)
dfyq.loc[:,"合计逾期"]=dfyq.apply(lambda x:x.sum(),axis=1)
dfyq1=pd.merge(dfyq,df_zw.groupby(["下单月份"]).agg({"采购金额":np.sum}),left_on="下单月份",right_on="下单月份",how="left")
dfyq1.loc["总计"]=dfyq1.apply(lambda x:x.sum(),axis=0)
dfyq1["金额逾期率"]=dfyq1["合计逾期"]/dfyq1["采购金额"]
dfyq1["金额逾期率"]=dfyq1["金额逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M1+逾期率"]=(dfyq1["M6"]+dfyq1["M3"]+dfyq1["M2"]+dfyq1["M1"])/dfyq1["采购金额"]
dfyq1["M1+逾期率"]=dfyq1["M1+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M3+逾期率"]=(dfyq1["M6"]+dfyq1["M3"])/dfyq1["采购金额"]
dfyq1["M3+逾期率"]=dfyq1["M3+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1["M6+逾期率"]=dfyq1["M6"]/dfyq1["采购金额"]
dfyq1["M6+逾期率"]=dfyq1["M6+逾期率"].apply(lambda x:format(x,".2%"))
dfyq1[['M0','M1','M2','M3','M6','合计逾期','采购金额','金额逾期率','M1+逾期率','M3+逾期率','M6+逾期率']] #,'M4','M5'

,M0,M1,M2,M3,M6,合计逾期,采购金额,金额逾期率,M1+逾期率,M3+逾期率,M6+逾期率
下单月份,,,,,,,,,,,
2023-02,0.00,0.00,0.00,0.00,4329.00,4329.00,24698.0,17.53%,17.53%,17.53%,17.53%
2023-03,0.00,0.00,0.00,0.00,7587.47,7587.47,13995.0,54.22%,54.22%,54.22%,54.22%
2023-06,0.00,0.00,0.00,0.00,10476.63,10476.63,45910.0,22.82%,22.82%,22.82%,22.82%
2023-07,0.00,0.00,0.00,0.00,10891.60,10891.60,41960.0,25.96%,25.96%,25.96%,25.96%
2023-08,0.00,0.00,0.00,3456.54,22091.21,25547.75,84769.0,30.14%,30.14%,30.14%,26.06%
2023-09,0.00,0.00,0.00,6889.52,23699.71,30589.23,140380.0,21.79%,21.79%,21.79%,16.88%
2023-10,0.00,0.00,2011.23,13675.44,51844.93,67531.60,279724.0,24.14%,24.14%,23.42%,18.53%
2023-11,0.00,743.17,6135.10,6445.20,115281.56,128605.03,538620.0,23.88%,23.88%,22.60%,21.40%
2023-12,2513.93,17768.37,0.00,34897.67,135792.09,190972.06,956324.0,19.97%,19.71%,17.85%,14.20%


## 6_12.总体订单口径每月当前逾期率

In [110]:
# （1）
dfzh3["还款状态"] = dfzh3["rembursement_status"]

#总体每月当前逾期率 cpd1
df0 = pd.crosstab(dfzh3["下单月份"],dfzh3["还款状态"],margins=True)
df0["逾期率"]=df0["已逾期"]/df0[["已买断","已完成","已逾期","续租中","还款中","检测中"]].apply(lambda x:x.sum(),axis=1)
df0["逾期率"]=df0["逾期率"].apply(lambda x:format(x,".2%"))
df0["买断率"]=df0["已买断"]/df0["All"]
df0["买断率"]=df0["买断率"].apply(lambda x:format(x,".2%"))
df0["归还率"]=(df0["已完成"]+df0["检测中"])/df0["All"]
df0["归还率"]=df0["归还率"].apply(lambda x:format(x,".2%"))
df0["续租率"]=df0["续租中"]/df0["All"]
df0["续租率"]=df0["续租率"].apply(lambda x:format(x,".2%"))
df0#[["已买断","All","买断率"]]#[14:]

dfzh3["续租期数"] = dfzh3["relet_periods"] 
dfzh3["已还期数"] = dfzh3["paid_periods"] 

还款状态,已买断,已完成,已逾期,未到首期还款日,检测中,续租中,还款中,All,逾期率,买断率,归还率,续租率
下单月份,,,,,,,,,,,,
2022-07,10,2,8,0,0,2,0,22,36.36%,45.45%,9.09%,9.09%
2022-08,20,4,30,0,1,1,0,56,53.57%,35.71%,8.93%,1.79%
2022-09,18,6,19,0,0,1,0,44,43.18%,40.91%,13.64%,2.27%
2022-10,20,6,12,0,1,2,0,41,29.27%,48.78%,17.07%,4.88%
2022-11,21,7,13,0,0,2,0,43,30.23%,48.84%,16.28%,4.65%
2022-12,34,4,24,0,0,4,0,66,36.36%,51.52%,6.06%,6.06%
2023-01,108,20,90,0,3,0,0,221,40.72%,48.87%,10.41%,0.00%
2023-02,166,48,116,0,1,0,1,332,34.94%,50.00%,14.76%,0.00%
2023-03,143,45,122,0,3,3,0,316,38.61%,45.25%,15.19%,0.95%


In [111]:
# （2）
# 以下代码按顺序运行完到到期状态数据
# pd.crosstab(dfzh1["下单月份"],dfzh1["已还期数"],margins=True)
df_yq = dfzh3[dfzh3["还款状态"]=="已逾期"]

# df_yq=df[df["还款状态"]=="已逾期"]
# pd.crosstab(df_yq["下单月份"],df_yq["续租期数"],margins=True)

df_xz = dfzh3[dfzh3["续租期数"]>0]
# pd.crosstab(df_xz["下单月份"],df_xz["还款状态"],margins=True)

df_timeup = dfzh3[dfzh3["已还期数"]==12]


df_timeon = dfzh3[dfzh3["已还期数"]<12]
df_timeon_1 = pd.crosstab(df_timeon["下单月份"],df_timeon["还款状态"],margins=True).rename(columns={"已买断":"提前买断","已完成":"提前归还","已逾期":"租期逾期"})
df_timeup_xz = dfzh3[(dfzh3["已还期数"]==12)&(dfzh3["续租期数"]>0)]
df_timeup_xz_1 = pd.crosstab(df_timeup_xz["下单月份"],df_timeup_xz["还款状态"],margins=True).rename(columns={"已买断":"续租后买断","已完成":"续租后归还","已逾期":"续租后逾期","检测中":"续租归还检测中"})
df_timeup_nxz = dfzh3[(dfzh3["已还期数"]==12)&(dfzh3["续租期数"]<1)]
df_timeup_nxz_1 = pd.crosstab(df_timeup_nxz["下单月份"],df_timeup_nxz["还款状态"],margins=True).rename(columns={"已买断":"到期买断","已完成":"到期归还","已逾期":"到期逾期","检测中":"到期归还检测中","续租中":"待续租","还款中":"已付总租金待归还"})

In [112]:
# （3）
#以及每个下单月份的订单在12期租期到期后到期买断、到期归还、到期逾期、到期续租的订单数，
#以及每个下单月份的订单在到期续租后买断、逾期、归还的订单数
# note 20241216 修改
# df_timeon_1.drop(columns=['检测中','续租中'],inplace = True)
df_timeon_1.drop(columns=['续租中'],inplace = True)
df_due = pd.merge(pd.merge(df_timeon_1,df_timeup_nxz_1,left_on="下单月份",right_on="下单月份",how="left"),df_timeup_xz_1,left_on="下单月份",right_on="下单月份",how="left")
df_due=df_due.fillna(0)
df_due["合计"]=df_due["All_x"]+df_due["All_y"]+df_due["All"]
#df_due["合计"]=df_due["合计"].astype("int")
df_due.drop(columns=["All_x","All_y","All"],axis=1,inplace=True)
df_due=df_due.astype("int")
df_due#[["提前买断","到期买断","续租后买断"]]#[14:]#[14:]

还款状态,提前买断,提前归还,租期逾期,未到首期还款日,检测中,还款中,到期买断,到期归还,到期逾期,到期归还检测中,待续租,已付总租金待归还,续租后买断,续租后归还,续租后逾期,续租归还检测中,续租中,合计
下单月份,,,,,,,,,,,,,,,,,,
2022-07,0,1,8,0,0,0,6,1,0,0,0,0,4,0,0,0,2,22
2022-08,2,1,21,0,0,0,7,2,7,0,0,0,11,1,2,1,1,56
2022-09,4,2,10,0,0,0,4,2,5,0,0,0,10,2,4,0,1,44
2022-10,1,1,8,0,0,0,8,2,1,1,0,0,11,3,3,0,2,41
2022-11,5,0,8,0,0,0,6,6,1,0,0,0,10,1,4,0,2,43
2022-12,8,2,16,0,0,0,13,2,6,0,0,0,13,0,2,0,4,66
2023-01,17,4,64,0,0,0,43,16,13,3,0,0,48,0,13,0,0,221
2023-02,26,5,71,0,0,1,89,43,30,1,0,0,51,0,15,0,0,332
2023-03,24,13,85,0,0,0,69,32,25,3,0,0,50,0,12,0,3,316


In [113]:
# （4）
#以下为分别统计每个下单月的订单在12期租期内租期逾期、提前买断、提前归还的的订单占比，
#以及每个下单月份的订单在12期租期到期后到期买断、到期归还、到期逾期、到期续租的订单占比，
#以及每个下单月份的订单在到期续租后买断、逾期、归还的订单占比
#该代码需先运行上述代码方不会报错
# applymap(lambda x:format(x,".2%"))
# df_due.divide(df_due["合计"],axis=0).applymap(lambda x:format(x,".2%"))#[["提前买断","到期买断","续租后买断"]]
df_due=df_due.divide(df_due["合计"],axis=0).applymap(lambda x:format(x,".2%"))#[["提前买断","到期买断","续租后买断"]]

df_due

还款状态,提前买断,提前归还,租期逾期,未到首期还款日,检测中,还款中,到期买断,到期归还,到期逾期,到期归还检测中,待续租,已付总租金待归还,续租后买断,续租后归还,续租后逾期,续租归还检测中,续租中,合计
下单月份,,,,,,,,,,,,,,,,,,
2022-07,0.00%,4.55%,36.36%,0.00%,0.00%,0.00%,27.27%,4.55%,0.00%,0.00%,0.00%,0.00%,18.18%,0.00%,0.00%,0.00%,9.09%,100.00%
2022-08,3.57%,1.79%,37.50%,0.00%,0.00%,0.00%,12.50%,3.57%,12.50%,0.00%,0.00%,0.00%,19.64%,1.79%,3.57%,1.79%,1.79%,100.00%
2022-09,9.09%,4.55%,22.73%,0.00%,0.00%,0.00%,9.09%,4.55%,11.36%,0.00%,0.00%,0.00%,22.73%,4.55%,9.09%,0.00%,2.27%,100.00%
2022-10,2.44%,2.44%,19.51%,0.00%,0.00%,0.00%,19.51%,4.88%,2.44%,2.44%,0.00%,0.00%,26.83%,7.32%,7.32%,0.00%,4.88%,100.00%
2022-11,11.63%,0.00%,18.60%,0.00%,0.00%,0.00%,13.95%,13.95%,2.33%,0.00%,0.00%,0.00%,23.26%,2.33%,9.30%,0.00%,4.65%,100.00%
2022-12,12.12%,3.03%,24.24%,0.00%,0.00%,0.00%,19.70%,3.03%,9.09%,0.00%,0.00%,0.00%,19.70%,0.00%,3.03%,0.00%,6.06%,100.00%
2023-01,7.69%,1.81%,28.96%,0.00%,0.00%,0.00%,19.46%,7.24%,5.88%,1.36%,0.00%,0.00%,21.72%,0.00%,5.88%,0.00%,0.00%,100.00%
2023-02,7.83%,1.51%,21.39%,0.00%,0.00%,0.30%,26.81%,12.95%,9.04%,0.30%,0.00%,0.00%,15.36%,0.00%,4.52%,0.00%,0.00%,100.00%
2023-03,7.59%,4.11%,26.90%,0.00%,0.00%,0.00%,21.84%,10.13%,7.91%,0.95%,0.00%,0.00%,15.82%,0.00%,3.80%,0.00%,0.95%,100.00%


In [114]:
# （5）以下为每月预收租金期数占比
dfzh3["预收期数"] = dfzh3["advance_periods"] 
df_ys=pd.crosstab(dfzh3["下单月份"],dfzh3["预收期数"],margins=True)
df_ys.divide(df_ys["All"],axis=0).applymap(lambda x:format(x,".2%"))

预收期数,0,1,2,3,4,5,6,7,8,9,10,12,All
下单月份,,,,,,,,,,,,,
2022-07,0.00%,13.64%,36.36%,45.45%,4.55%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%
2022-08,0.00%,1.79%,28.57%,48.21%,12.50%,5.36%,3.57%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%
2022-09,0.00%,0.00%,34.09%,45.45%,18.18%,2.27%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%
2022-10,2.44%,24.39%,41.46%,14.63%,12.20%,4.88%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%
2022-11,0.00%,11.63%,51.16%,16.28%,18.60%,2.33%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%
2022-12,0.00%,18.18%,42.42%,19.70%,18.18%,1.52%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,100.00%
2023-01,0.45%,16.29%,54.30%,19.00%,5.88%,2.26%,0.45%,0.45%,0.00%,0.00%,0.00%,0.90%,100.00%
2023-02,0.00%,5.72%,61.14%,17.47%,8.13%,4.82%,1.81%,0.00%,0.30%,0.30%,0.00%,0.30%,100.00%
2023-03,0.63%,15.51%,59.81%,10.76%,4.11%,3.16%,5.70%,0.00%,0.32%,0.00%,0.00%,0.00%,100.00%


In [115]:
# （6）平台套餐统计
dfzh3['平台套餐'] = dfzh3['platform_package']
dfzh3['租赁方案'] = np.where(dfzh3.平台套餐=='租完即送','租完即送','租完归还/可买断')
dfzh3['租赁方案'].value_counts()

zl_class_df = pd.crosstab(dfzh3['下单月份'],dfzh3['租赁方案'],margins=True)
zl_class_df.divide(zl_class_df["All"],axis=0).applymap(lambda x:format(x,".2%"))

租赁方案
租完归还/可买断    29199
租完即送         2558
Name: count, dtype: int64

租赁方案,租完即送,租完归还/可买断,All
下单月份,,,
2022-07,27.27%,72.73%,100.00%
2022-08,16.07%,83.93%,100.00%
2022-09,2.27%,97.73%,100.00%
2022-10,0.00%,100.00%,100.00%
2022-11,6.98%,93.02%,100.00%
2022-12,12.12%,87.88%,100.00%
2023-01,19.46%,80.54%,100.00%
2023-02,9.04%,90.96%,100.00%
2023-03,14.87%,85.13%,100.00%
